__Date:__ 01/27/2023
<br>__Author:__ Yaren Bilge Kaya
<br>__Project:__ Routing Strategies for Equitable Service Coordination

<br>__Department:__ Industrial Engineering
<br>__University:__ Northeastern University
<br>__Adviser:__ Dr. Kayse Lee Maass


In [1]:
#Libraries
import numpy as np
import pandas as pd
import random
from numpy import random
import statistics
from statistics import mean

from pandas import ExcelWriter
from pandas import ExcelFile

from openpyxl import load_workbook

First, we fill the shelters using the warm-up code

In [2]:
################################################
##   WARM-UP Fill the shelter within 20 days  ##
################################################
warmup_days = 20
#arrival_per_day = 16
warmup_youth_number_0 = 61
warmup_youth_number_1 = 195
warmup_youth_number_2 = 28
warmup_youth_number_3 = 30
#round(warmup_days*arrival_per_day)

df_warm_0 = pd.read_excel(r'C:\Users\yaren\Desktop\Dissertation\Routing Strategies\Data\Routing_warm-up.xlsx', sheet_name = 'Shelter_0')  
df_warm_0 = df_warm_0.iloc[:warmup_youth_number_0,1:]

df_warm_1 = pd.read_excel(r'C:\Users\yaren\Desktop\Dissertation\Routing Strategies\Data\Routing_warm-up.xlsx', sheet_name = 'Shelter_1')  
df_warm_1 = df_warm_1.iloc[:warmup_youth_number_1,1:]

df_warm_2 = pd.read_excel(r'C:\Users\yaren\Desktop\Dissertation\Routing Strategies\Data\Routing_warm-up.xlsx', sheet_name = 'Shelter_2')  
df_warm_2 = df_warm_2.iloc[:warmup_youth_number_2,1:]

df_warm_3 = pd.read_excel(r'C:\Users\yaren\Desktop\Dissertation\Routing Strategies\Data\Routing_warm-up.xlsx', sheet_name = 'Shelter_3')  
df_warm_3 = df_warm_3.iloc[:warmup_youth_number_3,1:]

__We create the youth and shelter profiles with code below. Shelters look at:__
- Youth's age
- Youth's gender
- Youth's sexual orientation
- Youth's child status 
- Youth's immigrant status
- Youth's HT victim status
<br> __There are services that shelter provides:__
- Bed
- Education
- Mental health
- Medical
<br> We assume that when youth receives a bed, they autimatically receive the other services that they requested. 

In [192]:
#set seed
np.random.seed(0)

#####################################
##   Creating youths and shelters  ##
#####################################

#Time frame
num_days = 360
arrival_per_day = 6
total_arrival = arrival_per_day*num_days

#Number of youths
Y= total_arrival

#Number of shelters (queues)
S= 4

#Get data from excel sheets for youth profiles
df_youth = pd.read_excel(r'C:\Users\yaren\Desktop\Dissertation\Routing Strategies\Data\10YouthProfiles.xlsx' ,
                         sheet_name = '10')
df_youth = df_youth.iloc[:Y,0:]

obj_y = {}
for y in range(Y): 
    #                         age                       female               male              trans-female       trans-male             genderqueer     non-binary        
    obj_y['youth'+str(y)] = [df_youth.iloc[y,0], df_youth.iloc[y,1], df_youth.iloc[y,2] , df_youth.iloc[y,3], df_youth.iloc[y,4], df_youth.iloc[y,5] , df_youth.iloc[y,6],
                             df_youth.iloc[y,7] , df_youth.iloc[y,8], df_youth.iloc[y,9], df_youth.iloc[y,10] , df_youth.iloc[y,11],
                            df_youth.iloc[y,12], df_youth.iloc[y,13] , df_youth.iloc[y,14]]

#Get data from excel sheets for shelter profiles
df_shelter = pd.read_excel(r'C:\Users\yaren\Desktop\Dissertation\Routing Strategies\Data\CrisisEmergency_Inputs.xlsx'  , sheet_name = 'ShelterClean')
df_shelter = df_shelter.iloc[:S,0:]

obj_s = {}
for s in range(S): 
    #                         age                       female               male              trans-female       trans-male             genderqueer     non-binary        
    obj_s['shelter'+str(s)] = [df_shelter.iloc[s,0], df_shelter.iloc[s,1], df_shelter.iloc[s,2] , df_shelter.iloc[s,3], df_shelter.iloc[s,4], df_shelter.iloc[s,5] , df_shelter.iloc[s,6],
                             df_shelter.iloc[s,7] , df_shelter.iloc[s,8], df_shelter.iloc[s,9], df_shelter.iloc[s,10] , df_shelter.iloc[s,11],
                            df_shelter.iloc[s,12], df_shelter.iloc[s,13] , df_shelter.iloc[s,14]]
    

#obj_s['shelter'+str(0)] = [24,1,1,1,1,1,1,1,1,1,1,1,1,1,1]


We pull youth needs, as well as the services provided in shelters.  

In [193]:
#Youth need profiles
#Get data from excel sheets for youth profiles
df_needs = pd.read_excel(r'C:\Users\yaren\Desktop\Dissertation\Routing Strategies\Data\10YouthProfiles.xlsx' ,
                         sheet_name = 'N10')
df_needs = df_needs.iloc[:Y,0:]

need_y= {}
for y in range(Y):    
    #                               Mental Health         Medical         Substance Abuse        Crisis           Long-Term
    need_y['youth_need'+str(y)] = [df_needs.iloc[y,0], df_needs.iloc[y,1], df_needs.iloc[y,2], df_needs.iloc[y,3], df_needs.iloc[y,4] , df_needs.iloc[y,5]
                                  , df_needs.iloc[y,6], df_needs.iloc[y,7] , df_needs.iloc[y,8] , df_needs.iloc[y,9] , df_needs.iloc[y,10] 
                                   , df_needs.iloc[y,11] , df_needs.iloc[y,12]]

#decides on whether youth leaves the system earlier or not
early_depart= {}
for y in range(Y): 
    early_depart[y]= random.randint(2)

#Services(13 of them through bed): Mental Health, Medical, Substance Abuse, Crisis 24 hour services, Long term housing
#Legal, Service coordination, Practical, Financial, Life Skills, Employment, Education, Childcare

#Shelter service profiles
df_services = pd.read_excel(r'C:\Users\yaren\Desktop\Dissertation\Routing Strategies\Data\CrisisEmergency_Inputs.xlsx' , sheet_name = 'ServicesClean')
df_services = df_services.iloc[:Y,0:]

services_s= {}
for s in range(S):    
    #                                         Mental Health         Medical         Substance Abuse        Crisis           Long-Term
    services_s['shelter_service'+str(s)] = [df_services.iloc[s,0], df_services.iloc[s,1],  df_services.iloc[s,2], df_services.iloc[s,3], df_services.iloc[s,4] , df_services.iloc[s,5]
                                  , df_services.iloc[s,6], df_services.iloc[s,7] , df_services.iloc[s,8] , df_services.iloc[s,9] , df_services.iloc[s,10] 
                                   , df_services.iloc[s,11] , df_services.iloc[s,12] ]
    
#Shelter beds
bed_s = [53, 164, 24, 26]


We combine the youth demographic information with the needs to keep track of youth arrivals. Then assign arrival times to each of them. Now the info and info_df include all information of youth arriving. 

In [194]:
###  Arrival Process ###

#Poisson, exponential inter-arrival times
arrival_time = np.cumsum(random.exponential(scale=1/total_arrival ,size=(total_arrival, 1)))*num_days + warmup_days

info=np.empty([total_arrival, 32])
for i in range(total_arrival):
    info[i,0]= i #youth number
    info[i,1]= obj_y['youth'+str(i)][0] #youth characteristics:age
    info[i,2]= obj_y['youth'+str(i)][1] #youth characteristics:gender-female
    info[i,3]= obj_y['youth'+str(i)][2] #youth characteristics:gender-male
    info[i,4]= obj_y['youth'+str(i)][3] #youth characteristics:gender-transfemale
    info[i,5]= obj_y['youth'+str(i)][4] #youth characteristics:gender-transmale
    info[i,6]= obj_y['youth'+str(i)][5] #youth characteristics:gender-queer
    info[i,7]= obj_y['youth'+str(i)][6] #youth characteristics:gender-nonbinary
    info[i,8]= obj_y['youth'+str(i)][7] #youth characteristics:SexOri-straight
    info[i,9]= obj_y['youth'+str(i)][8] #youth characteristics:SexOri-lesbian
    info[i,10]= obj_y['youth'+str(i)][9] #youth characteristics:SexOri-gay
    info[i,11]= obj_y['youth'+str(i)][10] #youth characteristics:SexOri-bisexual
    info[i,12]= obj_y['youth'+str(i)][11] #youth characteristics:SexOri-queer
    info[i,13]= obj_y['youth'+str(i)][12] #youth characteristics:Child
    info[i,14]= obj_y['youth'+str(i)][13] #youth characteristics:Immigrant
    info[i,15]= obj_y['youth'+str(i)][14] #youth characteristics:Child
    
    info[i,16]= need_y['youth_need'+str(i)][0] #youth needs:MentalHealth 
    info[i,17]= need_y['youth_need'+str(i)][1] #youth needs:Medical
    info[i,18]= need_y['youth_need'+str(i)][2] #youth needs:SubstanceAbuse
    info[i,19]= need_y['youth_need'+str(i)][3] #youth needs:Crisis 24 hour services
    info[i,20]= need_y['youth_need'+str(i)][4] #youth needs:Long term housing 
    info[i,21]= need_y['youth_need'+str(i)][5] #youth needs:Legal
    info[i,22]= need_y['youth_need'+str(i)][6] #youth needs:Service coordination
    info[i,23]= need_y['youth_need'+str(i)][7] #youth needs:Practical
    info[i,24]= need_y['youth_need'+str(i)][8] #youth needs:Financial
    info[i,25]= need_y['youth_need'+str(i)][9] #youth needs:Life Skills
    info[i,26]= need_y['youth_need'+str(i)][10] #youth needs:Employment
    info[i,27]= need_y['youth_need'+str(i)][11] #youth needs:Education
    info[i,28]= need_y['youth_need'+str(i)][12] #youth needs:Childcare
    info[i,29]= early_depart[i] #does youth have shorter LOS or not

    
    info[i,30]= arrival_time[i] #youth arrival time to system
    info[i,31]= random.randint(0, len(obj_s)) #which shelter they're going to

np.set_printoptions(precision=3)
info_df = pd.DataFrame(info, columns =['Youth No', 'Age',
            'Cis-gender Female', 'Cis-gender Male','Transgender Female','Transgender Male','Genderqueer','Non-binary'
            ,'Straight', 'Lesbian', 'Gay', 'Bisexual', 'Queer','Child', 'Immigrant', 'HT Victim', 
            'MentalHealth', 'Medical', 'SubstanceAbuse', 'Crisis',' Long-term', 'Legal', 'Service-coordination', 
            'Practical', 'Financial', 'Life Skills', 'Employment', 'Education', 'Childcare', 'Leave Early?',
                                       'Arrival Day', 'Shelter No'])


We first look at the listof shelters that accept youth. This array is called the __accepted_shelter_subset__ and exists for each youth. If this list is all 0's, means that no shelter accepts that youth due to their demographic. 

In [195]:
accepted_shelter_subset= np.zeros([len(info), 4])

i=0
while i< len(info):
    if info[i][1] <= obj_s['shelter'+str(0)][0] and info[i][2] <= obj_s['shelter'+str(0)][1] and info[i][3] <= obj_s['shelter'+str(0)][2] and info[i][4] <= obj_s['shelter'+str(0)][3]and info[i][5] <= obj_s['shelter'+str(0)][4]and info[i][6] <= obj_s['shelter'+str(0)][5]and info[i][7] <= obj_s['shelter'+str(0)][6] and info[i][8] <= obj_s['shelter'+str(0)][7] and info[i][9] <= obj_s['shelter'+str(0)][8] and info[i][10] <= obj_s['shelter'+str(0)][9]and info[i][11] <= obj_s['shelter'+str(0)][10] and info[i][12] <= obj_s['shelter'+str(0)][11]and info[i][13] <= obj_s['shelter'+str(0)][12] and info[i][14] <= obj_s['shelter'+str(0)][13]and info[i][15] <= obj_s['shelter'+str(0)][14]:
        accepted_shelter_subset[i,0]=1

    if info[i][1] <= obj_s['shelter'+str(1)][0] and info[i][2] <= obj_s['shelter'+str(1)][1] and info[i][3] <= obj_s['shelter'+str(1)][2] and info[i][4] <= obj_s['shelter'+str(1)][3]and info[i][5] <= obj_s['shelter'+str(1)][4]and info[i][6] <= obj_s['shelter'+str(1)][5]and info[i][7] <= obj_s['shelter'+str(1)][6] and info[i][8] <= obj_s['shelter'+str(1)][7] and info[i][9] <= obj_s['shelter'+str(1)][8] and info[i][10] <= obj_s['shelter'+str(1)][9]and info[i][11] <= obj_s['shelter'+str(1)][10] and info[i][12] <= obj_s['shelter'+str(1)][11]  and info[i][13] <= obj_s['shelter'+str(1)][12] and info[i][14] <= obj_s['shelter'+str(1)][13] and info[i][15] <= obj_s['shelter'+str(1)][14]: 
        accepted_shelter_subset[i,1]=1

    if info[i][1] <= obj_s['shelter'+str(2)][0] and info[i][2] <= obj_s['shelter'+str(2)][1] and info[i][3] <= obj_s['shelter'+str(2)][2] and info[i][4] <= obj_s['shelter'+str(2)][3]and info[i][5] <= obj_s['shelter'+str(2)][4]and info[i][6] <= obj_s['shelter'+str(2)][5]and info[i][7] <= obj_s['shelter'+str(2)][6] and info[i][8] <= obj_s['shelter'+str(2)][7] and info[i][9] <= obj_s['shelter'+str(2)][8] and info[i][10] <= obj_s['shelter'+str(2)][9]and info[i][11] <= obj_s['shelter'+str(2)][10] and info[i][12] <= obj_s['shelter'+str(2)][11] and info[i][13] <= obj_s['shelter'+str(2)][12] and info[i][14] <= obj_s['shelter'+str(2)][13] and info[i][15] <= obj_s['shelter'+str(2)][14]: 
        accepted_shelter_subset[i,2]=1

    if info[i][1] <= obj_s['shelter'+str(3)][0] and info[i][2] <= obj_s['shelter'+str(3)][1] and info[i][3] <= obj_s['shelter'+str(3)][2] and info[i][4] <= obj_s['shelter'+str(3)][3]and info[i][5] <= obj_s['shelter'+str(3)][4]and info[i][6] <= obj_s['shelter'+str(3)][5]and info[i][7] <= obj_s['shelter'+str(3)][6] and info[i][8] <= obj_s['shelter'+str(3)][7] and info[i][9] <= obj_s['shelter'+str(3)][8] and info[i][10] <= obj_s['shelter'+str(3)][9]and info[i][11] <= obj_s['shelter'+str(3)][10] and info[i][12] <= obj_s['shelter'+str(3)][11] and info[i][13] <= obj_s['shelter'+str(3)][12] and info[i][14] <= obj_s['shelter'+str(3)][13] and info[i][15] <= obj_s['shelter'+str(3)][14]: 
        accepted_shelter_subset[i,3]=1
        
    i=i+1

#the shelters youth would go after abandoning due to demograohic for once
accepted_shelter_subset =np.multiply([1,2,3,4], accepted_shelter_subset)


Now we remove youth who cannot enter the system due to demographic mismatch, and add them to __demog_unaccepted__ list. Rest of the there arrivals are added to a new list called __arrival_stream__.  

In [196]:
demog_unaccepted = np.empty([0, 32])
unaccepted_index = []
    
#figuring out which shelter youth is going to go among the acceptable shelters
for i in range(len(info)):
    if np.all((accepted_shelter_subset[i] == 0)):
        tmp = info[i]
        demog_unaccepted = np.append(demog_unaccepted, [tmp], axis=0)
        unaccepted_index.append(i)

demog_unaccepted_list = demog_unaccepted.tolist()
info_list = info.tolist()
accepted_shelter_subset = np.delete(accepted_shelter_subset, unaccepted_index, axis=0)

arrival_stream =[i for i in info_list if i not in demog_unaccepted_list]

For our analysis let's look at how many youth can enter each shelter, considering the demographics served. 

In [197]:
number_can_enter_shelter_0 = 0
number_can_enter_shelter_1 = 0
number_can_enter_shelter_2 = 0
number_can_enter_shelter_3 = 0

for i in range(len(arrival_stream)):
    if 0 in accepted_shelter_subset[i][np.nonzero(accepted_shelter_subset[i])]-1:
        number_can_enter_shelter_0 = number_can_enter_shelter_0 +1
    if 1 in accepted_shelter_subset[i][np.nonzero(accepted_shelter_subset[i])]-1:
        number_can_enter_shelter_1 = number_can_enter_shelter_1 +1
    if 2 in accepted_shelter_subset[i][np.nonzero(accepted_shelter_subset[i])]-1:
        number_can_enter_shelter_2 = number_can_enter_shelter_2 +1
    if 3 in accepted_shelter_subset[i][np.nonzero(accepted_shelter_subset[i])]-1:
        number_can_enter_shelter_3 = number_can_enter_shelter_3 +1
        
print('Number of youth that can receive services from shelter 0 is: ' +str(number_can_enter_shelter_0))
print('Number of youth that can receive services from shelter 1 is: ' +str(number_can_enter_shelter_1))
print('Number of youth that can receive services from shelter 2 is: ' +str(number_can_enter_shelter_2))
print('Number of youth that can receive services from shelter 3 is: ' +str(number_can_enter_shelter_3))       
    

Number of youth that can receive services from shelter 0 is: 480
Number of youth that can receive services from shelter 1 is: 1916
Number of youth that can receive services from shelter 2 is: 1916
Number of youth that can receive services from shelter 3 is: 1850


Now among the shelters accept youth, following the __Most Needs Served__ routing strategy, we find which shelter to direct youth so that the shelter fulfills most of their needs.  

In [198]:
#figuring out which shelter youth is going to go among the acceptable shelters
youth_destination = []
for i in range(len(arrival_stream)): #mismatches are eliminated
    num_needs_fulfilled =[]
    
    for j in (accepted_shelter_subset[i][np.nonzero(accepted_shelter_subset[i])]-1): #we look at all the shelters that accept them
        j= int(j)
        match_tmp = [sum(k) for k in zip( need_y['youth_need'+str(i)], services_s['shelter_service'+str(j)] )] #we sum up youth needs and services offered to see how they match 
        num_needs_fulfilled.append(match_tmp.count(2)) #count the number of 2's (service+need ==2 means service is provided and needed)
        
    # max_value = max(num_needs_fulfilled) #find which shelter provides most of their needs
    index = random.choice([i for i, x in enumerate(num_needs_fulfilled) if x == max(num_needs_fulfilled)]) #find which shelter provides most of their needs, if there are multiple max, pick randomly
    youth_destination.append((accepted_shelter_subset[i][np.nonzero(accepted_shelter_subset[i])]-1)[index])
            

Add youth's destination to their arrival stream information. If the shelter youth arrived in the first place and the best matching shelter we found according to __Most Needs Served__ strategy are different, add couple hours to their original arrival time to make it more realistic. 

In [199]:
youth_destination = np.reshape(youth_destination, (-1, 1))
arrival_stream = np.hstack((arrival_stream, youth_destination))

for i in range(len(arrival_stream)):
    if arrival_stream[i][31] != arrival_stream[i][32]:
        arrival_stream[i][30]+ random.triangular(0, 2, 7, size=None) #this number can be changed later

Now we direct youth to shelters that they are supposed to go according to __Most Needs Served__ strategy. 

In [200]:
#empty lists of arrivals
arrivals_to_0 = np.empty([0, 33])
arrivals_to_1 = np.empty([0, 33])
arrivals_to_2 = np.empty([0, 33])
arrivals_to_3 = np.empty([0, 33])

#figuring out which shelter youth is going to go among the acceptable shelters
for i in range(len(arrival_stream)):
    if arrival_stream[i][32] ==0 :
        tmp = arrival_stream[i]
        arrivals_to_0 = np.append(arrivals_to_0, [tmp], axis=0)
    if arrival_stream[i][32] ==1 :
        tmp = arrival_stream[i]
        arrivals_to_1 = np.append(arrivals_to_1, [tmp], axis=0)
    if arrival_stream[i][32] ==2 :
        tmp = arrival_stream[i]
        arrivals_to_2 = np.append(arrivals_to_2, [tmp], axis=0)
    if arrival_stream[i][32] ==3 :
        tmp = arrival_stream[i]
        arrivals_to_3 = np.append(arrivals_to_3, [tmp], axis=0)
            
#update arrivals to shelters (demographic mismatch abandonments are added)

#shelter 0
arrivals_to_0 = np.delete(arrivals_to_0, [31,32],1)
arrivals_to_0 = arrivals_to_0[np.argsort(arrivals_to_0[:, 30])]

#shelter 1
arrivals_to_1 = np.delete(arrivals_to_1, [31,32],1)
arrivals_to_1 = arrivals_to_1[np.argsort(arrivals_to_1[:, 30])]

#shelter 2
arrivals_to_2 = np.delete(arrivals_to_2, [31,32],1)
arrivals_to_2 = arrivals_to_2[np.argsort(arrivals_to_2[:, 30])]

#shelter 3
arrivals_to_3 = np.delete(arrivals_to_3, [31,32],1)
arrivals_to_3 = arrivals_to_3[np.argsort(arrivals_to_3[:, 30])]


In [201]:
directed_to_0 = len(arrivals_to_0)
directed_to_1 = len(arrivals_to_1)
directed_to_2 = len(arrivals_to_2)
directed_to_3 = len(arrivals_to_3)

print('Total number of youth arriving to shelter 0 is: ' +str(len(arrivals_to_0)))
print('Total number of youth arriving to shelter 1 is: ' +str(len(arrivals_to_1)))
print('Total number of youth arriving to shelter 2 is: ' +str(len(arrivals_to_2)))
print('Total number of youth arriving to shelter 3 is: ' +str(len(arrivals_to_3)))

Total number of youth arriving to shelter 0 is: 47
Total number of youth arriving to shelter 1 is: 1139
Total number of youth arriving to shelter 2 is: 428
Total number of youth arriving to shelter 3 is: 523


In [202]:
# Set service parameters for each of the shelters #

# Service time is :length of stay (exponential)
avg_service_time_0= 60
avg_service_time_1= 60
avg_service_time_2= 60
avg_service_time_3= 60

stdev_service_time_0 = 5
stdev_service_time_1 = 5
stdev_service_time_2 = 5
stdev_service_time_3 = 5

# Patience time (exponential)
avg_patience = 5
stdev_patience = 2

After figuring out the routes for each youth, we start the service and queue modeling starts

In [203]:
# Shelter 0 #

#Define 3 lists that are going to help us keep track of the serviced youth, waiting youth, and unserviced youth
serviced_list_0 = []
waiting_list_0 = []
non_serviced_list_0 =[]

# Initialize the code by turning df_warm into df_serviced
serviced_list_0 = df_warm_0.values.tolist()

#for i in range(warmup_youth_number, warmup_youth_number+Y):
i= warmup_youth_number_0 - 1 # to keep track of serviced list 
j= 0                   # to keep track of arriving youth
    
while len(arrivals_to_0)>0:  
    #if number of beds open in the system is larger than 46, all youth can enter the system with no prioritizing
        
    arrivals_no_priority = []
    for k in range(len(arrivals_to_0)):
        arrivals_no_priority.append(arrivals_to_0[k]) 
        
    #characteristics of arriving youth
    number = arrivals_no_priority[j][0]
    age= arrivals_no_priority[j][1]
    female = arrivals_no_priority[j][2]
    male = arrivals_no_priority[j][3]
    trans_female = arrivals_no_priority[j][4]
    trans_male = arrivals_no_priority[j][5]
    g_queer = arrivals_no_priority[j][6]
    nonbinary = arrivals_no_priority[j][7]
    straight = arrivals_no_priority[j][8]
    lesbian= arrivals_no_priority[j][9]
    gay= arrivals_no_priority[j][10]
    bisexual= arrivals_no_priority[j][11]
    queer= arrivals_no_priority[j][12]
    child= arrivals_no_priority[j][13]
    immigrant = arrivals_no_priority[j][14]
    ht_victim = arrivals_no_priority[j][15]
    
    #serviced requested by youth
    mentalhealth = arrivals_no_priority[j][16]
    medical= arrivals_no_priority[j][17]
    substance_abuse= arrivals_no_priority[j][18]
    crisis = arrivals_no_priority[j][19]
    long_term = arrivals_no_priority[j][20]
    legal = arrivals_no_priority[j][21]
    service_coor = arrivals_no_priority[j][22]
    practical = arrivals_no_priority[j][23]
    financial = arrivals_no_priority[j][24]
    life_skills= arrivals_no_priority[j][25]
    employment= arrivals_no_priority[j][26]
    education = arrivals_no_priority[j][27]
    childcare = arrivals_no_priority[j][28]
    early = arrivals_no_priority[j][29]    

    #times of arriving youth
    arrival_time = arrivals_no_priority[j][30]
    service_time = random.exponential(scale=avg_service_time_0, size=1)[0]         #length of stay of youth
    patience_time = random.exponential(scale=avg_patience, size=1)[0]        #patience time            
        
    #find the number of youth in the system
    num_in_system_arrivals = []
    for k in range(0,i):
        num_in_system_arrivals.append(serviced_list_0[k][30])

    num_in_system_departures = []
    for k in range(0,i):
        num_in_system_departures.append(serviced_list_0[k][34])
            
    #number in system is equal to number of people who arrived till then minus the number of people who left the system
    #note that i include the youth arriving to the system as well (+1)
    number_in_system = sum(num_in_system_arrivals[0:i]<=arrival_time)- sum(num_in_system_departures[0:i]<=arrival_time)+1              #number of youths in system at the arrival
        
    tmp = sorted(num_in_system_departures)
    #if arrival is later than the first possible departure, service should start at the arrival time
    if arrival_time > tmp[len(serviced_list_0)- bed_s[0]]:
        start_service_time = arrival_time                                  #time youth starts service

    #if arrival is before the departure, service should start at the departure time
    else:
        start_service_time = tmp[len(serviced_list_0)- bed_s[0]]              #time youth starts service

    departure_time = start_service_time + service_time                #time youth leaves system
    wait_time = start_service_time -arrival_time                      #amount of time youth waits                                                         
    num_open_beds = max(bed_s[0] - number_in_system,0)                     #number of open beds
    utilization = (1-num_open_beds/bed_s[0])                               #utilization during the time youth is staying
    
    if wait_time >= patience_time:
        #create a list for the un-served youth
        actual_wait_time= patience_time                                #updated wait time based on abandonment
        received_service = 0                                           #binary variable that shows whether youth received service or not
          
        non_serviced_stacked = [number, age, female, male, trans_female, trans_male, g_queer, nonbinary, straight, lesbian, gay, bisexual, queer, child, immigrant, ht_victim, mentalhealth, medical, substance_abuse, crisis, long_term, legal, service_coor, practical, financial, life_skills, employment, education, childcare, early
                        ,arrival_time, service_time, patience_time, 
                        start_service_time, departure_time, wait_time, number_in_system , actual_wait_time, received_service, num_open_beds, utilization]
        non_serviced_list_0.append(non_serviced_stacked)
  
        #now we have to delete the youth that has been appended to non-serviced list from the arriving list
        for k in range(len(arrivals_to_0)):
            if arrivals_to_0[k][0] == number:
                arrivals_to_0 = np.delete(arrivals_to_0, k, 0)
                break
            
    else:
        actual_wait_time= wait_time                                     #updated wait time based on abandonment
        received_service = 1                                            #binary variable that shows whether youth received service or not
        #create a list for the serviced youths
        serviced_stacked = [number, age, female, male, trans_female, trans_male, g_queer, nonbinary, straight, lesbian, gay, bisexual, queer, child, immigrant, ht_victim, mentalhealth, medical, substance_abuse, crisis, long_term, legal, service_coor, practical, financial, life_skills, employment, education, childcare, early
                        ,arrival_time, service_time, patience_time, 
                        start_service_time, departure_time, wait_time, number_in_system , actual_wait_time, received_service, num_open_beds, utilization]
        serviced_list_0.append(serviced_stacked)
            
        #now we have to delete the youth that has been appended to serviced list from the arriving list
        for k in range(len(arrivals_to_0)):
            if arrivals_to_0[k][0] == number:
                arrivals_to_0 = np.delete(arrivals_to_0, k, 0)
                break
                    
        #if a youth received service, increase i by 1
        i= i+1
        
#Turn serviced and non-serviced lists into pandas data frames
serviced_df_0 = pd.DataFrame(serviced_list_0, columns =['number', 'age',' female', 'male', 'trans_female', 'trans_male', 'g_queer', 'nonbinary', 'straight', 'lesbian', 'gay', 
                                                        'bisexual', 'queer', 'child', 'immigrant', 'ht_victim', 'mentalhealth', 'medical', 'substance_abuse', 'crisis', 'long_term', 'legal', 
                                                        'service_coor', 'practical', 'financial', 'life_skills', 'employment', 'education', 'childcare', 'early'
                                                        ,'arrival time', 'service time', 'patience time', 
                                                        'start service time', 'departure time', 'wait time', 'number in system' , 'actual wait time', 'received service', 'num opennbeds', 'utilization']) 


non_serviced_df_0 = pd.DataFrame(non_serviced_list_0, columns =['number', 'age',' female', 'male', 'trans_female', 'trans_male', 'g_queer', 'nonbinary', 'straight', 'lesbian', 'gay', 
                                                        'bisexual', 'queer', 'child', 'immigrant', 'ht_victim', 'mentalhealth', 'medical', 'substance_abuse', 'crisis', 'long_term', 'legal', 
                                                        'service_coor', 'practical', 'financial', 'life_skills', 'employment', 'education', 'childcare', 'early'
                                                        ,'arrival time', 'service time', 'patience time', 
                                                        'start service time', 'departure time', 'wait time', 'number in system' , 'actual wait time', 'received service', 'num opennbeds', 'utilization']) 


In [204]:
# Shelter 1 #

#Define 3 lists that are going to help us keep track of the serviced youth, waiting youth, and unserviced youth
serviced_list_1 = []
waiting_list_1 = []
non_serviced_list_1 =[]

# Initialize the code by turning df_warm into df_serviced
serviced_list_1 = df_warm_1.values.tolist()

#for i in range(warmup_youth_number, warmup_youth_number+Y):
i= warmup_youth_number_1 - 1 # to keep track of serviced list 
j= 0                   # to keep track of arriving youth
    
while len(arrivals_to_1)>0:  
    #if number of beds open in the system is larger than 46, all youth can enter the system with no prioritizing
        
    arrivals_no_priority = []
    for k in range(len(arrivals_to_1)):
        arrivals_no_priority.append(arrivals_to_1[k]) 
        
    #characteristics of arriving youth
    number = arrivals_no_priority[j][0]
    age= arrivals_no_priority[j][1]
    female = arrivals_no_priority[j][2]
    male = arrivals_no_priority[j][3]
    trans_female = arrivals_no_priority[j][4]
    trans_male = arrivals_no_priority[j][5]
    g_queer = arrivals_no_priority[j][6]
    nonbinary = arrivals_no_priority[j][7]
    straight = arrivals_no_priority[j][8]
    lesbian= arrivals_no_priority[j][9]
    gay= arrivals_no_priority[j][10]
    bisexual= arrivals_no_priority[j][11]
    queer= arrivals_no_priority[j][12]
    child= arrivals_no_priority[j][13]
    immigrant = arrivals_no_priority[j][14]
    ht_victim = arrivals_no_priority[j][15]
    
    #serviced requested by youth
    mentalhealth = arrivals_no_priority[j][16]
    medical= arrivals_no_priority[j][17]
    substance_abuse= arrivals_no_priority[j][18]
    crisis = arrivals_no_priority[j][19]
    long_term = arrivals_no_priority[j][20]
    legal = arrivals_no_priority[j][21]
    service_coor = arrivals_no_priority[j][22]
    practical = arrivals_no_priority[j][23]
    financial = arrivals_no_priority[j][24]
    life_skills= arrivals_no_priority[j][25]
    employment= arrivals_no_priority[j][26]
    education = arrivals_no_priority[j][27]
    childcare = arrivals_no_priority[j][28]
    early = arrivals_no_priority[j][29]    

    #times of arriving youth
    arrival_time = arrivals_no_priority[j][30]
    service_time = random.exponential(scale=avg_service_time_1, size=1)[0]         #length of stay of youth
    patience_time = random.exponential(scale=avg_patience, size=1)[0]        #patience time            
        
    #find the number of youth in the system
    num_in_system_arrivals = []
    for k in range(0,i):
        num_in_system_arrivals.append(serviced_list_1[k][30])

    num_in_system_departures = []
    for k in range(0,i):
        num_in_system_departures.append(serviced_list_1[k][34])
            
    #number in system is equal to number of people who arrived till then minus the number of people who left the system
    #note that i include the youth arriving to the system as well (+1)
    number_in_system = sum(num_in_system_arrivals[0:i]<=arrival_time)- sum(num_in_system_departures[0:i]<=arrival_time)+1              #number of youths in system at the arrival
        
    tmp = sorted(num_in_system_departures)
    #if arrival is later than the first possible departure, service should start at the arrival time
    if arrival_time > tmp[len(serviced_list_1)- bed_s[1]]:
        start_service_time = arrival_time                                  #time youth starts service

    #if arrival is before the departure, service should start at the departure time
    else:
        start_service_time = tmp[len(serviced_list_1)- bed_s[1]]              #time youth starts service

    departure_time = start_service_time + service_time                #time youth leaves system
    wait_time = start_service_time -arrival_time                      #amount of time youth waits                                                         
    num_open_beds = max(bed_s[1] - number_in_system,0)                     #number of open beds
    utilization = (1-num_open_beds/bed_s[1])                               #utilization during the time youth is staying
    
    if wait_time >= patience_time:
        #create a list for the un-served youth
        actual_wait_time= patience_time                                #updated wait time based on abandonment
        received_service = 0                                           #binary variable that shows whether youth received service or not
          
        non_serviced_stacked = [number, age, female, male, trans_female, trans_male, g_queer, nonbinary, straight, lesbian, gay, bisexual, queer, child, immigrant, ht_victim, mentalhealth, medical, substance_abuse, crisis, long_term, legal, service_coor, practical, financial, life_skills, employment, education, childcare, early
                        ,arrival_time, service_time, patience_time, 
                        start_service_time, departure_time, wait_time, number_in_system , actual_wait_time, received_service, num_open_beds, utilization]
        non_serviced_list_1.append(non_serviced_stacked)
  
        #now we have to delete the youth that has been appended to non-serviced list from the arriving list
        for k in range(len(arrivals_to_1)):
            if arrivals_to_1[k][0] == number:
                arrivals_to_1 = np.delete(arrivals_to_1, k, 0)
                break
            
    else:
        actual_wait_time= wait_time                                     #updated wait time based on abandonment
        received_service = 1                                            #binary variable that shows whether youth received service or not
        #create a list for the serviced youths
        serviced_stacked = [number, age, female, male, trans_female, trans_male, g_queer, nonbinary, straight, lesbian, gay, bisexual, queer, child, immigrant, ht_victim, mentalhealth, medical, substance_abuse, crisis, long_term, legal, service_coor, practical, financial, life_skills, employment, education, childcare, early
                        ,arrival_time, service_time, patience_time, 
                        start_service_time, departure_time, wait_time, number_in_system , actual_wait_time, received_service, num_open_beds, utilization]
        serviced_list_1.append(serviced_stacked)
            
        #now we have to delete the youth that has been appended to serviced list from the arriving list
        for k in range(len(arrivals_to_1)):
            if arrivals_to_1[k][0] == number:
                arrivals_to_1 = np.delete(arrivals_to_1, k, 0)
                break
                    
        #if a youth received service, increase i by 1
        i= i+1
        
#Turn serviced and non-serviced lists into pandas data frames
serviced_df_1 = pd.DataFrame(serviced_list_1, columns =['number', 'age',' female', 'male', 'trans_female', 'trans_male', 'g_queer', 'nonbinary', 'straight', 'lesbian', 'gay', 
                                                        'bisexual', 'queer', 'child', 'immigrant', 'ht_victim', 'mentalhealth', 'medical', 'substance_abuse', 'crisis', 'long_term', 'legal', 
                                                        'service_coor', 'practical', 'financial', 'life_skills', 'employment', 'education', 'childcare', 'early'
                                                        ,'arrival time', 'service time', 'patience time', 
                                                        'start service time', 'departure time', 'wait time', 'number in system' , 'actual wait time', 'received service', 'num opennbeds', 'utilization']) 


non_serviced_df_1 = pd.DataFrame(non_serviced_list_1, columns =['number', 'age',' female', 'male', 'trans_female', 'trans_male', 'g_queer', 'nonbinary', 'straight', 'lesbian', 'gay', 
                                                        'bisexual', 'queer', 'child', 'immigrant', 'ht_victim', 'mentalhealth', 'medical', 'substance_abuse', 'crisis', 'long_term', 'legal', 
                                                        'service_coor', 'practical', 'financial', 'life_skills', 'employment', 'education', 'childcare', 'early'
                                                        ,'arrival time', 'service time', 'patience time', 
                                                        'start service time', 'departure time', 'wait time', 'number in system' , 'actual wait time', 'received service', 'num opennbeds', 'utilization']) 


In [205]:
# Shelter 2 #

#Define 3 lists that are going to help us keep track of the serviced youth, waiting youth, and unserviced youth
serviced_list_2 = []
waiting_list_2 = []
non_serviced_list_2 =[]

# Initialize the code by turning df_warm into df_serviced
serviced_list_2 = df_warm_2.values.tolist()

#for i in range(warmup_youth_number, warmup_youth_number+Y):
i= warmup_youth_number_2 - 1 # to keep track of serviced list 
j= 0                   # to keep track of arriving youth
    
while len(arrivals_to_2)>0:  
    #if number of beds open in the system is larger than 46, all youth can enter the system with no prioritizing
        
    arrivals_no_priority = []
    for k in range(len(arrivals_to_2)):
        arrivals_no_priority.append(arrivals_to_2[k]) 
        
    #characteristics of arriving youth
    number = arrivals_no_priority[j][0]
    age= arrivals_no_priority[j][1]
    female = arrivals_no_priority[j][2]
    male = arrivals_no_priority[j][3]
    trans_female = arrivals_no_priority[j][4]
    trans_male = arrivals_no_priority[j][5]
    g_queer = arrivals_no_priority[j][6]
    nonbinary = arrivals_no_priority[j][7]
    straight = arrivals_no_priority[j][8]
    lesbian= arrivals_no_priority[j][9]
    gay= arrivals_no_priority[j][10]
    bisexual= arrivals_no_priority[j][11]
    queer= arrivals_no_priority[j][12]
    child= arrivals_no_priority[j][13]
    immigrant = arrivals_no_priority[j][14]
    ht_victim = arrivals_no_priority[j][15]
    
    #serviced requested by youth
    mentalhealth = arrivals_no_priority[j][16]
    medical= arrivals_no_priority[j][17]
    substance_abuse= arrivals_no_priority[j][18]
    crisis = arrivals_no_priority[j][19]
    long_term = arrivals_no_priority[j][20]
    legal = arrivals_no_priority[j][21]
    service_coor = arrivals_no_priority[j][22]
    practical = arrivals_no_priority[j][23]
    financial = arrivals_no_priority[j][24]
    life_skills= arrivals_no_priority[j][25]
    employment= arrivals_no_priority[j][26]
    education = arrivals_no_priority[j][27]
    childcare = arrivals_no_priority[j][28]
    early = arrivals_no_priority[j][29]    

    #times of arriving youth
    arrival_time = arrivals_no_priority[j][30]
    service_time = random.exponential(scale=avg_service_time_2, size=1)[0]         #length of stay of youth
    patience_time = random.exponential(scale=avg_patience, size=1)[0]        #patience time            
        
    #find the number of youth in the system
    num_in_system_arrivals = []
    for k in range(0,i):
        num_in_system_arrivals.append(serviced_list_2[k][30])

    num_in_system_departures = []
    for k in range(0,i):
        num_in_system_departures.append(serviced_list_2[k][34])
            
    #number in system is equal to number of people who arrived till then minus the number of people who left the system
    #note that i include the youth arriving to the system as well (+1)
    number_in_system = sum(num_in_system_arrivals[0:i]<=arrival_time)- sum(num_in_system_departures[0:i]<=arrival_time)+1              #number of youths in system at the arrival
        
    tmp = sorted(num_in_system_departures)
    #if arrival is later than the first possible departure, service should start at the arrival time
    if arrival_time > tmp[len(serviced_list_2)- bed_s[2]]:
        start_service_time = arrival_time                                  #time youth starts service

    #if arrival is before the departure, service should start at the departure time
    else:
        start_service_time = tmp[len(serviced_list_2)- bed_s[2]]              #time youth starts service

    departure_time = start_service_time + service_time                #time youth leaves system
    wait_time = start_service_time -arrival_time                      #amount of time youth waits                                                         
    num_open_beds = max(bed_s[2] - number_in_system,0)                     #number of open beds
    utilization = (1-num_open_beds/bed_s[2])                               #utilization during the time youth is staying
    
    if wait_time >= patience_time:
        #create a list for the un-served youth
        actual_wait_time= patience_time                                #updated wait time based on abandonment
        received_service = 0                                           #binary variable that shows whether youth received service or not
          
        non_serviced_stacked = [number, age, female, male, trans_female, trans_male, g_queer, nonbinary, straight, lesbian, gay, bisexual, queer, child, immigrant, ht_victim, mentalhealth, medical, substance_abuse, crisis, long_term, legal, service_coor, practical, financial, life_skills, employment, education, childcare, early
                        ,arrival_time, service_time, patience_time, 
                        start_service_time, departure_time, wait_time, number_in_system , actual_wait_time, received_service, num_open_beds, utilization]
        non_serviced_list_2.append(non_serviced_stacked)
  
        #now we have to delete the youth that has been appended to non-serviced list from the arriving list
        for k in range(len(arrivals_to_2)):
            if arrivals_to_2[k][0] == number:
                arrivals_to_2 = np.delete(arrivals_to_2, k, 0)
                break
            
    else:
        actual_wait_time= wait_time                                     #updated wait time based on abandonment
        received_service = 1                                            #binary variable that shows whether youth received service or not
        #create a list for the serviced youths
        serviced_stacked = [number, age, female, male, trans_female, trans_male, g_queer, nonbinary, straight, lesbian, gay, bisexual, queer, child, immigrant, ht_victim, mentalhealth, medical, substance_abuse, crisis, long_term, legal, service_coor, practical, financial, life_skills, employment, education, childcare, early
                        ,arrival_time, service_time, patience_time, 
                        start_service_time, departure_time, wait_time, number_in_system , actual_wait_time, received_service, num_open_beds, utilization]
        serviced_list_2.append(serviced_stacked)
            
        #now we have to delete the youth that has been appended to serviced list from the arriving list
        for k in range(len(arrivals_to_2)):
            if arrivals_to_2[k][0] == number:
                arrivals_to_2 = np.delete(arrivals_to_2, k, 0)
                break
                    
        #if a youth received service, increase i by 1
        i= i+1
        
#Turn serviced and non-serviced lists into pandas data frames
serviced_df_2 = pd.DataFrame(serviced_list_2, columns =['number', 'age',' female', 'male', 'trans_female', 'trans_male', 'g_queer', 'nonbinary', 'straight', 'lesbian', 'gay', 
                                                        'bisexual', 'queer', 'child', 'immigrant', 'ht_victim', 'mentalhealth', 'medical', 'substance_abuse', 'crisis', 'long_term', 'legal', 
                                                        'service_coor', 'practical', 'financial', 'life_skills', 'employment', 'education', 'childcare', 'early'
                                                        ,'arrival time', 'service time', 'patience time', 
                                                        'start service time', 'departure time', 'wait time', 'number in system' , 'actual wait time', 'received service', 'num opennbeds', 'utilization']) 


non_serviced_df_2 = pd.DataFrame(non_serviced_list_2, columns =['number', 'age',' female', 'male', 'trans_female', 'trans_male', 'g_queer', 'nonbinary', 'straight', 'lesbian', 'gay', 
                                                        'bisexual', 'queer', 'child', 'immigrant', 'ht_victim', 'mentalhealth', 'medical', 'substance_abuse', 'crisis', 'long_term', 'legal', 
                                                        'service_coor', 'practical', 'financial', 'life_skills', 'employment', 'education', 'childcare', 'early'
                                                        ,'arrival time', 'service time', 'patience time', 
                                                        'start service time', 'departure time', 'wait time', 'number in system' , 'actual wait time', 'received service', 'num opennbeds', 'utilization']) 


In [206]:
# Shelter 3 #

#Define 3 lists that are going to help us keep track of the serviced youth, waiting youth, and unserviced youth
serviced_list_3 = []
waiting_list_3 = []
non_serviced_list_3 =[]

# Initialize the code by turning df_warm into df_serviced
serviced_list_3 = df_warm_3.values.tolist()

#for i in range(warmup_youth_number, warmup_youth_number+Y):
i= warmup_youth_number_3 - 1 # to keep track of serviced list 
j= 0                   # to keep track of arriving youth
    
while len(arrivals_to_3)>0:  
    #if number of beds open in the system is larger than 46, all youth can enter the system with no prioritizing
        
    arrivals_no_priority = []
    for k in range(len(arrivals_to_3)):
        arrivals_no_priority.append(arrivals_to_3[k]) 
        
    #characteristics of arriving youth
    number = arrivals_no_priority[j][0]
    age= arrivals_no_priority[j][1]
    female = arrivals_no_priority[j][2]
    male = arrivals_no_priority[j][3]
    trans_female = arrivals_no_priority[j][4]
    trans_male = arrivals_no_priority[j][5]
    g_queer = arrivals_no_priority[j][6]
    nonbinary = arrivals_no_priority[j][7]
    straight = arrivals_no_priority[j][8]
    lesbian= arrivals_no_priority[j][9]
    gay= arrivals_no_priority[j][10]
    bisexual= arrivals_no_priority[j][11]
    queer= arrivals_no_priority[j][12]
    child= arrivals_no_priority[j][13]
    immigrant = arrivals_no_priority[j][14]
    ht_victim = arrivals_no_priority[j][15]
    
    #serviced requested by youth
    mentalhealth = arrivals_no_priority[j][16]
    medical= arrivals_no_priority[j][17]
    substance_abuse= arrivals_no_priority[j][18]
    crisis = arrivals_no_priority[j][19]
    long_term = arrivals_no_priority[j][20]
    legal = arrivals_no_priority[j][21]
    service_coor = arrivals_no_priority[j][22]
    practical = arrivals_no_priority[j][23]
    financial = arrivals_no_priority[j][24]
    life_skills= arrivals_no_priority[j][25]
    employment= arrivals_no_priority[j][26]
    education = arrivals_no_priority[j][27]
    childcare = arrivals_no_priority[j][28]
    early = arrivals_no_priority[j][29]    

    #times of arriving youth
    arrival_time = arrivals_no_priority[j][30]
    service_time = random.exponential(scale=avg_service_time_3, size=1)[0]         #length of stay of youth
    patience_time = random.exponential(scale=avg_patience, size=1)[0]        #patience time            
        
    #find the number of youth in the system
    num_in_system_arrivals = []
    for k in range(0,i):
        num_in_system_arrivals.append(serviced_list_3[k][30])

    num_in_system_departures = []
    for k in range(0,i):
        num_in_system_departures.append(serviced_list_3[k][34])
            
    #number in system is equal to number of people who arrived till then minus the number of people who left the system
    #note that i include the youth arriving to the system as well (+1)
    number_in_system = sum(num_in_system_arrivals[0:i]<=arrival_time)- sum(num_in_system_departures[0:i]<=arrival_time)+1              #number of youths in system at the arrival
        
    tmp = sorted(num_in_system_departures)
    #if arrival is later than the first possible departure, service should start at the arrival time
    if arrival_time > tmp[len(serviced_list_3)- bed_s[3]]:
        start_service_time = arrival_time                                  #time youth starts service

    #if arrival is before the departure, service should start at the departure time
    else:
        start_service_time = tmp[len(serviced_list_3)- bed_s[3]]              #time youth starts service

    departure_time = start_service_time + service_time                #time youth leaves system
    wait_time = start_service_time -arrival_time                      #amount of time youth waits                                                         
    num_open_beds = max(bed_s[3] - number_in_system,0)                     #number of open beds
    utilization = (1-num_open_beds/bed_s[3])                               #utilization during the time youth is staying
    
    if wait_time >= patience_time:
        #create a list for the un-served youth
        actual_wait_time= patience_time                                #updated wait time based on abandonment
        received_service = 0                                           #binary variable that shows whether youth received service or not
          
        non_serviced_stacked = [number, age, female, male, trans_female, trans_male, g_queer, nonbinary, straight, lesbian, gay, bisexual, queer, child, immigrant, ht_victim, mentalhealth, medical, substance_abuse, crisis, long_term, legal, service_coor, practical, financial, life_skills, employment, education, childcare, early
                        ,arrival_time, service_time, patience_time, 
                        start_service_time, departure_time, wait_time, number_in_system , actual_wait_time, received_service, num_open_beds, utilization]
        non_serviced_list_3.append(non_serviced_stacked)
  
        #now we have to delete the youth that has been appended to non-serviced list from the arriving list
        for k in range(len(arrivals_to_3)):
            if arrivals_to_3[k][0] == number:
                arrivals_to_3 = np.delete(arrivals_to_3, k, 0)
                break
            
    else:
        actual_wait_time= wait_time                                     #updated wait time based on abandonment
        received_service = 1                                            #binary variable that shows whether youth received service or not
        #create a list for the serviced youths
        serviced_stacked = [number, age, female, male, trans_female, trans_male, g_queer, nonbinary, straight, lesbian, gay, bisexual, queer, child, immigrant, ht_victim, mentalhealth, medical, substance_abuse, crisis, long_term, legal, service_coor, practical, financial, life_skills, employment, education, childcare, early
                        ,arrival_time, service_time, patience_time, 
                        start_service_time, departure_time, wait_time, number_in_system , actual_wait_time, received_service, num_open_beds, utilization]
        serviced_list_3.append(serviced_stacked)
            
        #now we have to delete the youth that has been appended to serviced list from the arriving list
        for k in range(len(arrivals_to_3)):
            if arrivals_to_3[k][0] == number:
                arrivals_to_3 = np.delete(arrivals_to_3, k, 0)
                break
                    
        #if a youth received service, increase i by 1
        i= i+1
        
#Turn serviced and non-serviced lists into pandas data frames
serviced_df_3 = pd.DataFrame(serviced_list_3, columns =['number', 'age',' female', 'male', 'trans_female', 'trans_male', 'g_queer', 'nonbinary', 'straight', 'lesbian', 'gay', 
                                                        'bisexual', 'queer', 'child', 'immigrant', 'ht_victim', 'mentalhealth', 'medical', 'substance_abuse', 'crisis', 'long_term', 'legal', 
                                                        'service_coor', 'practical', 'financial', 'life_skills', 'employment', 'education', 'childcare', 'early'
                                                        ,'arrival time', 'service time', 'patience time', 
                                                        'start service time', 'departure time', 'wait time', 'number in system' , 'actual wait time', 'received service', 'num opennbeds', 'utilization']) 


non_serviced_df_3 = pd.DataFrame(non_serviced_list_3, columns =['number', 'age',' female', 'male', 'trans_female', 'trans_male', 'g_queer', 'nonbinary', 'straight', 'lesbian', 'gay', 
                                                        'bisexual', 'queer', 'child', 'immigrant', 'ht_victim', 'mentalhealth', 'medical', 'substance_abuse', 'crisis', 'long_term', 'legal', 
                                                        'service_coor', 'practical', 'financial', 'life_skills', 'employment', 'education', 'childcare', 'early'
                                                        ,'arrival time', 'service time', 'patience time', 
                                                        'start service time', 'departure time', 'wait time', 'number in system' , 'actual wait time', 'received service', 'num opennbeds', 'utilization']) 


__Here we look at the overall abandonment rate and the average wait at different shelters.__

In [207]:
################################
# Quality metrics for shelters # 
################################

serviced_without_warmup_0 = serviced_df_0.iloc[warmup_youth_number_0:]
serviced_without_warmup_1 = serviced_df_1.iloc[warmup_youth_number_1:]
serviced_without_warmup_2 = serviced_df_2.iloc[warmup_youth_number_2:]
serviced_without_warmup_3 = serviced_df_3.iloc[warmup_youth_number_3:]

#Abandonment Rate 
prop_abandoned_0 = len(non_serviced_df_0)/ (len(non_serviced_df_0)+ len(serviced_without_warmup_0))
prop_abandoned_1 = len(non_serviced_df_1)/ (len(non_serviced_df_1)+ len(serviced_without_warmup_1))
prop_abandoned_2 = len(non_serviced_df_2)/ (len(non_serviced_df_2)+ len(serviced_without_warmup_2))
prop_abandoned_3 = len(non_serviced_df_3)/ (len(non_serviced_df_3)+ len(serviced_without_warmup_3))

#Wait Times 
avg_wait_0 = serviced_without_warmup_0['actual wait time'].mean(axis=0)
avg_wait_1 = serviced_without_warmup_1['actual wait time'].mean(axis=0)
avg_wait_2 = serviced_without_warmup_2['actual wait time'].mean(axis=0)
avg_wait_3 = serviced_without_warmup_3['actual wait time'].mean(axis=0)

#Average queue length 
queue_lenght_0 = []
for i in range(len(serviced_without_warmup_0)):
    queue_lenght_0.append(max(0,serviced_without_warmup_0.iloc[i,36] - bed_s[0]))
avg_queue_0 = statistics.mean(queue_lenght_0)

queue_lenght_1 = []
for i in range(len(serviced_without_warmup_1)):
    queue_lenght_1.append(max(0,serviced_without_warmup_1.iloc[i,36] - bed_s[1]))
avg_queue_1 = statistics.mean(queue_lenght_1)

queue_lenght_2 = []
for i in range(len(serviced_without_warmup_2)):
    queue_lenght_2.append(max(0,serviced_without_warmup_2.iloc[i,36] - bed_s[2]))
avg_queue_2 = statistics.mean(queue_lenght_2)

queue_lenght_3 = []
for i in range(len(serviced_without_warmup_3)):
    queue_lenght_3.append(max(0,serviced_without_warmup_3.iloc[i,36] - bed_s[3]))
avg_queue_3 = statistics.mean(queue_lenght_3)

utilization_overall_0 = (serviced_without_warmup_0.iloc[1]['start service time'] - serviced_df_0.iloc[warmup_youth_number_0]['start service time'])* serviced_without_warmup_0.iloc[1]['utilization'] 

for i in range(2, len(serviced_without_warmup_0)):
    tmp_0 = (serviced_without_warmup_0.iloc[i]['start service time'] - serviced_without_warmup_0.iloc[i-1]['start service time'])
    util_timeframe_0 = tmp_0*serviced_without_warmup_0.iloc[i]['utilization'] 
    utilization_overall_0 = utilization_overall_0 + util_timeframe_0 
    
utilization_0 = utilization_overall_0/(serviced_without_warmup_0.iloc[-1]['start service time'] - serviced_without_warmup_0.iloc[1]['start service time']) 


utilization_overall_1 = (serviced_without_warmup_1.iloc[1]['start service time'] - serviced_df_1.iloc[warmup_youth_number_1]['start service time'])* serviced_without_warmup_1.iloc[1]['utilization'] 

for i in range(2, len(serviced_without_warmup_1)):
    tmp_1 = (serviced_without_warmup_1.iloc[i]['start service time'] - serviced_without_warmup_1.iloc[i-1]['start service time'])
    util_timeframe_1 = tmp_1*serviced_without_warmup_1.iloc[i]['utilization'] 
    utilization_overall_1 = utilization_overall_1 + util_timeframe_1 
    
utilization_1 = utilization_overall_1/(serviced_without_warmup_1.iloc[-1]['start service time'] - serviced_without_warmup_1.iloc[1]['start service time']) 


utilization_overall_2 = (serviced_without_warmup_2.iloc[1]['start service time'] - serviced_df_2.iloc[warmup_youth_number_2]['start service time'])* serviced_without_warmup_2.iloc[1]['utilization'] 

for i in range(2, len(serviced_without_warmup_2)):
    tmp_2 = (serviced_without_warmup_2.iloc[i]['start service time'] - serviced_without_warmup_2.iloc[i-1]['start service time'])
    util_timeframe_2 = tmp_2*serviced_without_warmup_2.iloc[i]['utilization'] 
    utilization_overall_2 = utilization_overall_2 + util_timeframe_2 
    
utilization_2 = utilization_overall_2/(serviced_without_warmup_2.iloc[-1]['start service time'] - serviced_without_warmup_2.iloc[1]['start service time']) 


utilization_overall_3 = (serviced_without_warmup_3.iloc[1]['start service time'] - serviced_df_3.iloc[warmup_youth_number_3]['start service time'])* serviced_without_warmup_3.iloc[1]['utilization'] 

for i in range(2, len(serviced_without_warmup_3)):
    tmp_3 = (serviced_without_warmup_3.iloc[i]['start service time'] - serviced_without_warmup_3.iloc[i-1]['start service time'])
    util_timeframe_3 = tmp_3*serviced_without_warmup_3.iloc[i]['utilization'] 
    utilization_overall_3 = utilization_overall_3 + util_timeframe_3 
    
utilization_3 = utilization_overall_3/(serviced_without_warmup_3.iloc[-1]['start service time'] - serviced_without_warmup_3.iloc[1]['start service time']) 


# print('---------------------')
# print('_Per Shelter Outputs_')
# print('---------------------')
# print('-Abandonment')
# print('Shelter 0: {:.4}'.format(prop_abandoned_0*100)+'% of youth has abandoned the system ')
# print('Shelter 1: {:.4}'.format(prop_abandoned_1*100)+'% of youth has abandoned the system ')
# print('Shelter 2: {:.4}'.format(prop_abandoned_2*100)+'% of youth has abandoned the system ')
# print('Shelter 3: {:.4}'.format(prop_abandoned_3*100)+'% of youth has abandoned the system ')

# print('')
# print('-Average Wait')
# print('Shelter 0: {:.4}'.format(avg_wait_0)+' days')
# print('Shelter 1: {:.4}'.format(avg_wait_1)+' days')
# print('Shelter 2: {:.4}'.format(avg_wait_2)+' days')
# print('Shelter 3: {:.4}'.format(avg_wait_3)+' days')


# print('')
# print('-Average Queue Length')
# print('Shelter 0: ' + str(avg_queue_0)+' youths')
# print('Shelter 1: ' + str(avg_queue_1)+' youths')
# print('Shelter 2: ' + str(avg_queue_2)+' youths')
# print('Shelter 3: ' + str(avg_queue_3)+' youths')

# print('')
# print('-Average Utilization')
# print('Shelter 0: ' + str(utilization_0))
# print('Shelter 1: ' + str(utilization_1))
# print('Shelter 2: ' + str(utilization_2))
# print('Shelter 3: ' + str(utilization_3))

#Summary Table for shelter and system outputs
num_total_arrivals = sum([directed_to_0, directed_to_1, directed_to_2, directed_to_3])
total_utilization = statistics.mean([utilization_0, utilization_1, utilization_2, utilization_3])
total_queue_length = statistics.mean([avg_queue_0, avg_queue_1, avg_queue_2, avg_queue_3])
total_abandonment = (len(non_serviced_df_0) +len(non_serviced_df_1)+len(non_serviced_df_2)+len(non_serviced_df_3))/ ((len(non_serviced_df_0) +len(non_serviced_df_1)+len(non_serviced_df_2)+len(non_serviced_df_3))+ (len(serviced_without_warmup_0)+len(serviced_without_warmup_1)+len(serviced_without_warmup_2)+len(serviced_without_warmup_3)))
total_avg_wait = statistics.mean([avg_wait_0, avg_wait_1, avg_wait_2, avg_wait_3])

number_arriving_list = [directed_to_0, directed_to_1, directed_to_2, directed_to_3, num_total_arrivals]
percent_of_arrivals_list = [directed_to_0/Y, directed_to_1/Y, directed_to_2/Y, directed_to_3/Y, num_total_arrivals/Y]
utilization_list = [utilization_0, utilization_1, utilization_2, utilization_3, total_utilization]
queue_length_list = [avg_queue_0, avg_queue_1, avg_queue_2, avg_queue_3,total_queue_length]
abandonment_list = [prop_abandoned_0, prop_abandoned_1, prop_abandoned_2, prop_abandoned_3, total_abandonment]
wait_time_list = [avg_wait_0, avg_wait_1, avg_wait_2, avg_wait_3, total_avg_wait]

dict = {'Shelter' : ['0','1','2','3', 'All'],
        '# of arriving' :number_arriving_list,
        '% of total arrivals' : percent_of_arrivals_list,
        'Utilization' : utilization_list,
        'Average queue length' : queue_length_list,
        '% of abandonment': abandonment_list, 
        'Average wait time':wait_time_list 
       }
df_overall_summary = pd.DataFrame(dict)
  

# displaying the DataFrame
display(df_overall_summary)

,Shelter,# of arriving,% of total arrivals,Utilization,Average queue length,% of abandonment,Average wait time
0,0,47,0.021759,0.229503,0.000000,0.000000,0.000000
1,1,1139,0.527315,0.980863,0.949254,0.117647,0.467591
2,2,428,0.198148,1.004242,2.262069,0.660422,4.733506
3,3,523,0.242130,1.000222,2.474026,0.704981,4.751223
4,All,2137,0.989352,0.803708,1.421337,0.367385,2.488080


__Now, we look at demographic groups seperately, and see what portion of them received services or abandoned the queue due to long waits. Also we look into how long on average each youth demographic group waits at each shelter.__
<br> females who did not receive services/ (females who did not receive services+ females who received services)

In [208]:
#Results based on demographic information

# Shelter 0

if prop_abandoned_0 >0:
    if len(serviced_without_warmup_0[serviced_without_warmup_0[' female'] == 1]) >0:
        prop_abandoned_0_female = len(non_serviced_df_0[non_serviced_df_0[' female'] == 1])/ (len(non_serviced_df_0[non_serviced_df_0[' female'] == 1])+ len(serviced_without_warmup_0[serviced_without_warmup_0[' female'] == 1]))

        number_arriving_0_female = len(serviced_without_warmup_0[serviced_without_warmup_0[' female'] == 1]) + len(non_serviced_df_0[non_serviced_df_0[' female'] == 1])
    else:
        prop_abandoned_0_female = 0
        number_arriving_0_female = 0
        
    if len(serviced_without_warmup_0[serviced_without_warmup_0['male'] == 1]) >0:
        prop_abandoned_0_male   = len(non_serviced_df_0[non_serviced_df_0['male'] == 1])/ (len(non_serviced_df_0[non_serviced_df_0['male'] == 1])+ len(serviced_without_warmup_0[serviced_without_warmup_0['male'] == 1]))
        number_arriving_0_male = len(serviced_without_warmup_0[serviced_without_warmup_0['male'] == 1]) + len(non_serviced_df_0[non_serviced_df_0['male'] == 1])
    
    else:
        prop_abandoned_0_male   = 0
        number_arriving_0_male = 0
        
    if (len(serviced_without_warmup_0[serviced_without_warmup_0['g_queer'] == 1])+
                             len(serviced_without_warmup_0[serviced_without_warmup_0['nonbinary'] == 1])+
                            len(serviced_without_warmup_0[serviced_without_warmup_0['trans_female'] == 1])+
                             len(serviced_without_warmup_0[serviced_without_warmup_0['trans_male'] == 1])) >0:
        prop_abandoned_0_noncis = (len(non_serviced_df_0[non_serviced_df_0['nonbinary'] == 1])+
                                   len(non_serviced_df_0[non_serviced_df_0['g_queer'] == 1])+
                                   len(non_serviced_df_0[non_serviced_df_0['trans_female'] == 1])+
                                   len(non_serviced_df_0[non_serviced_df_0['trans_male'] == 1]))/ (len(non_serviced_df_0[non_serviced_df_0['nonbinary'] == 1])+
                                 len(non_serviced_df_0[non_serviced_df_0['g_queer'] == 1])+
                                 len(non_serviced_df_0[non_serviced_df_0['trans_female'] == 1])+
                                 len(non_serviced_df_0[non_serviced_df_0['trans_male'] == 1])+
                                 len(serviced_without_warmup_0[serviced_without_warmup_0['g_queer'] == 1])+
                                 len(serviced_without_warmup_0[serviced_without_warmup_0['nonbinary'] == 1])+
                                len(serviced_without_warmup_0[serviced_without_warmup_0['trans_female'] == 1])+
                                 len(serviced_without_warmup_0[serviced_without_warmup_0['trans_male'] == 1]))
        number_arriving_0_noncis = (len(non_serviced_df_0[non_serviced_df_0['nonbinary'] == 1])+
                                 len(non_serviced_df_0[non_serviced_df_0['g_queer'] == 1])+
                                 len(non_serviced_df_0[non_serviced_df_0['trans_female'] == 1])+
                                 len(non_serviced_df_0[non_serviced_df_0['trans_male'] == 1])+
                                 len(serviced_without_warmup_0[serviced_without_warmup_0['g_queer'] == 1])+
                                 len(serviced_without_warmup_0[serviced_without_warmup_0['nonbinary'] == 1])+
                                len(serviced_without_warmup_0[serviced_without_warmup_0['trans_female'] == 1])+
                                 len(serviced_without_warmup_0[serviced_without_warmup_0['trans_male'] == 1]))
            
            
    else:
        prop_abandoned_0__noncis = 0
        number_arriving_0_noncis =0
        
    if len(serviced_without_warmup_0[serviced_without_warmup_0['age'] > 21])>0:
        prop_abandoned_0_older21 = len(non_serviced_df_0[non_serviced_df_0['age'] > 21])/ (len(non_serviced_df_0[non_serviced_df_0['age'] > 21])+ len(serviced_without_warmup_0[serviced_without_warmup_0['age'] > 21]))
        number_arriving_0_older21 = len(non_serviced_df_0[non_serviced_df_0['age'] > 21])+ len(serviced_without_warmup_0[serviced_without_warmup_0['age'] > 21])
                                    
    else:
        prop_abandoned_0_older21 = 0
        number_arriving_0_older21 = 0
        
    if len(serviced_without_warmup_0[serviced_without_warmup_0['age'] <= 21])>0:
        number_arriving_0_younger21 = (len(non_serviced_df_0[non_serviced_df_0['age'] <= 21])+ len(serviced_without_warmup_0[serviced_without_warmup_0['age'] <= 21]))
        prop_abandoned_0_younger21 = len(non_serviced_df_0[non_serviced_df_0['age'] <= 21])/ (len(non_serviced_df_0[non_serviced_df_0['age'] <= 21])+ len(serviced_without_warmup_0[serviced_without_warmup_0['age'] <= 21]))
    else:
        prop_abandoned_0_younger21 = 0
        number_arriving_0_younger21 = 0
        
    if len(serviced_without_warmup_0[serviced_without_warmup_0['child'] == 1]) >0:
        number_arriving_0_child = (len(non_serviced_df_0[non_serviced_df_0['child'] == 1])+ len(serviced_without_warmup_0[serviced_without_warmup_0['child'] == 1]))                                                    
        prop_abandoned_0_child = len(non_serviced_df_0[non_serviced_df_0['child'] == 1])/ (len(non_serviced_df_0[non_serviced_df_0['child'] == 1])+ len(serviced_without_warmup_0[serviced_without_warmup_0['child'] == 1]))
    else:
        prop_abandoned_0_child = 0
        number_arriving_0_child = 0
        
    if len(serviced_without_warmup_0[serviced_without_warmup_0['child'] == 0]) >0:
        number_arriving_0_no_child = (len(non_serviced_df_0[non_serviced_df_0['child'] == 0])+ len(serviced_without_warmup_0[serviced_without_warmup_0['child'] == 0]))
        prop_abandoned_0_no_child = len(non_serviced_df_0[non_serviced_df_0['child'] == 0])/ (len(non_serviced_df_0[non_serviced_df_0['child'] == 0])+ len(serviced_without_warmup_0[serviced_without_warmup_0['child'] == 0]))
    else:
        prop_abandoned_0_no_child = 0
        number_arriving_0_no_child = 0
                                                                                           
    if len(serviced_without_warmup_0[serviced_without_warmup_0['immigrant'] == 1])>0: 
        number_arriving_0_immigrant = (len(non_serviced_df_0[non_serviced_df_0['immigrant'] == 1])+ len(serviced_without_warmup_0[serviced_without_warmup_0['immigrant'] == 1]))
        prop_abandoned_0_immigrant = len(non_serviced_df_0[non_serviced_df_0['immigrant'] == 1])/ (len(non_serviced_df_0[non_serviced_df_0['immigrant'] == 1])+ len(serviced_without_warmup_0[serviced_without_warmup_0['immigrant'] == 1]))
    else:
        prop_abandoned_0_immigrant = 0
        number_arriving_0_immigrant = 0
        
    if len(serviced_without_warmup_0[serviced_without_warmup_0['immigrant'] == 0])>0:
        number_arriving_0_not_immigrant = (len(non_serviced_df_0[non_serviced_df_0['immigrant'] == 0])+ len(serviced_without_warmup_0[serviced_without_warmup_0['immigrant'] == 0]))
        prop_abandoned_0_not_immigrant = len(non_serviced_df_0[non_serviced_df_0['immigrant'] == 0])/ (len(non_serviced_df_0[non_serviced_df_0['immigrant'] == 0])+ len(serviced_without_warmup_0[serviced_without_warmup_0['immigrant'] == 0]))
    else:
        prop_abandoned_0_not_immigrant = 0
        number_arriving_0_not_immigrant = 0
                                                                                           
    if  len(serviced_without_warmup_0[serviced_without_warmup_0['ht_victim'] == 1])>0:
        number_arriving_0_ht = (len(non_serviced_df_0[non_serviced_df_0['ht_victim'] == 1])+ len(serviced_without_warmup_0[serviced_without_warmup_0['ht_victim'] == 1]))
        prop_abandoned_0_ht = len(non_serviced_df_0[non_serviced_df_0['ht_victim'] == 1])/ (len(non_serviced_df_0[non_serviced_df_0['ht_victim'] == 1])+ len(serviced_without_warmup_0[serviced_without_warmup_0['ht_victim'] == 1]))
    else:
        prop_abandoned_0_ht = 0  
        number_arriving_0_ht = 0
        
    if  len(serviced_without_warmup_0[serviced_without_warmup_0['ht_victim'] == 0])>0:
        number_arriving_0_not_ht = (len(non_serviced_df_0[non_serviced_df_0['ht_victim'] == 0])+ len(serviced_without_warmup_0[serviced_without_warmup_0['ht_victim'] == 0]))                           
        prop_abandoned_0_not_ht = len(non_serviced_df_0[non_serviced_df_0['ht_victim'] == 0])/ (len(non_serviced_df_0[non_serviced_df_0['ht_victim'] == 0])+ len(serviced_without_warmup_0[serviced_without_warmup_0['ht_victim'] == 0]))
    else:
        prop_abandoned_0_not_ht = 0  
        number_arriving_0_not_ht = 0
        
else:
    number_arriving_0_female = len(serviced_without_warmup_0[serviced_without_warmup_0[' female'] == 1]) + len(non_serviced_df_0[non_serviced_df_0[' female'] == 1])
    number_arriving_0_male = len(serviced_without_warmup_0[serviced_without_warmup_0['male'] == 1]) + len(non_serviced_df_0[non_serviced_df_0['male'] == 1])
    number_arriving_0_noncis = (len(non_serviced_df_0[non_serviced_df_0['nonbinary'] == 1])+
                                     len(non_serviced_df_0[non_serviced_df_0['g_queer'] == 1])+
                                     len(non_serviced_df_0[non_serviced_df_0['trans_female'] == 1])+
                                     len(non_serviced_df_0[non_serviced_df_0['trans_male'] == 1])+
                                     len(serviced_without_warmup_0[serviced_without_warmup_0['g_queer'] == 1])+
                                     len(serviced_without_warmup_0[serviced_without_warmup_0['nonbinary'] == 1])+
                                    len(serviced_without_warmup_0[serviced_without_warmup_0['trans_female'] == 1])+
                                     len(serviced_without_warmup_0[serviced_without_warmup_0['trans_male'] == 1]))
    number_arriving_0_older21 = len(non_serviced_df_0[non_serviced_df_0['age'] > 21])+ len(serviced_without_warmup_0[serviced_without_warmup_0['age'] > 21])
    number_arriving_0_younger21 = (len(non_serviced_df_0[non_serviced_df_0['age'] <= 21])+ len(serviced_without_warmup_0[serviced_without_warmup_0['age'] <= 21]))
    number_arriving_0_child = (len(non_serviced_df_0[non_serviced_df_0['child'] == 1])+ len(serviced_without_warmup_0[serviced_without_warmup_0['child'] == 1]))
    number_arriving_0_no_child = (len(non_serviced_df_0[non_serviced_df_0['child'] == 0])+ len(serviced_without_warmup_0[serviced_without_warmup_0['child'] == 0]))
    number_arriving_0_immigrant = (len(non_serviced_df_0[non_serviced_df_0['immigrant'] == 1])+ len(serviced_without_warmup_0[serviced_without_warmup_0['immigrant'] == 1]))
    number_arriving_0_not_immigrant = (len(non_serviced_df_0[non_serviced_df_0['immigrant'] == 0])+ len(serviced_without_warmup_0[serviced_without_warmup_0['immigrant'] == 0]))
    number_arriving_0_ht = (len(non_serviced_df_0[non_serviced_df_0['ht_victim'] == 1])+ len(serviced_without_warmup_0[serviced_without_warmup_0['ht_victim'] == 1]))
    number_arriving_0_not_ht = (len(non_serviced_df_0[non_serviced_df_0['ht_victim'] == 0])+ len(serviced_without_warmup_0[serviced_without_warmup_0['ht_victim'] == 0]))
    
#Wait Times based on the vulnerability groups
avg_wait_0_female = serviced_without_warmup_0[serviced_without_warmup_0[' female'] == 1]['actual wait time'].mean(axis=0)
avg_wait_0_male = serviced_without_warmup_0[serviced_without_warmup_0['male'] == 1]['actual wait time'].mean(axis=0)
avg_wait_0_noncis = statistics.mean([serviced_without_warmup_0[serviced_without_warmup_0['trans_female'] == 1]['actual wait time'].mean(axis=0), 
                            serviced_without_warmup_0[serviced_without_warmup_0['trans_male'] == 1]['actual wait time'].mean(axis=0), 
                            serviced_without_warmup_0[serviced_without_warmup_0['nonbinary'] == 1]['actual wait time'].mean(axis=0), 
                            serviced_without_warmup_0[serviced_without_warmup_0['g_queer'] == 1]['actual wait time'].mean(axis=0)])
avg_wait_0_older21 = serviced_without_warmup_0[serviced_without_warmup_0['age'] > 21]['actual wait time'].mean(axis=0)
avg_wait_0_younger21 = serviced_without_warmup_0[serviced_without_warmup_0['age'] <= 21]['actual wait time'].mean(axis=0)
avg_wait_0_child = serviced_without_warmup_0[serviced_without_warmup_0['child'] == 1]['actual wait time'].mean(axis=0)
avg_wait_0_no_child = serviced_without_warmup_0[serviced_without_warmup_0['child'] == 0]['actual wait time'].mean(axis=0)
avg_wait_0_immigrant = serviced_without_warmup_0[serviced_without_warmup_0['immigrant'] == 1]['actual wait time'].mean(axis=0)
avg_wait_0_not_immigrant = serviced_without_warmup_0[serviced_without_warmup_0['immigrant'] == 0]['actual wait time'].mean(axis=0)
avg_wait_0_ht = serviced_without_warmup_0[serviced_without_warmup_0['ht_victim'] == 1]['actual wait time'].mean(axis=0)
avg_wait_0_not_ht = serviced_without_warmup_0[serviced_without_warmup_0['ht_victim'] == 0]['actual wait time'].mean(axis=0)


# print('----------------')
# print('__  Shelter 0 __')
# print('----------------')
# if prop_abandoned_0 >0:
#     print('-Abandonment')
#     print('Female: {:.4}'.format(float(prop_abandoned_0_female*100))+'% among their demographic group ')
#     print('Male: {:.4}'.format(float(prop_abandoned_0_male*100))+'% among their demographic group ')
#     print('Non-cisgender: {:.4}'.format(float(prop_abandoned_0_noncis*100))+'% among their demographic group ')
#     print('Older than 21: {:.4}'.format(float(prop_abandoned_0_older21*100))+'% among their demographic group ')
#     print('Has child: {:.4}'.format(float(prop_abandoned_0_child*100))+'% among their demographic group ')
#     print('Immigrant: {:.4}'.format(float(prop_abandoned_0_immigrant*100))+'% among their demographic group ')
#     print('HT Victim: {:.4}'.format(float(prop_abandoned_0_ht*100))+'% among their demographic group ')
# else: 
#     print('No youth abandoned the system.')
    
    
# print('')
# print('-Average Wait')
# print('Female: {:.4}'.format(float(avg_wait_0_female))+' days')
# print('Male: {:.4}'.format(float(avg_wait_0_male))+' days')
# print('Non-cisgender: {:.4}'.format(float(avg_wait_0_noncis))+' days')
# print('Older than 21: {:.4}'.format(float(avg_wait_0_older21))+' days')
# print('Has child: {:.4}'.format(float(avg_wait_0_child))+' days')
# print('Immigrant: {:.4}'.format(float(avg_wait_0_immigrant))+' days')
# print('HT Victim: {:.4}'.format(float(avg_wait_0_ht))+' days')

# Shelter 1

if prop_abandoned_1 >0:
    if len(serviced_without_warmup_1[serviced_without_warmup_1[' female'] == 1]) >0:
        prop_abandoned_1_female = len(non_serviced_df_1[non_serviced_df_1[' female'] == 1])/ (len(non_serviced_df_1[non_serviced_df_1[' female'] == 1])+ len(serviced_without_warmup_1[serviced_without_warmup_1[' female'] == 1]))
        number_arriving_1_female = len(serviced_without_warmup_1[serviced_without_warmup_1[' female'] == 1]) + len(non_serviced_df_1[non_serviced_df_1[' female'] == 1])                     
                                    
    else:
        prop_abandoned_1_female = 0
        number_arriving_1_female = 0
        
    if len(serviced_without_warmup_1[serviced_without_warmup_1['male'] == 1]) >0:
        number_arriving_1_male = len(serviced_without_warmup_1[serviced_without_warmup_1['male'] == 1]) + len(non_serviced_df_1[non_serviced_df_1['male'] == 1])
        prop_abandoned_1_male   = len(non_serviced_df_1[non_serviced_df_1['male'] == 1])/ (len(non_serviced_df_1[non_serviced_df_1['male'] == 1])+ len(serviced_without_warmup_1[serviced_without_warmup_1['male'] == 1]))
    else:
        prop_abandoned_1_male   = 0
        number_arriving_1_male =0
        
    if (len(serviced_without_warmup_1[serviced_without_warmup_1['g_queer'] == 1])+
                             len(serviced_without_warmup_1[serviced_without_warmup_1['nonbinary'] == 1])+
                            len(serviced_without_warmup_1[serviced_without_warmup_1['trans_female'] == 1])+
                             len(serviced_without_warmup_1[serviced_without_warmup_1['trans_male'] == 1])) >0:
        prop_abandoned_1_noncis = (len(non_serviced_df_1[non_serviced_df_1['nonbinary'] == 1])+
                                   len(non_serviced_df_1[non_serviced_df_1['g_queer'] == 1])+
                                   len(non_serviced_df_1[non_serviced_df_1['trans_female'] == 1])+
                                   len(non_serviced_df_1[non_serviced_df_1['trans_male'] == 1]))/ (len(non_serviced_df_1[non_serviced_df_1['nonbinary'] == 1])+
                                 len(non_serviced_df_1[non_serviced_df_1['g_queer'] == 1])+
                                 len(non_serviced_df_1[non_serviced_df_1['trans_female'] == 1])+
                                 len(non_serviced_df_1[non_serviced_df_1['trans_male'] == 1])+
                                 len(serviced_without_warmup_1[serviced_without_warmup_1['g_queer'] == 1])+
                                 len(serviced_without_warmup_1[serviced_without_warmup_1['nonbinary'] == 1])+
                                len(serviced_without_warmup_1[serviced_without_warmup_1['trans_female'] == 1])+
                                 len(serviced_without_warmup_1[serviced_without_warmup_1['trans_male'] == 1]))
        number_arriving_1_noncis = (len(non_serviced_df_1[non_serviced_df_1['nonbinary'] == 1])+
                                 len(non_serviced_df_1[non_serviced_df_1['g_queer'] == 1])+
                                 len(non_serviced_df_1[non_serviced_df_1['trans_female'] == 1])+
                                 len(non_serviced_df_1[non_serviced_df_1['trans_male'] == 1])+
                                 len(serviced_without_warmup_1[serviced_without_warmup_1['g_queer'] == 1])+
                                 len(serviced_without_warmup_1[serviced_without_warmup_1['nonbinary'] == 1])+
                                len(serviced_without_warmup_1[serviced_without_warmup_1['trans_female'] == 1])+
                                 len(serviced_without_warmup_1[serviced_without_warmup_1['trans_male'] == 1]))
    else:
        prop_abandoned_1_noncis = 0
        number_arriving_1_noncis =0
        
    if len(serviced_without_warmup_1[serviced_without_warmup_1['age'] > 21])>0:
        prop_abandoned_1_older21 = len(non_serviced_df_1[non_serviced_df_1['age'] > 21])/ (len(non_serviced_df_1[non_serviced_df_1['age'] > 21])+ len(serviced_without_warmup_1[serviced_without_warmup_1['age'] > 21]))
        number_arriving_1_older21 = len(non_serviced_df_1[non_serviced_df_1['age'] > 21])+ len(serviced_without_warmup_1[serviced_without_warmup_1['age'] > 21])

    else:
        prop_abandoned_1_older21 = 0
        number_arriving_1_older21 =0
        
    if len(serviced_without_warmup_1[serviced_without_warmup_1['age'] <= 21])>0:
        number_arriving_1_younger21 = (len(non_serviced_df_1[non_serviced_df_1['age'] <= 21])+ len(serviced_without_warmup_1[serviced_without_warmup_1['age'] <= 21]))
        prop_abandoned_1_younger21 = len(non_serviced_df_1[non_serviced_df_1['age'] <= 21])/ (len(non_serviced_df_1[non_serviced_df_1['age'] <= 21])+ len(serviced_without_warmup_1[serviced_without_warmup_1['age'] <= 21]))
    else:
        prop_abandoned_1_younger21 = 0
        number_arriving_1_younger21 = 0
        
    if len(serviced_without_warmup_1[serviced_without_warmup_1['child'] == 1]) >0:
        number_arriving_1_child = (len(non_serviced_df_1[non_serviced_df_1['child'] == 1])+ len(serviced_without_warmup_1[serviced_without_warmup_1['child'] == 1]))
        prop_abandoned_1_child = len(non_serviced_df_1[non_serviced_df_1['child'] == 1])/ (len(non_serviced_df_1[non_serviced_df_1['child'] == 1])+ len(serviced_without_warmup_1[serviced_without_warmup_1['child'] == 1]))
    else:
        prop_abandoned_1_child = 0
        number_arriving_1_child = 0
        
    if len(serviced_without_warmup_1[serviced_without_warmup_1['child'] == 0]) >0:
        number_arriving_1_no_child = (len(non_serviced_df_1[non_serviced_df_1['child'] == 0])+ len(serviced_without_warmup_1[serviced_without_warmup_1['child'] == 0]))
        prop_abandoned_1_no_child = len(non_serviced_df_1[non_serviced_df_1['child'] == 0])/ (len(non_serviced_df_1[non_serviced_df_1['child'] == 0])+ len(serviced_without_warmup_1[serviced_without_warmup_1['child'] == 0]))
    else:
        prop_abandoned_1_no_child = 0
        number_arriving_1_no_child = 0
                                                                                           
    if len(serviced_without_warmup_1[serviced_without_warmup_1['immigrant'] == 1])>0:        
        number_arriving_1_immigrant = (len(non_serviced_df_1[non_serviced_df_1['immigrant'] == 1])+ len(serviced_without_warmup_1[serviced_without_warmup_1['immigrant'] == 1]))
        prop_abandoned_1_immigrant = len(non_serviced_df_1[non_serviced_df_1['immigrant'] == 1])/ (len(non_serviced_df_1[non_serviced_df_1['immigrant'] == 1])+ len(serviced_without_warmup_1[serviced_without_warmup_1['immigrant'] == 1]))
    else:
        prop_abandoned_1_immigrant = 0
        number_arriving_1_immigrant = 0
        
    if len(serviced_without_warmup_1[serviced_without_warmup_1['immigrant'] == 0])>0:                                                                                  
        prop_abandoned_1_not_immigrant = len(non_serviced_df_1[non_serviced_df_1['immigrant'] == 0])/ (len(non_serviced_df_1[non_serviced_df_1['immigrant'] == 0])+ len(serviced_without_warmup_1[serviced_without_warmup_1['immigrant'] == 0]))
        number_arriving_1_not_immigrant = (len(non_serviced_df_1[non_serviced_df_1['immigrant'] == 0])+ len(serviced_without_warmup_1[serviced_without_warmup_1['immigrant'] == 0]))

    else:
        prop_abandoned_1_not_immigrant = 0
        number_arriving_1_not_immigrant = 0
                                                                                           
    if  len(serviced_without_warmup_1[serviced_without_warmup_1['ht_victim'] == 1])>0:
        number_arriving_1_ht = (len(non_serviced_df_1[non_serviced_df_1['ht_victim'] == 1])+ len(serviced_without_warmup_1[serviced_without_warmup_1['ht_victim'] == 1]))
        prop_abandoned_1_ht = len(non_serviced_df_1[non_serviced_df_1['ht_victim'] == 1])/ (len(non_serviced_df_1[non_serviced_df_1['ht_victim'] == 1])+ len(serviced_without_warmup_1[serviced_without_warmup_1['ht_victim'] == 1]))
    else:
        prop_abandoned_1_ht = 0 
        number_arriving_1_ht = 0
    
    if  len(serviced_without_warmup_1[serviced_without_warmup_1['ht_victim'] == 0])>0:
        number_arriving_1_not_ht = (len(non_serviced_df_1[non_serviced_df_1['ht_victim'] == 0])+ len(serviced_without_warmup_1[serviced_without_warmup_1['ht_victim'] == 0]))      
        prop_abandoned_1_not_ht = len(non_serviced_df_1[non_serviced_df_1['ht_victim'] == 0])/ (len(non_serviced_df_1[non_serviced_df_1['ht_victim'] == 0])+ len(serviced_without_warmup_1[serviced_without_warmup_1['ht_victim'] == 0]))
    else:
        prop_abandoned_1_not_ht = 0  
        number_arriving_1_not_ht = 0

else:
    number_arriving_1_female = len(serviced_without_warmup_1[serviced_without_warmup_1[' female'] == 1]) + len(non_serviced_df_1[non_serviced_df_1[' female'] == 1])
    number_arriving_1_male = len(serviced_without_warmup_1[serviced_without_warmup_1['male'] == 1]) + len(non_serviced_df_1[non_serviced_df_1['male'] == 1])
    number_arriving_1_noncis = (len(non_serviced_df_1[non_serviced_df_1['nonbinary'] == 1])+
                                     len(non_serviced_df_1[non_serviced_df_1['g_queer'] == 1])+
                                     len(non_serviced_df_1[non_serviced_df_1['trans_female'] == 1])+
                                     len(non_serviced_df_1[non_serviced_df_1['trans_male'] == 1])+
                                     len(serviced_without_warmup_1[serviced_without_warmup_1['g_queer'] == 1])+
                                     len(serviced_without_warmup_1[serviced_without_warmup_1['nonbinary'] == 1])+
                                    len(serviced_without_warmup_1[serviced_without_warmup_1['trans_female'] == 1])+
                                     len(serviced_without_warmup_1[serviced_without_warmup_1['trans_male'] == 1]))
    number_arriving_1_older21 = len(non_serviced_df_1[non_serviced_df_1['age'] > 21])+ len(serviced_without_warmup_1[serviced_without_warmup_1['age'] > 21])
    number_arriving_1_younger21 = (len(non_serviced_df_1[non_serviced_df_1['age'] <= 21])+ len(serviced_without_warmup_1[serviced_without_warmup_1['age'] <= 21]))
    number_arriving_1_child = (len(non_serviced_df_1[non_serviced_df_1['child'] == 1])+ len(serviced_without_warmup_1[serviced_without_warmup_1['child'] == 1]))
    number_arriving_1_no_child = (len(non_serviced_df_1[non_serviced_df_1['child'] == 0])+ len(serviced_without_warmup_1[serviced_without_warmup_1['child'] == 0]))
    number_arriving_1_immigrant = (len(non_serviced_df_1[non_serviced_df_1['immigrant'] == 1])+ len(serviced_without_warmup_1[serviced_without_warmup_1['immigrant'] == 1]))
    number_arriving_1_not_immigrant = (len(non_serviced_df_1[non_serviced_df_1['immigrant'] == 0])+ len(serviced_without_warmup_1[serviced_without_warmup_1['immigrant'] == 0]))
    number_arriving_1_ht = (len(non_serviced_df_1[non_serviced_df_1['ht_victim'] == 1])+ len(serviced_without_warmup_1[serviced_without_warmup_1['ht_victim'] == 1]))
    number_arriving_1_not_ht = (len(non_serviced_df_1[non_serviced_df_1['ht_victim'] == 0])+ len(serviced_without_warmup_1[serviced_without_warmup_1['ht_victim'] == 0]))
        
#Wait Times based on the vulnerability groups
avg_wait_1_female = serviced_without_warmup_1[serviced_without_warmup_1[' female'] == 1]['actual wait time'].mean(axis=0)
avg_wait_1_male = serviced_without_warmup_1[serviced_without_warmup_1['male'] == 1]['actual wait time'].mean(axis=0)
avg_wait_1_noncis = statistics.mean([serviced_without_warmup_1[serviced_without_warmup_1['trans_female'] == 1]['actual wait time'].mean(axis=0), 
                            serviced_without_warmup_1[serviced_without_warmup_1['trans_male'] == 1]['actual wait time'].mean(axis=0), 
                            serviced_without_warmup_1[serviced_without_warmup_1['nonbinary'] == 1]['actual wait time'].mean(axis=0), 
                            serviced_without_warmup_1[serviced_without_warmup_1['g_queer'] == 1]['actual wait time'].mean(axis=0)])
avg_wait_1_older21 = serviced_without_warmup_1[serviced_without_warmup_1['age'] > 21]['actual wait time'].mean(axis=0)
avg_wait_1_younger21 = serviced_without_warmup_1[serviced_without_warmup_1['age'] <= 21]['actual wait time'].mean(axis=0)
avg_wait_1_child = serviced_without_warmup_1[serviced_without_warmup_1['child'] == 1]['actual wait time'].mean(axis=0)
avg_wait_1_no_child = serviced_without_warmup_1[serviced_without_warmup_1['child'] == 0]['actual wait time'].mean(axis=0)
avg_wait_1_immigrant = serviced_without_warmup_1[serviced_without_warmup_1['immigrant'] == 1]['actual wait time'].mean(axis=0)
avg_wait_1_not_immigrant = serviced_without_warmup_1[serviced_without_warmup_1['immigrant'] == 0]['actual wait time'].mean(axis=0)
avg_wait_1_ht = serviced_without_warmup_1[serviced_without_warmup_1['ht_victim'] == 1]['actual wait time'].mean(axis=0)
avg_wait_1_not_ht = serviced_without_warmup_1[serviced_without_warmup_1['ht_victim'] == 0]['actual wait time'].mean(axis=0)


# print('----------------')
# print('__  Shelter 1 __')
# print('----------------')
# if prop_abandoned_1 >0:
#     print('-Abandonment')
#     print('Female: {:.4}'.format(float(prop_abandoned_1_female*100))+'% among their demographic group ')
#     print('Male: {:.4}'.format(float(prop_abandoned_1_male*100))+'% among their demographic group ')
#     print('Non cis gender: {:.4}'.format(float(prop_abandoned_1_noncis*100))+'% among their demographic group ')
#     print('Older than 21: {:.4}'.format(float(prop_abandoned_1_older21*100))+'% among their demographic group ')
#     print('Has child: {:.4}'.format(float(prop_abandoned_1_child*100))+'% among their demographic group ')
#     print('Immigrant: {:.4}'.format(float(prop_abandoned_1_immigrant*100))+'% among their demographic group ')
#     print('HT Victim: {:.4}'.format(float(prop_abandoned_1_ht*100))+'% among their demographic group ')
# else: 
#     print('No youth abandoned the system.')
    
    
# print('')
# print('-Average Wait')
# print('Female: {:.4}'.format(float(avg_wait_1_female))+' days')
# print('Male: {:.4}'.format(float(avg_wait_1_male))+' days')
# print('Non cis gender: {:.4}'.format(float(avg_wait_1_noncis))+' days')
# print('Older than 21: {:.4}'.format(float(avg_wait_1_older21))+' days')
# print('Has child: {:.4}'.format(float(avg_wait_1_child))+' days')
# print('Immigrant: {:.4}'.format(float(avg_wait_1_immigrant))+' days')
# print('HT Victim: {:.4}'.format(float(avg_wait_1_ht))+' days')

# Shelter 2

if prop_abandoned_2 >0:
    if len(serviced_without_warmup_2[serviced_without_warmup_2[' female'] == 1]) >0:
        number_arriving_2_female = len(serviced_without_warmup_2[serviced_without_warmup_2[' female'] == 1]) + len(non_serviced_df_2[non_serviced_df_2[' female'] == 1])
        prop_abandoned_2_female = len(non_serviced_df_2[non_serviced_df_2[' female'] == 1])/ (len(non_serviced_df_2[non_serviced_df_2[' female'] == 1])+ len(serviced_without_warmup_2[serviced_without_warmup_2[' female'] == 1]))
    else:
        prop_abandoned_2_female = 0
        number_arriving_2_female = 0
        
    if len(serviced_without_warmup_2[serviced_without_warmup_2['male'] == 1]) >0:
        number_arriving_2_male = len(serviced_without_warmup_2[serviced_without_warmup_2['male'] == 1]) +len(non_serviced_df_2[non_serviced_df_2['male'] == 1])
        prop_abandoned_2_male   = len(non_serviced_df_2[non_serviced_df_2['male'] == 1])/ (len(non_serviced_df_2[non_serviced_df_2['male'] == 1])+ len(serviced_without_warmup_2[serviced_without_warmup_2['male'] == 1]))
    else:
        prop_abandoned_2_male   = 0
        number_arriving_2_male =0
        
    if (len(serviced_without_warmup_2[serviced_without_warmup_2['g_queer'] == 1])+
                             len(serviced_without_warmup_2[serviced_without_warmup_2['nonbinary'] == 1])+
                            len(serviced_without_warmup_2[serviced_without_warmup_2['trans_female'] == 1])+
                             len(serviced_without_warmup_2[serviced_without_warmup_2['trans_male'] == 1])) >0:
        prop_abandoned_2_noncis = (len(non_serviced_df_2[non_serviced_df_2['nonbinary'] == 1])+
                                   len(non_serviced_df_2[non_serviced_df_2['g_queer'] == 1])+
                                   len(non_serviced_df_2[non_serviced_df_2['trans_female'] == 1])+
                                   len(non_serviced_df_2[non_serviced_df_2['trans_male'] == 1]))/ (len(non_serviced_df_2[non_serviced_df_2['nonbinary'] == 1])+
                                 len(non_serviced_df_2[non_serviced_df_2['g_queer'] == 1])+
                                 len(non_serviced_df_2[non_serviced_df_2['trans_female'] == 1])+
                                 len(non_serviced_df_2[non_serviced_df_2['trans_male'] == 1])+
                                 len(serviced_without_warmup_2[serviced_without_warmup_2['g_queer'] == 1])+
                                 len(serviced_without_warmup_2[serviced_without_warmup_2['nonbinary'] == 1])+
                                len(serviced_without_warmup_2[serviced_without_warmup_2['trans_female'] == 1])+
                                 len(serviced_without_warmup_2[serviced_without_warmup_2['trans_male'] == 1]))
        number_arriving_2_noncis = (len(non_serviced_df_2[non_serviced_df_2['nonbinary'] == 1])+
                                 len(non_serviced_df_2[non_serviced_df_2['g_queer'] == 1])+
                                 len(non_serviced_df_2[non_serviced_df_2['trans_female'] == 1])+
                                 len(non_serviced_df_2[non_serviced_df_2['trans_male'] == 1])+
                                 len(serviced_without_warmup_2[serviced_without_warmup_2['g_queer'] == 1])+
                                 len(serviced_without_warmup_2[serviced_without_warmup_2['nonbinary'] == 1])+
                                len(serviced_without_warmup_2[serviced_without_warmup_2['trans_female'] == 1])+
                                 len(serviced_without_warmup_2[serviced_without_warmup_2['trans_male'] == 1]))
                                                                                                                                                                             
    else:
        prop_abandoned_2_noncis = 0
        number_arriving_2_noncis = 0
        
    if len(serviced_without_warmup_2[serviced_without_warmup_2['age'] > 21])>0:
        number_arriving_2_older21 = len(non_serviced_df_2[non_serviced_df_2['age'] > 21])+ len(serviced_without_warmup_2[serviced_without_warmup_2['age'] > 21])
        prop_abandoned_2_older21 = len(non_serviced_df_2[non_serviced_df_2['age'] > 21])/ (len(non_serviced_df_2[non_serviced_df_2['age'] > 21])+ len(serviced_without_warmup_2[serviced_without_warmup_2['age'] > 21]))
    else:
        prop_abandoned_2_older21 = 0
        number_arriving_2_older21 =0
        
    if len(serviced_without_warmup_2[serviced_without_warmup_2['age'] <= 21])>0:
        number_arriving_2_younger21 = (len(non_serviced_df_2[non_serviced_df_2['age'] <= 21])+ len(serviced_without_warmup_2[serviced_without_warmup_2['age'] <= 21]))
        prop_abandoned_2_younger21 = len(non_serviced_df_2[non_serviced_df_2['age'] <= 21])/ (len(non_serviced_df_2[non_serviced_df_2['age'] <= 21])+ len(serviced_without_warmup_2[serviced_without_warmup_2['age'] <= 21]))
    else:
        prop_abandoned_2_younger21 = 0
        number_arriving_2_younger21 = 0
        
    if len(serviced_without_warmup_2[serviced_without_warmup_2['child'] == 1]) >0:
        number_arriving_2_child = (len(non_serviced_df_2[non_serviced_df_2['child'] == 1])+ len(serviced_without_warmup_2[serviced_without_warmup_2['child'] == 1]))
        prop_abandoned_2_child = len(non_serviced_df_2[non_serviced_df_2['child'] == 1])/ (len(non_serviced_df_2[non_serviced_df_2['child'] == 1])+ len(serviced_without_warmup_2[serviced_without_warmup_2['child'] == 1]))
    else:
        prop_abandoned_2_child = 0
        number_arriving_2_child = 0
        
    if len(serviced_without_warmup_2[serviced_without_warmup_2['child'] == 0]) >0:
        number_arriving_2_no_child = (len(non_serviced_df_2[non_serviced_df_2['child'] == 0])+ len(serviced_without_warmup_2[serviced_without_warmup_2['child'] == 0]))
        prop_abandoned_2_no_child = len(non_serviced_df_2[non_serviced_df_2['child'] == 0])/ (len(non_serviced_df_2[non_serviced_df_2['child'] == 0])+ len(serviced_without_warmup_2[serviced_without_warmup_2['child'] == 0]))
    else:
        prop_abandoned_2_no_child = 0
        number_arriving_2_no_child = 0
                                                                                           
    if len(serviced_without_warmup_2[serviced_without_warmup_2['immigrant'] == 1])>0:    
        number_arriving_2_immigrant = (len(non_serviced_df_2[non_serviced_df_2['immigrant'] == 1])+ len(serviced_without_warmup_2[serviced_without_warmup_2['immigrant'] == 1]))  
        prop_abandoned_2_immigrant = len(non_serviced_df_2[non_serviced_df_2['immigrant'] == 1])/ (len(non_serviced_df_2[non_serviced_df_2['immigrant'] == 1])+ len(serviced_without_warmup_2[serviced_without_warmup_2['immigrant'] == 1]))
    else:
        prop_abandoned_2_immigrant = 0
        number_arriving_2_immigrant = 0
        
    if len(serviced_without_warmup_2[serviced_without_warmup_2['immigrant'] == 0])>0: 
        number_arriving_2_not_immigrant = (len(non_serviced_df_2[non_serviced_df_2['immigrant'] == 0])+ len(serviced_without_warmup_2[serviced_without_warmup_2['immigrant'] == 0]))
        prop_abandoned_2_not_immigrant = len(non_serviced_df_2[non_serviced_df_2['immigrant'] == 0])/ (len(non_serviced_df_2[non_serviced_df_2['immigrant'] == 0])+ len(serviced_without_warmup_2[serviced_without_warmup_2['immigrant'] == 0]))
    else:
        prop_abandoned_2_not_immigrant = 0
        number_arriving_2_not_immigrant = 0
                                                                                           
    if  len(serviced_without_warmup_2[serviced_without_warmup_2['ht_victim'] == 1])>0:
        number_arriving_2_ht = (len(non_serviced_df_2[non_serviced_df_2['ht_victim'] == 1])+ len(serviced_without_warmup_2[serviced_without_warmup_2['ht_victim'] == 1]))
        prop_abandoned_2_ht = len(non_serviced_df_2[non_serviced_df_2['ht_victim'] == 1])/ (len(non_serviced_df_2[non_serviced_df_2['ht_victim'] == 1])+ len(serviced_without_warmup_2[serviced_without_warmup_2['ht_victim'] == 1]))
    else:
        prop_abandoned_2_ht = 0      
        number_arriving_2_ht = 0
        
    if  len(serviced_without_warmup_2[serviced_without_warmup_2['ht_victim'] == 0])>0:
        number_arriving_2_not_ht = (len(non_serviced_df_2[non_serviced_df_2['ht_victim'] == 0])+ len(serviced_without_warmup_2[serviced_without_warmup_2['ht_victim'] == 0]))    
        prop_abandoned_2_not_ht = len(non_serviced_df_2[non_serviced_df_2['ht_victim'] == 0])/ (len(non_serviced_df_2[non_serviced_df_2['ht_victim'] == 0])+ len(serviced_without_warmup_2[serviced_without_warmup_2['ht_victim'] == 0]))
    else:
        prop_abandoned_2_not_ht = 0  
        number_arriving_2_not_ht = 0

else:
    number_arriving_2_female = len(serviced_without_warmup_2[serviced_without_warmup_2[' female'] == 1]) + len(non_serviced_df_2[non_serviced_df_2[' female'] == 1])
    number_arriving_2_male = len(serviced_without_warmup_2[serviced_without_warmup_2['male'] == 1]) + len(non_serviced_df_2[non_serviced_df_2['male'] == 1])
    number_arriving_2_noncis = (len(non_serviced_df_2[non_serviced_df_2['nonbinary'] == 1])+
                                     len(non_serviced_df_2[non_serviced_df_2['g_queer'] == 1])+
                                     len(non_serviced_df_2[non_serviced_df_2['trans_female'] == 1])+
                                     len(non_serviced_df_2[non_serviced_df_2['trans_male'] == 1])+
                                     len(serviced_without_warmup_2[serviced_without_warmup_2['g_queer'] == 1])+
                                     len(serviced_without_warmup_2[serviced_without_warmup_2['nonbinary'] == 1])+
                                    len(serviced_without_warmup_2[serviced_without_warmup_2['trans_female'] == 1])+
                                     len(serviced_without_warmup_2[serviced_without_warmup_2['trans_male'] == 1]))
    number_arriving_2_older21 = len(non_serviced_df_2[non_serviced_df_2['age'] > 21])+ len(serviced_without_warmup_2[serviced_without_warmup_2['age'] > 21])
    number_arriving_2_younger21 = (len(non_serviced_df_2[non_serviced_df_2['age'] <= 21])+ len(serviced_without_warmup_2[serviced_without_warmup_2['age'] <= 21]))
    number_arriving_2_child = (len(non_serviced_df_2[non_serviced_df_2['child'] == 1])+ len(serviced_without_warmup_2[serviced_without_warmup_2['child'] == 1]))
    number_arriving_2_no_child = (len(non_serviced_df_2[non_serviced_df_2['child'] == 0])+ len(serviced_without_warmup_2[serviced_without_warmup_2['child'] == 0]))
    number_arriving_2_immigrant = (len(non_serviced_df_2[non_serviced_df_2['immigrant'] == 1])+ len(serviced_without_warmup_2[serviced_without_warmup_2['immigrant'] == 1]))
    number_arriving_2_not_immigrant = (len(non_serviced_df_2[non_serviced_df_2['immigrant'] == 0])+ len(serviced_without_warmup_2[serviced_without_warmup_2['immigrant'] == 0]))
    number_arriving_2_ht = (len(non_serviced_df_2[non_serviced_df_2['ht_victim'] == 1])+ len(serviced_without_warmup_2[serviced_without_warmup_2['ht_victim'] == 1]))
    number_arriving_2_not_ht = (len(non_serviced_df_2[non_serviced_df_2['ht_victim'] == 0])+ len(serviced_without_warmup_2[serviced_without_warmup_2['ht_victim'] == 0]))
    
#Wait Times based on the vulnerability groups
avg_wait_2_female = serviced_without_warmup_2[serviced_without_warmup_2[' female'] == 1]['actual wait time'].mean(axis=0)
avg_wait_2_male = serviced_without_warmup_2[serviced_without_warmup_2['male'] == 1]['actual wait time'].mean(axis=0)
avg_wait_2_noncis = statistics.mean([serviced_without_warmup_2[serviced_without_warmup_2['trans_female'] == 1]['actual wait time'].mean(axis=0), 
                            serviced_without_warmup_2[serviced_without_warmup_2['trans_male'] == 1]['actual wait time'].mean(axis=0), 
                            serviced_without_warmup_2[serviced_without_warmup_2['nonbinary'] == 1]['actual wait time'].mean(axis=0), 
                            serviced_without_warmup_2[serviced_without_warmup_2['g_queer'] == 1]['actual wait time'].mean(axis=0)])
avg_wait_2_older21 = serviced_without_warmup_2[serviced_without_warmup_2['age'] > 21]['actual wait time'].mean(axis=0)
avg_wait_2_younger21 = serviced_without_warmup_2[serviced_without_warmup_2['age'] <= 21]['actual wait time'].mean(axis=0)
avg_wait_2_child = serviced_without_warmup_2[serviced_without_warmup_2['child'] == 1]['actual wait time'].mean(axis=0)
avg_wait_2_no_child = serviced_without_warmup_2[serviced_without_warmup_2['child'] == 0]['actual wait time'].mean(axis=0)
avg_wait_2_immigrant = serviced_without_warmup_2[serviced_without_warmup_2['immigrant'] == 1]['actual wait time'].mean(axis=0)
avg_wait_2_not_immigrant = serviced_without_warmup_2[serviced_without_warmup_2['immigrant'] == 0]['actual wait time'].mean(axis=0)
avg_wait_2_ht = serviced_without_warmup_2[serviced_without_warmup_2['ht_victim'] == 1]['actual wait time'].mean(axis=0)
avg_wait_2_not_ht = serviced_without_warmup_2[serviced_without_warmup_2['ht_victim'] == 0]['actual wait time'].mean(axis=0)

# print('----------------')
# print('__  Shelter 2 __')
# print('----------------')
# if prop_abandoned_2 >0:
#     print('-Abandonment')
#     print('Female: {:.4}'.format(float(prop_abandoned_2_female*100))+'% among their demographic group ')
#     print('Male: {:.4}'.format(float(prop_abandoned_2_male*100))+'% among their demographic group ')
#     print('Non cis gender: {:.4}'.format(float(prop_abandoned_2_noncis*100))+'% among their demographic group ')
#     print('Older than 21: {:.4}'.format(float(prop_abandoned_2_older21*100))+'% among their demographic group ')
#     print('Has child: {:.4}'.format(float(prop_abandoned_2_child*100))+'% among their demographic group ')
#     print('Immigrant: {:.4}'.format(float(prop_abandoned_2_immigrant*100))+'% among their demographic group ')
#     print('HT Victim: {:.4}'.format(float(prop_abandoned_2_ht*100))+'% among their demographic group ')
# else: 
#     print('No youth abandoned the system.')
    
    
# print('')
# print('-Average Wait')
# print('Female: {:.4}'.format(float(avg_wait_2_female))+' days')
# print('Male: {:.4}'.format(float(avg_wait_2_male))+' days')
# print('Non cis gender: {:.4}'.format(float(avg_wait_2_noncis))+' days')
# print('Older than 21: {:.4}'.format(float(avg_wait_2_older21))+' days')
# print('Has child: {:.4}'.format(float(avg_wait_2_child))+' days')
# print('Immigrant: {:.4}'.format(float(avg_wait_2_immigrant))+' days')
# print('HT Victim: {:.4}'.format(float(avg_wait_2_ht))+' days')

# Shelter 3

if prop_abandoned_3 >0:
    if len(serviced_without_warmup_3[serviced_without_warmup_3[' female'] == 1]) >0:
        prop_abandoned_3_female = len(non_serviced_df_3[non_serviced_df_3[' female'] == 1])/ (len(non_serviced_df_3[non_serviced_df_3[' female'] == 1])+ len(serviced_without_warmup_3[serviced_without_warmup_3[' female'] == 1]))
        number_arriving_3_female = len(serviced_without_warmup_3[serviced_without_warmup_3[' female'] == 1]) + len(non_serviced_df_3[non_serviced_df_3[' female'] == 1])
    else:
        prop_abandoned_3_female = 0
        number_arriving_3_female =0
        
    if len(serviced_without_warmup_3[serviced_without_warmup_3['male'] == 1]) >0:
        number_arriving_3_male = len(serviced_without_warmup_3[serviced_without_warmup_3['male'] == 1]) + len(non_serviced_df_3[non_serviced_df_3['male'] == 1])
        prop_abandoned_3_male   = len(non_serviced_df_3[non_serviced_df_3['male'] == 1])/ (len(non_serviced_df_3[non_serviced_df_3['male'] == 1])+ len(serviced_without_warmup_3[serviced_without_warmup_3['male'] == 1]))
    else:
        prop_abandoned_3_male   = 0
        number_arriving_3_male = 0
        
    if (len(serviced_without_warmup_3[serviced_without_warmup_3['g_queer'] == 1])+
                             len(serviced_without_warmup_3[serviced_without_warmup_3['nonbinary'] == 1])+
                            len(serviced_without_warmup_3[serviced_without_warmup_3['trans_female'] == 1])+
                             len(serviced_without_warmup_3[serviced_without_warmup_3['trans_male'] == 1])) >0:
        prop_abandoned_3_noncis = (len(non_serviced_df_3[non_serviced_df_3['nonbinary'] == 1])+
                                   len(non_serviced_df_3[non_serviced_df_3['g_queer'] == 1])+
                                   len(non_serviced_df_3[non_serviced_df_3['trans_female'] == 1])+
                                   len(non_serviced_df_3[non_serviced_df_3['trans_male'] == 1]))/ (len(non_serviced_df_3[non_serviced_df_3['nonbinary'] == 1])+
                                 len(non_serviced_df_3[non_serviced_df_3['g_queer'] == 1])+
                                 len(non_serviced_df_3[non_serviced_df_3['trans_female'] == 1])+
                                 len(non_serviced_df_3[non_serviced_df_3['trans_male'] == 1])+
                                 len(serviced_without_warmup_3[serviced_without_warmup_3['g_queer'] == 1])+
                                 len(serviced_without_warmup_3[serviced_without_warmup_3['nonbinary'] == 1])+
                                len(serviced_without_warmup_3[serviced_without_warmup_3['trans_female'] == 1])+
                                 len(serviced_without_warmup_3[serviced_without_warmup_3['trans_male'] == 1]))
        number_arriving_3_noncis = (len(non_serviced_df_3[non_serviced_df_3['nonbinary'] == 1])+
                                 len(non_serviced_df_3[non_serviced_df_3['g_queer'] == 1])+
                                 len(non_serviced_df_3[non_serviced_df_3['trans_female'] == 1])+
                                 len(non_serviced_df_3[non_serviced_df_3['trans_male'] == 1])+
                                 len(serviced_without_warmup_3[serviced_without_warmup_3['g_queer'] == 1])+
                                 len(serviced_without_warmup_3[serviced_without_warmup_3['nonbinary'] == 1])+
                                len(serviced_without_warmup_3[serviced_without_warmup_3['trans_female'] == 1])+
                                 len(serviced_without_warmup_3[serviced_without_warmup_3['trans_male'] == 1]))
                                                                                                                                                                            
    else:
        prop_abandoned_3_noncis = 0
        number_arriving_3_noncis = 0
        
    if len(serviced_without_warmup_3[serviced_without_warmup_3['age'] > 21])>0:
        number_arriving_3_older21 = len(non_serviced_df_3[non_serviced_df_3['age'] > 21])+ len(serviced_without_warmup_3[serviced_without_warmup_3['age'] > 21])
        prop_abandoned_3_older21 = len(non_serviced_df_3[non_serviced_df_3['age'] > 21])/ (len(non_serviced_df_3[non_serviced_df_3['age'] > 21])+ len(serviced_without_warmup_3[serviced_without_warmup_3['age'] > 21]))
    else:
        prop_abandoned_3_older21 = 0
        number_arriving_3_older21 = 0
        
    if len(serviced_without_warmup_3[serviced_without_warmup_3['age'] <= 21])>0:
        number_arriving_3_younger21 = (len(non_serviced_df_3[non_serviced_df_3['age'] <= 21])+ len(serviced_without_warmup_3[serviced_without_warmup_3['age'] <= 21]))
        prop_abandoned_3_younger21 = len(non_serviced_df_3[non_serviced_df_3['age'] <= 21])/ (len(non_serviced_df_3[non_serviced_df_3['age'] <= 21])+ len(serviced_without_warmup_3[serviced_without_warmup_3['age'] <= 21]))
    else:
        prop_abandoned_3_younger21 = 0
        number_arriving_3_younger21 = 0
        
    if len(serviced_without_warmup_3[serviced_without_warmup_3['child'] == 1]) >0:
        number_arriving_3_child = (len(non_serviced_df_3[non_serviced_df_3['child'] == 1])+ len(serviced_without_warmup_3[serviced_without_warmup_3['child'] == 1]))
        prop_abandoned_3_child = len(non_serviced_df_3[non_serviced_df_3['child'] == 1])/ (len(non_serviced_df_3[non_serviced_df_3['child'] == 1])+ len(serviced_without_warmup_3[serviced_without_warmup_3['child'] == 1]))
    else:
        prop_abandoned_3_child = 0
        number_arriving_3_child = 0
        
    if len(serviced_without_warmup_3[serviced_without_warmup_3['child'] == 0]) >0:
        number_arriving_3_no_child = (len(non_serviced_df_3[non_serviced_df_3['child'] == 0])+ len(serviced_without_warmup_3[serviced_without_warmup_3['child'] == 0]))
        prop_abandoned_3_no_child = len(non_serviced_df_3[non_serviced_df_3['child'] == 0])/ (len(non_serviced_df_3[non_serviced_df_3['child'] == 0])+ len(serviced_without_warmup_3[serviced_without_warmup_3['child'] == 0]))
    else:
        prop_abandoned_3_no_child = 0
        number_arriving_3_no_child = 0
                                                                                           
    if len(serviced_without_warmup_3[serviced_without_warmup_3['immigrant'] == 1])>0:     
        number_arriving_3_immigrant = (len(non_serviced_df_3[non_serviced_df_3['immigrant'] == 1])+ len(serviced_without_warmup_3[serviced_without_warmup_3['immigrant'] == 1]))  
        prop_abandoned_3_immigrant = len(non_serviced_df_3[non_serviced_df_3['immigrant'] == 1])/ (len(non_serviced_df_3[non_serviced_df_3['immigrant'] == 1])+ len(serviced_without_warmup_3[serviced_without_warmup_3['immigrant'] == 1]))
    else:
        prop_abandoned_3_immigrant = 0
        number_arriving_3_immigrant = 0
        
    if len(serviced_without_warmup_3[serviced_without_warmup_3['immigrant'] == 0])>0:     
        number_arriving_3_not_immigrant = (len(non_serviced_df_3[non_serviced_df_3['immigrant'] == 0])+ len(serviced_without_warmup_3[serviced_without_warmup_3['immigrant'] == 0]))
        prop_abandoned_3_not_immigrant = len(non_serviced_df_3[non_serviced_df_3['immigrant'] == 0])/ (len(non_serviced_df_3[non_serviced_df_3['immigrant'] == 0])+ len(serviced_without_warmup_3[serviced_without_warmup_3['immigrant'] == 0]))
    else:
        prop_abandoned_3_not_immigrant = 0
        number_arriving_3_not_immigrant = 0
                                                                                           
    if  len(serviced_without_warmup_3[serviced_without_warmup_3['ht_victim'] == 1])>0:
        number_arriving_3_ht = (len(non_serviced_df_3[non_serviced_df_3['ht_victim'] == 1])+ len(serviced_without_warmup_3[serviced_without_warmup_3['ht_victim'] == 1]))
        prop_abandoned_3_ht = len(non_serviced_df_3[non_serviced_df_3['ht_victim'] == 1])/ (len(non_serviced_df_3[non_serviced_df_3['ht_victim'] == 1])+ len(serviced_without_warmup_3[serviced_without_warmup_3['ht_victim'] == 1]))
    else:
        prop_abandoned_3_ht = 0  
        number_arriving_3_ht = 0
        
    if  len(serviced_without_warmup_3[serviced_without_warmup_3['ht_victim'] == 0])>0:
        number_arriving_3_not_ht = (len(non_serviced_df_3[non_serviced_df_3['ht_victim'] == 0])+ len(serviced_without_warmup_3[serviced_without_warmup_3['ht_victim'] == 0]))    
        prop_abandoned_3_not_ht = len(non_serviced_df_3[non_serviced_df_3['ht_victim'] == 0])/ (len(non_serviced_df_3[non_serviced_df_3['ht_victim'] == 0])+ len(serviced_without_warmup_3[serviced_without_warmup_3['ht_victim'] == 0]))
    else:
        prop_abandoned_3_not_ht = 0  
        number_arriving_3_not_ht = 0

else:
    number_arriving_3_female = len(serviced_without_warmup_3[serviced_without_warmup_3[' female'] == 1]) + len(non_serviced_df_3[non_serviced_df_3[' female'] == 1])
    number_arriving_3_male = len(serviced_without_warmup_3[serviced_without_warmup_3['male'] == 1]) + len(non_serviced_df_3[non_serviced_df_3['male'] == 1])
    number_arriving_3_noncis = (len(non_serviced_df_3[non_serviced_df_3['nonbinary'] == 1])+
                                     len(non_serviced_df_3[non_serviced_df_3['g_queer'] == 1])+
                                     len(non_serviced_df_3[non_serviced_df_3['trans_female'] == 1])+
                                     len(non_serviced_df_3[non_serviced_df_3['trans_male'] == 1])+
                                     len(serviced_without_warmup_3[serviced_without_warmup_3['g_queer'] == 1])+
                                     len(serviced_without_warmup_3[serviced_without_warmup_3['nonbinary'] == 1])+
                                    len(serviced_without_warmup_3[serviced_without_warmup_3['trans_female'] == 1])+
                                     len(serviced_without_warmup_3[serviced_without_warmup_3['trans_male'] == 1]))
    number_arriving_3_older21 = len(non_serviced_df_3[non_serviced_df_3['age'] > 21])+ len(serviced_without_warmup_3[serviced_without_warmup_3['age'] > 21])
    number_arriving_3_younger21 = (len(non_serviced_df_3[non_serviced_df_3['age'] <= 21])+ len(serviced_without_warmup_3[serviced_without_warmup_3['age'] <= 21]))
    number_arriving_3_child = (len(non_serviced_df_3[non_serviced_df_3['child'] == 1])+ len(serviced_without_warmup_3[serviced_without_warmup_3['child'] == 1]))
    number_arriving_3_no_child = (len(non_serviced_df_3[non_serviced_df_3['child'] == 0])+ len(serviced_without_warmup_3[serviced_without_warmup_3['child'] == 0]))
    number_arriving_3_immigrant = (len(non_serviced_df_3[non_serviced_df_3['immigrant'] == 1])+ len(serviced_without_warmup_3[serviced_without_warmup_3['immigrant'] == 1]))
    number_arriving_3_not_immigrant = (len(non_serviced_df_3[non_serviced_df_3['immigrant'] == 0])+ len(serviced_without_warmup_3[serviced_without_warmup_3['immigrant'] == 0]))
    number_arriving_3_ht = (len(non_serviced_df_3[non_serviced_df_3['ht_victim'] == 1])+ len(serviced_without_warmup_3[serviced_without_warmup_3['ht_victim'] == 1]))
    number_arriving_3_not_ht = (len(non_serviced_df_3[non_serviced_df_3['ht_victim'] == 0])+ len(serviced_without_warmup_3[serviced_without_warmup_3['ht_victim'] == 0]))
    
        
    
#Wait Times based on the vulnerability groups
avg_wait_3_female = serviced_without_warmup_3[serviced_without_warmup_3[' female'] == 1]['actual wait time'].mean(axis=0)
avg_wait_3_male = serviced_without_warmup_3[serviced_without_warmup_3['male'] == 1]['actual wait time'].mean(axis=0)
avg_wait_3_noncis = statistics.mean([serviced_without_warmup_3[serviced_without_warmup_3['trans_female'] == 1]['actual wait time'].mean(axis=0), 
                            serviced_without_warmup_3[serviced_without_warmup_3['trans_male'] == 1]['actual wait time'].mean(axis=0), 
                            serviced_without_warmup_3[serviced_without_warmup_3['nonbinary'] == 1]['actual wait time'].mean(axis=0), 
                            serviced_without_warmup_3[serviced_without_warmup_3['g_queer'] == 1]['actual wait time'].mean(axis=0)])
avg_wait_3_older21 = serviced_without_warmup_3[serviced_without_warmup_3['age'] > 21]['actual wait time'].mean(axis=0)
avg_wait_3_younger21 = serviced_without_warmup_3[serviced_without_warmup_3['age'] <= 21]['actual wait time'].mean(axis=0)
avg_wait_3_child = serviced_without_warmup_3[serviced_without_warmup_3['child'] == 1]['actual wait time'].mean(axis=0)
avg_wait_3_no_child = serviced_without_warmup_3[serviced_without_warmup_3['child'] == 0]['actual wait time'].mean(axis=0)
avg_wait_3_immigrant = serviced_without_warmup_3[serviced_without_warmup_3['immigrant'] == 1]['actual wait time'].mean(axis=0)
avg_wait_3_not_immigrant = serviced_without_warmup_3[serviced_without_warmup_3['immigrant'] == 0]['actual wait time'].mean(axis=0)
avg_wait_3_ht = serviced_without_warmup_3[serviced_without_warmup_3['ht_victim'] == 1]['actual wait time'].mean(axis=0)
avg_wait_3_not_ht = serviced_without_warmup_3[serviced_without_warmup_3['ht_victim'] == 0]['actual wait time'].mean(axis=0)


# print('----------------')
# print('__  Shelter 3 __')
# print('----------------')
# if prop_abandoned_3 >0:
#     print('-Abandonment')
#     print('Female: {:.4}'.format(float(prop_abandoned_3_female*100))+'% among their demographic group ')
#     print('Male: {:.4}'.format(float(prop_abandoned_3_male*100))+'% among their demographic group ')
#     print('Non cis gender: {:.4}'.format(float(prop_abandoned_3_noncis*100))+'% among their demographic group ')
#     print('Older than 21: {:.4}'.format(float(prop_abandoned_3_older21*100))+'% among their demographic group ')
#     print('Has child: {:.4}'.format(float(prop_abandoned_3_child*100))+'% among their demographic group ')
#     print('Immigrant: {:.4}'.format(float(prop_abandoned_3_immigrant*100))+'% among their demographic group ')
#     print('HT Victim: {:.4}'.format(float(prop_abandoned_3_ht*100))+'% among their demographic group ')
# else: 
#     print('No youth abandoned the system.')
    
    
# print('')
# print('-Average Wait')
# print('Female: {:.4}'.format(float(avg_wait_3_female))+' days')
# print('Male: {:.4}'.format(float(avg_wait_3_male))+' days')
# print('Non cis gender: {:.4}'.format(float(avg_wait_3_noncis))+' days')
# print('Older than 21: {:.4}'.format(float(avg_wait_3_older21))+' days')
# print('Has child: {:.4}'.format(float(avg_wait_3_child))+' days')
# print('Immigrant: {:.4}'.format(float(avg_wait_3_immigrant))+' days')
# print('HT Victim: {:.4}'.format(float(avg_wait_3_ht))+' days')

# All shelters

if (len(serviced_without_warmup_0[serviced_without_warmup_0[' female'] == 1]) + len(serviced_without_warmup_1[serviced_without_warmup_1[' female'] == 1])+
    len(serviced_without_warmup_2[serviced_without_warmup_2[' female'] == 1])+ len(serviced_without_warmup_3[serviced_without_warmup_3[' female'] == 1]))>0:
    
    prop_abandoned_all_female = (len(non_serviced_df_0[non_serviced_df_0[' female'] == 1]) + 
                                              len(non_serviced_df_1[non_serviced_df_1[' female'] == 1])+
                                                len(non_serviced_df_2[non_serviced_df_2[' female'] == 1])+ 
                                              len(non_serviced_df_3[non_serviced_df_3[' female'] == 1]))/ (len(serviced_without_warmup_0[serviced_without_warmup_0[' female'] == 1]) + 
                                              len(serviced_without_warmup_1[serviced_without_warmup_1[' female'] == 1])+
                                                len(serviced_without_warmup_2[serviced_without_warmup_2[' female'] == 1])+ 
                                              len(serviced_without_warmup_3[serviced_without_warmup_3[' female'] == 1])+len(non_serviced_df_0[non_serviced_df_0[' female'] == 1]) + 
                                              len(non_serviced_df_1[non_serviced_df_1[' female'] == 1])+
                                                len(non_serviced_df_2[non_serviced_df_2[' female'] == 1])+ 
                                              len(non_serviced_df_3[non_serviced_df_3[' female'] == 1]))
                                            
else:
    prop_abandoned_all_female = 0

    
if (len(serviced_without_warmup_0[serviced_without_warmup_0['male'] == 1]) + len(serviced_without_warmup_1[serviced_without_warmup_1['male'] == 1])+
    len(serviced_without_warmup_2[serviced_without_warmup_2['male'] == 1])+ len(serviced_without_warmup_3[serviced_without_warmup_3['male'] == 1]))>0:
    
    prop_abandoned_all_male   = (len(non_serviced_df_0[non_serviced_df_0['male'] == 1]) + 
                                            len(non_serviced_df_1[non_serviced_df_1['male'] == 1])+
                                            len(non_serviced_df_2[non_serviced_df_2['male'] == 1])+ 
                                            len(non_serviced_df_3[non_serviced_df_3['male'] == 1]))/ (len(serviced_without_warmup_0[serviced_without_warmup_0['male'] == 1]) + 
                                            len(serviced_without_warmup_1[serviced_without_warmup_1['male'] == 1])+
                                            len(serviced_without_warmup_2[serviced_without_warmup_2['male'] == 1])+ 
                                            len(serviced_without_warmup_3[serviced_without_warmup_3['male'] == 1])+len(non_serviced_df_0[non_serviced_df_0['male'] == 1]) + 
                                            len(non_serviced_df_1[non_serviced_df_1['male'] == 1])+
                                            len(non_serviced_df_2[non_serviced_df_2['male'] == 1])+ 
                                            len(non_serviced_df_3[non_serviced_df_3['male'] == 1]))
else:
    prop_abandoned_all_male   = 0
        
if (len(serviced_without_warmup_3[serviced_without_warmup_3['g_queer'] == 1])+
                            len(serviced_without_warmup_3[serviced_without_warmup_3['nonbinary'] == 1])+
                        len(serviced_without_warmup_3[serviced_without_warmup_3['trans_female'] == 1])+
                            len(serviced_without_warmup_3[serviced_without_warmup_3['trans_male'] == 1])+
   len(serviced_without_warmup_2[serviced_without_warmup_2['g_queer'] == 1])+
                            len(serviced_without_warmup_2[serviced_without_warmup_2['nonbinary'] == 1])+
                        len(serviced_without_warmup_2[serviced_without_warmup_2['trans_female'] == 1])+
                            len(serviced_without_warmup_2[serviced_without_warmup_2['trans_male'] == 1])+
   len(serviced_without_warmup_1[serviced_without_warmup_1['g_queer'] == 1])+
                            len(serviced_without_warmup_1[serviced_without_warmup_1['nonbinary'] == 1])+
                        len(serviced_without_warmup_1[serviced_without_warmup_1['trans_female'] == 1])+
                            len(serviced_without_warmup_1[serviced_without_warmup_1['trans_male'] == 1])+
      len(serviced_without_warmup_0[serviced_without_warmup_0['g_queer'] == 1])+
                            len(serviced_without_warmup_0[serviced_without_warmup_0['nonbinary'] == 1])+
                        len(serviced_without_warmup_0[serviced_without_warmup_0['trans_female'] == 1])+
                            len(serviced_without_warmup_0[serviced_without_warmup_0['trans_male'] == 1])) >0:
    
    
    
    prop_abandoned_all_noncis = (len(non_serviced_df_3[non_serviced_df_3['g_queer'] == 1])+
                            len(non_serviced_df_3[non_serviced_df_3['nonbinary'] == 1])+
                        len(non_serviced_df_3[non_serviced_df_3['trans_female'] == 1])+
                            len(non_serviced_df_3[non_serviced_df_3['trans_male'] == 1])+
   len(non_serviced_df_2[non_serviced_df_2['g_queer'] == 1])+
                            len(non_serviced_df_2[non_serviced_df_2['nonbinary'] == 1])+
                        len(non_serviced_df_2[non_serviced_df_2['trans_female'] == 1])+
                            len(non_serviced_df_2[non_serviced_df_2['trans_male'] == 1])+
   len(non_serviced_df_1[non_serviced_df_1['g_queer'] == 1])+
                            len(non_serviced_df_1[non_serviced_df_1['nonbinary'] == 1])+
                        len(non_serviced_df_1[non_serviced_df_1['trans_female'] == 1])+
                            len(non_serviced_df_1[non_serviced_df_1['trans_male'] == 1])+
      len(non_serviced_df_0[non_serviced_df_0['g_queer'] == 1])+
                            len(non_serviced_df_0[non_serviced_df_0['nonbinary'] == 1])+
                        len(non_serviced_df_0[non_serviced_df_0['trans_female'] == 1])+
                            len(non_serviced_df_0[non_serviced_df_0['trans_male'] == 1]))/ (len(serviced_without_warmup_3[serviced_without_warmup_3['g_queer'] == 1])+
                            len(serviced_without_warmup_3[serviced_without_warmup_3['nonbinary'] == 1])+
                        len(serviced_without_warmup_3[serviced_without_warmup_3['trans_female'] == 1])+
                            len(serviced_without_warmup_3[serviced_without_warmup_3['trans_male'] == 1])+
   len(serviced_without_warmup_2[serviced_without_warmup_2['g_queer'] == 1])+
                            len(serviced_without_warmup_2[serviced_without_warmup_2['nonbinary'] == 1])+
                        len(serviced_without_warmup_2[serviced_without_warmup_2['trans_female'] == 1])+
                            len(serviced_without_warmup_2[serviced_without_warmup_2['trans_male'] == 1])+
   len(serviced_without_warmup_1[serviced_without_warmup_1['g_queer'] == 1])+
                            len(serviced_without_warmup_1[serviced_without_warmup_1['nonbinary'] == 1])+
                        len(serviced_without_warmup_1[serviced_without_warmup_1['trans_female'] == 1])+
                            len(serviced_without_warmup_1[serviced_without_warmup_1['trans_male'] == 1])+
      len(serviced_without_warmup_0[serviced_without_warmup_0['g_queer'] == 1])+
                            len(serviced_without_warmup_0[serviced_without_warmup_0['nonbinary'] == 1])+
                        len(serviced_without_warmup_0[serviced_without_warmup_0['trans_female'] == 1])+
                            len(serviced_without_warmup_0[serviced_without_warmup_0['trans_male'] == 1])+(len(non_serviced_df_3[non_serviced_df_3['g_queer'] == 1])+
                            len(non_serviced_df_3[non_serviced_df_3['nonbinary'] == 1])+
                        len(non_serviced_df_3[non_serviced_df_3['trans_female'] == 1])+
                            len(non_serviced_df_3[non_serviced_df_3['trans_male'] == 1])+
   len(non_serviced_df_2[non_serviced_df_2['g_queer'] == 1])+
                            len(non_serviced_df_2[non_serviced_df_2['nonbinary'] == 1])+
                        len(non_serviced_df_2[non_serviced_df_2['trans_female'] == 1])+
                            len(non_serviced_df_2[non_serviced_df_2['trans_male'] == 1])+
   len(non_serviced_df_1[non_serviced_df_1['g_queer'] == 1])+
                            len(non_serviced_df_1[non_serviced_df_1['nonbinary'] == 1])+
                        len(non_serviced_df_1[non_serviced_df_1['trans_female'] == 1])+
                            len(non_serviced_df_1[non_serviced_df_1['trans_male'] == 1])+
      len(non_serviced_df_0[non_serviced_df_0['g_queer'] == 1])+
                            len(non_serviced_df_0[non_serviced_df_0['nonbinary'] == 1])+
                        len(non_serviced_df_0[non_serviced_df_0['trans_female'] == 1])+
                            len(non_serviced_df_0[non_serviced_df_0['trans_male'] == 1])))
else:
    prop_abandoned_all_noncis = 0
        
if (len(serviced_without_warmup_3[serviced_without_warmup_3['age'] > 21]) + len(serviced_without_warmup_2[serviced_without_warmup_2['age'] > 21])+
    len(serviced_without_warmup_1[serviced_without_warmup_1['age'] > 21]) +len(serviced_without_warmup_0[serviced_without_warmup_0['age'] > 21]))>0:
    
    prop_abandoned_all_older21 = (len(non_serviced_df_3[non_serviced_df_3['age'] > 21]) + 
                                               len(non_serviced_df_2[non_serviced_df_2['age'] > 21])+
                                                len(non_serviced_df_1[non_serviced_df_1['age'] > 21]) +
                                               len(non_serviced_df_0[non_serviced_df_0['age'] > 21]))/ (len(serviced_without_warmup_3[serviced_without_warmup_3['age'] > 21]) + 
                                               len(serviced_without_warmup_2[serviced_without_warmup_2['age'] > 21])+
                                                len(serviced_without_warmup_1[serviced_without_warmup_1['age'] > 21]) +
                                               len(serviced_without_warmup_0[serviced_without_warmup_0['age'] > 21])+
                                               len(non_serviced_df_3[non_serviced_df_3['age'] > 21]) + 
                                               len(non_serviced_df_2[non_serviced_df_2['age'] > 21])+
                                                len(non_serviced_df_1[non_serviced_df_1['age'] > 21]) +
                                               len(non_serviced_df_0[non_serviced_df_0['age'] > 21]))
else:
    prop_abandoned_all_older21 = 0
    
if (len(serviced_without_warmup_3[serviced_without_warmup_3['age'] <= 21]) + len(serviced_without_warmup_2[serviced_without_warmup_2['age'] <= 21])+
    len(serviced_without_warmup_1[serviced_without_warmup_1['age'] <= 21]) +len(serviced_without_warmup_0[serviced_without_warmup_0['age'] <= 21]))>0:
    
    prop_abandoned_all_younger21 = (len(non_serviced_df_3[non_serviced_df_3['age'] <= 21]) + 
                                               len(non_serviced_df_2[non_serviced_df_2['age'] <= 21])+
                                                len(non_serviced_df_1[non_serviced_df_1['age'] <= 21]) +
                                               len(non_serviced_df_0[non_serviced_df_0['age'] <= 21]))/ (len(serviced_without_warmup_3[serviced_without_warmup_3['age'] <= 21]) + 
                                               len(serviced_without_warmup_2[serviced_without_warmup_2['age'] <= 21])+
                                                len(serviced_without_warmup_1[serviced_without_warmup_1['age'] <= 21]) +
                                               len(serviced_without_warmup_0[serviced_without_warmup_0['age'] <= 21])+
                                               len(non_serviced_df_3[non_serviced_df_3['age'] <= 21]) + 
                                               len(non_serviced_df_2[non_serviced_df_2['age'] <= 21])+
                                                len(non_serviced_df_1[non_serviced_df_1['age'] <= 21]) +
                                               len(non_serviced_df_0[non_serviced_df_0['age'] <= 21]))
else:
    prop_abandoned_all_younger21 = 0   
    
 
if (len(serviced_without_warmup_3[serviced_without_warmup_3['child'] == 1]) +
    len(serviced_without_warmup_2[serviced_without_warmup_2['child'] == 1]) +
    len(serviced_without_warmup_1[serviced_without_warmup_1['child'] == 1])+ 
    len(serviced_without_warmup_0[serviced_without_warmup_0['child'] == 1]))>0:
    
    prop_abandoned_all_child = (len(non_serviced_df_3[non_serviced_df_3['child'] == 1]) +
    len(non_serviced_df_2[non_serviced_df_2['child'] == 1]) +
    len(non_serviced_df_1[non_serviced_df_1['child'] == 1])+ 
    len(non_serviced_df_0[non_serviced_df_0['child'] == 1]))/ (len(serviced_without_warmup_3[serviced_without_warmup_3['child'] == 1]) +
    len(serviced_without_warmup_2[serviced_without_warmup_2['child'] == 1]) +
    len(serviced_without_warmup_1[serviced_without_warmup_1['child'] == 1])+ 
    len(serviced_without_warmup_0[serviced_without_warmup_0['child'] == 1])+
    len(non_serviced_df_3[non_serviced_df_3['child'] == 1]) +
    len(non_serviced_df_2[non_serviced_df_2['child'] == 1]) +
    len(non_serviced_df_1[non_serviced_df_1['child'] == 1])+ 
    len(non_serviced_df_0[non_serviced_df_0['child'] == 1]))

else:
    prop_abandoned_all_child = 0
    
if (len(serviced_without_warmup_3[serviced_without_warmup_3['child'] == 0]) +
    len(serviced_without_warmup_2[serviced_without_warmup_2['child'] == 0]) +
    len(serviced_without_warmup_1[serviced_without_warmup_1['child'] == 0])+ 
    len(serviced_without_warmup_0[serviced_without_warmup_0['child'] == 0]))>0:
    
    prop_abandoned_all_no_child = (len(non_serviced_df_3[non_serviced_df_3['child'] == 0]) +
    len(non_serviced_df_2[non_serviced_df_2['child'] == 0]) +
    len(non_serviced_df_1[non_serviced_df_1['child'] == 0])+ 
    len(non_serviced_df_0[non_serviced_df_0['child'] == 0]))/ (len(serviced_without_warmup_3[serviced_without_warmup_3['child'] == 0]) +
    len(serviced_without_warmup_2[serviced_without_warmup_2['child'] == 0]) +
    len(serviced_without_warmup_1[serviced_without_warmup_1['child'] == 0])+ 
    len(serviced_without_warmup_0[serviced_without_warmup_0['child'] == 0])+
    len(non_serviced_df_3[non_serviced_df_3['child'] == 0]) +
    len(non_serviced_df_2[non_serviced_df_2['child'] == 0]) +
    len(non_serviced_df_1[non_serviced_df_1['child'] == 0])+ 
    len(non_serviced_df_0[non_serviced_df_0['child'] == 0]))

else:
    prop_abandoned_all_no_child = 0
                                                                                           
if (len(serviced_without_warmup_3[serviced_without_warmup_3['immigrant'] == 1]) +
    len(serviced_without_warmup_2[serviced_without_warmup_2['immigrant'] == 1])+
    len(serviced_without_warmup_1[serviced_without_warmup_1['immigrant'] == 1])+
    len(serviced_without_warmup_0[serviced_without_warmup_0['immigrant'] == 1]))>0:                                                                                  
    prop_abandoned_all_immigrant = (len(non_serviced_df_3[non_serviced_df_3['immigrant'] == 1]) +
                                                len(non_serviced_df_2[non_serviced_df_2['immigrant'] == 1])+
                                                len(non_serviced_df_1[non_serviced_df_1['immigrant'] == 1])+
                                                len(non_serviced_df_0[non_serviced_df_0['immigrant'] == 1]))/ (len(serviced_without_warmup_3[serviced_without_warmup_3['immigrant'] == 1]) +
                                                len(serviced_without_warmup_2[serviced_without_warmup_2['immigrant'] == 1])+
                                                len(serviced_without_warmup_1[serviced_without_warmup_1['immigrant'] == 1])+
                                                len(serviced_without_warmup_0[serviced_without_warmup_0['immigrant'] == 1])+
                                                len(non_serviced_df_3[non_serviced_df_3['immigrant'] == 1]) +
                                                len(non_serviced_df_2[non_serviced_df_2['immigrant'] == 1])+
                                                len(non_serviced_df_1[non_serviced_df_1['immigrant'] == 1])+
                                                len(non_serviced_df_0[non_serviced_df_0['immigrant'] == 1]))

else:
    prop_abandoned_all_immigrant = 0
    
if (len(serviced_without_warmup_3[serviced_without_warmup_3['immigrant'] == 0]) +
    len(serviced_without_warmup_2[serviced_without_warmup_2['immigrant'] == 0])+
    len(serviced_without_warmup_1[serviced_without_warmup_1['immigrant'] == 0])+
    len(serviced_without_warmup_0[serviced_without_warmup_0['immigrant'] == 0]))>0:                                                                                  
    prop_abandoned_all_not_immigrant = (len(non_serviced_df_3[non_serviced_df_3['immigrant'] == 0]) +
                                                len(non_serviced_df_2[non_serviced_df_2['immigrant'] == 0])+
                                                len(non_serviced_df_1[non_serviced_df_1['immigrant'] == 0])+
                                                len(non_serviced_df_0[non_serviced_df_0['immigrant'] == 0]))/ (len(serviced_without_warmup_3[serviced_without_warmup_3['immigrant'] == 0]) +
                                                len(serviced_without_warmup_2[serviced_without_warmup_2['immigrant'] == 0])+
                                                len(serviced_without_warmup_1[serviced_without_warmup_1['immigrant'] == 0])+
                                                len(serviced_without_warmup_0[serviced_without_warmup_0['immigrant'] == 0])+
                                                len(non_serviced_df_3[non_serviced_df_3['immigrant'] == 0]) +
                                                len(non_serviced_df_2[non_serviced_df_2['immigrant'] == 0])+
                                                len(non_serviced_df_1[non_serviced_df_1['immigrant'] == 0])+
                                                len(non_serviced_df_0[non_serviced_df_0['immigrant'] == 0]))

else:
    prop_abandoned_all_not_immigrant = 0
                                                                                           
if  (len(serviced_without_warmup_3[serviced_without_warmup_3['ht_victim'] == 1])+
    len(serviced_without_warmup_2[serviced_without_warmup_2['ht_victim'] == 1])+
    len(serviced_without_warmup_1[serviced_without_warmup_1['ht_victim'] == 1])+
    len(serviced_without_warmup_0[serviced_without_warmup_0['ht_victim'] == 1]))>0:
    prop_abandoned_all_ht = (len(non_serviced_df_3[non_serviced_df_3['ht_victim'] == 1])+
                                        len(non_serviced_df_2[non_serviced_df_2['ht_victim'] == 1])+
                                        len(non_serviced_df_1[non_serviced_df_1['ht_victim'] == 1])+
                                        len(non_serviced_df_0[non_serviced_df_0['ht_victim'] == 1]))/ (len(serviced_without_warmup_3[serviced_without_warmup_3['ht_victim'] == 1])+
                                        len(serviced_without_warmup_2[serviced_without_warmup_2['ht_victim'] == 1])+
                                        len(serviced_without_warmup_1[serviced_without_warmup_1['ht_victim'] == 1])+
                                        len(serviced_without_warmup_0[serviced_without_warmup_0['ht_victim'] == 1])+
                                        len(non_serviced_df_3[non_serviced_df_3['ht_victim'] == 1])+
                                        len(non_serviced_df_2[non_serviced_df_2['ht_victim'] == 1])+
                                        len(non_serviced_df_1[non_serviced_df_1['ht_victim'] == 1])+
                                        len(non_serviced_df_0[non_serviced_df_0['ht_victim'] == 1]))
else:
    prop_abandoned_all_ht = 0
    
if  (len(serviced_without_warmup_3[serviced_without_warmup_3['ht_victim'] == 0])+
    len(serviced_without_warmup_2[serviced_without_warmup_2['ht_victim'] == 0])+
    len(serviced_without_warmup_1[serviced_without_warmup_1['ht_victim'] == 0])+
    len(serviced_without_warmup_0[serviced_without_warmup_0['ht_victim'] == 0]))>0:
    prop_abandoned_all_not_ht = (len(non_serviced_df_3[non_serviced_df_3['ht_victim'] == 0])+
                                        len(non_serviced_df_2[non_serviced_df_2['ht_victim'] == 0])+
                                        len(non_serviced_df_1[non_serviced_df_1['ht_victim'] == 0])+
                                        len(non_serviced_df_0[non_serviced_df_0['ht_victim'] == 0]))/ (len(serviced_without_warmup_3[serviced_without_warmup_3['ht_victim'] == 0])+
                                        len(serviced_without_warmup_2[serviced_without_warmup_2['ht_victim'] == 0])+
                                        len(serviced_without_warmup_1[serviced_without_warmup_1['ht_victim'] == 0])+
                                        len(serviced_without_warmup_0[serviced_without_warmup_0['ht_victim'] == 0])+
                                        len(non_serviced_df_3[non_serviced_df_3['ht_victim'] == 0])+
                                        len(non_serviced_df_2[non_serviced_df_2['ht_victim'] == 0])+
                                        len(non_serviced_df_1[non_serviced_df_1['ht_victim'] == 0])+
                                        len(non_serviced_df_0[non_serviced_df_0['ht_victim'] == 0]))
else:
    prop_abandoned_all_not_ht = 0
                                    
# Number of youth arriving from different demographic groups
                                    
number_arriving_all_female = number_arriving_0_female + number_arriving_1_female +number_arriving_2_female+number_arriving_3_female
number_arriving_all_male = number_arriving_0_male + number_arriving_1_male+ number_arriving_2_male+ number_arriving_3_male
number_arriving_all_noncis = number_arriving_0_noncis+number_arriving_1_noncis+number_arriving_2_noncis+number_arriving_3_noncis
                                                                                                                                                                             
number_arriving_all_older21 = number_arriving_0_older21+number_arriving_1_older21+number_arriving_2_older21+number_arriving_3_older21
number_arriving_all_younger21 = number_arriving_0_younger21 +number_arriving_1_younger21 +number_arriving_2_younger21 +number_arriving_3_younger21 
number_arriving_all_child = number_arriving_0_child+number_arriving_1_child+number_arriving_2_child+number_arriving_3_child
number_arriving_all_no_child = number_arriving_0_no_child+number_arriving_1_no_child+number_arriving_2_no_child+number_arriving_3_no_child
number_arriving_all_immigrant = number_arriving_0_immigrant +number_arriving_1_immigrant +number_arriving_2_immigrant +number_arriving_3_immigrant 
number_arriving_all_not_immigrant = number_arriving_0_not_immigrant +number_arriving_1_not_immigrant +number_arriving_2_not_immigrant +number_arriving_3_not_immigrant 
number_arriving_all_ht = number_arriving_0_ht +number_arriving_1_ht +number_arriving_2_ht +number_arriving_3_ht
number_arriving_all_not_ht = number_arriving_0_not_ht+number_arriving_1_not_ht+number_arriving_2_not_ht+number_arriving_3_not_ht
    
    
#Average wait times for the system itself

system_female_wait = [*serviced_without_warmup_3[serviced_without_warmup_3[' female'] == 1]['actual wait time'], 
                      *serviced_without_warmup_2[serviced_without_warmup_2[' female'] == 1]['actual wait time'], 
                     *serviced_without_warmup_1[serviced_without_warmup_1[' female'] == 1]['actual wait time'],
                     *serviced_without_warmup_0[serviced_without_warmup_0[' female'] == 1]['actual wait time']]

avg_wait_all_female = mean(system_female_wait)

system_male_wait = [*serviced_without_warmup_3[serviced_without_warmup_3['male'] == 1]['actual wait time'], 
                      *serviced_without_warmup_2[serviced_without_warmup_2['male'] == 1]['actual wait time'], 
                     *serviced_without_warmup_1[serviced_without_warmup_1['male'] == 1]['actual wait time'],
                     *serviced_without_warmup_0[serviced_without_warmup_0['male'] == 1]['actual wait time']]

avg_wait_all_male = mean(system_male_wait)

system_noncis_wait = [*serviced_without_warmup_3[serviced_without_warmup_3['trans_female'] == 1]['actual wait time'], 
                      *serviced_without_warmup_2[serviced_without_warmup_2['trans_female'] == 1]['actual wait time'], 
                     *serviced_without_warmup_1[serviced_without_warmup_1['trans_female'] == 1]['actual wait time'],
                     *serviced_without_warmup_0[serviced_without_warmup_0['trans_female'] == 1]['actual wait time'], 
                      *serviced_without_warmup_3[serviced_without_warmup_3['trans_male'] == 1]['actual wait time'], 
                      *serviced_without_warmup_2[serviced_without_warmup_2['trans_male'] == 1]['actual wait time'], 
                     *serviced_without_warmup_1[serviced_without_warmup_1['trans_male'] == 1]['actual wait time'],
                     *serviced_without_warmup_0[serviced_without_warmup_0['trans_male'] == 1]['actual wait time'],
                      *serviced_without_warmup_3[serviced_without_warmup_3['nonbinary'] == 1]['actual wait time'], 
                      *serviced_without_warmup_2[serviced_without_warmup_2['nonbinary'] == 1]['actual wait time'], 
                     *serviced_without_warmup_1[serviced_without_warmup_1['nonbinary'] == 1]['actual wait time'],
                     *serviced_without_warmup_0[serviced_without_warmup_0['nonbinary'] == 1]['actual wait time'],
                      *serviced_without_warmup_3[serviced_without_warmup_3['g_queer'] == 1]['actual wait time'], 
                      *serviced_without_warmup_2[serviced_without_warmup_2['g_queer'] == 1]['actual wait time'], 
                     *serviced_without_warmup_1[serviced_without_warmup_1['g_queer'] == 1]['actual wait time'],
                     *serviced_without_warmup_0[serviced_without_warmup_0['g_queer'] == 1]['actual wait time']]
avg_wait_all_noncis = mean(system_noncis_wait)

system_older21_wait = [*serviced_without_warmup_3[serviced_without_warmup_3['age'] > 21]['actual wait time'], 
                      *serviced_without_warmup_2[serviced_without_warmup_2['age'] > 21]['actual wait time'], 
                     *serviced_without_warmup_1[serviced_without_warmup_1['age'] > 21]['actual wait time'],
                     *serviced_without_warmup_0[serviced_without_warmup_0['age'] > 21]['actual wait time']]

avg_wait_all_older21 = mean(system_older21_wait)

system_younger21_wait = [*serviced_without_warmup_3[serviced_without_warmup_3['age'] <= 21]['actual wait time'], 
                      *serviced_without_warmup_2[serviced_without_warmup_2['age'] <= 21]['actual wait time'], 
                     *serviced_without_warmup_1[serviced_without_warmup_1['age'] <= 21]['actual wait time'],
                     *serviced_without_warmup_0[serviced_without_warmup_0['age'] <= 21]['actual wait time']]

avg_wait_all_younger21 = mean(system_younger21_wait)

system_child_wait = [*serviced_without_warmup_3[serviced_without_warmup_3['child'] == 1]['actual wait time'], 
                      *serviced_without_warmup_2[serviced_without_warmup_2['child'] == 1]['actual wait time'], 
                     *serviced_without_warmup_1[serviced_without_warmup_1['child'] == 1]['actual wait time'],
                     *serviced_without_warmup_0[serviced_without_warmup_0['child'] == 1]['actual wait time']]

avg_wait_all_child = mean(system_child_wait)

system_no_child_wait = [*serviced_without_warmup_3[serviced_without_warmup_3['child'] == 0]['actual wait time'], 
                      *serviced_without_warmup_2[serviced_without_warmup_2['child'] == 0]['actual wait time'], 
                     *serviced_without_warmup_1[serviced_without_warmup_1['child'] == 0]['actual wait time'],
                     *serviced_without_warmup_0[serviced_without_warmup_0['child'] == 0]['actual wait time']]

avg_wait_all_no_child = mean(system_no_child_wait)

system_not_immigrant_wait = [*serviced_without_warmup_3[serviced_without_warmup_3['immigrant'] == 0]['actual wait time'], 
                      *serviced_without_warmup_2[serviced_without_warmup_2['immigrant'] == 0]['actual wait time'], 
                     *serviced_without_warmup_1[serviced_without_warmup_1['immigrant'] == 0]['actual wait time'],
                     *serviced_without_warmup_0[serviced_without_warmup_0['immigrant'] == 0]['actual wait time']]

avg_wait_all_not_immigrant = mean(system_not_immigrant_wait)

system_immigrant_wait = [*serviced_without_warmup_3[serviced_without_warmup_3['immigrant'] == 1]['actual wait time'], 
                      *serviced_without_warmup_2[serviced_without_warmup_2['immigrant'] == 1]['actual wait time'], 
                     *serviced_without_warmup_1[serviced_without_warmup_1['immigrant'] == 1]['actual wait time'],
                     *serviced_without_warmup_0[serviced_without_warmup_0['immigrant'] == 1]['actual wait time']]

avg_wait_all_immigrant = mean(system_not_immigrant_wait)

system_ht_wait = [*serviced_without_warmup_3[serviced_without_warmup_3['ht_victim'] == 1]['actual wait time'], 
                      *serviced_without_warmup_2[serviced_without_warmup_2['ht_victim'] == 1]['actual wait time'], 
                     *serviced_without_warmup_1[serviced_without_warmup_1['ht_victim'] == 1]['actual wait time'],
                     *serviced_without_warmup_0[serviced_without_warmup_0['ht_victim'] == 1]['actual wait time']]

avg_wait_all_ht = mean(system_ht_wait)

system_not_ht_wait = [*serviced_without_warmup_3[serviced_without_warmup_3['ht_victim'] == 0]['actual wait time'], 
                      *serviced_without_warmup_2[serviced_without_warmup_2['ht_victim'] == 0]['actual wait time'], 
                     *serviced_without_warmup_1[serviced_without_warmup_1['ht_victim'] == 0]['actual wait time'],
                     *serviced_without_warmup_0[serviced_without_warmup_0['ht_victim'] == 0]['actual wait time']]

avg_wait_all_not_ht = mean(system_not_ht_wait)


__Let's look at among youth who have abandoned what proportion of them identify as female, male, nonbinary or have children, is a HT victim.__

In [209]:
######################
# Among abandonments #
######################

# Shelter 0

if prop_abandoned_0 >0:
    if len(serviced_without_warmup_0[serviced_without_warmup_0[' female'] == 1]) >0:
        prop_abandoned_among_abands_0_female = len(non_serviced_df_0[non_serviced_df_0[' female'] == 1])/ len(non_serviced_df_0)
    else:
        prop_abandoned_among_abands_0_female = 0
        
    if len(serviced_without_warmup_0[serviced_without_warmup_0['male'] == 1]) >0:
        prop_abandoned_among_abands_0_male   = len(non_serviced_df_0[non_serviced_df_0['male'] == 1])/ len(non_serviced_df_0)
    else:
        prop_abandoned_among_abands_0_male   = 0
        
    if (len(serviced_without_warmup_0[serviced_without_warmup_0['g_queer'] == 1])+
                             len(serviced_without_warmup_0[serviced_without_warmup_0['nonbinary'] == 1])+
                            len(serviced_without_warmup_0[serviced_without_warmup_0['trans_female'] == 1])+
                             len(serviced_without_warmup_0[serviced_without_warmup_0['trans_male'] == 1])) >0:
        prop_abandoned_among_abands_0_noncis = (len(non_serviced_df_0[non_serviced_df_0['nonbinary'] == 1])+
                                   len(non_serviced_df_0[non_serviced_df_0['g_queer'] == 1])+
                                   len(non_serviced_df_0[non_serviced_df_0['trans_female'] == 1])+
                                   len(non_serviced_df_0[non_serviced_df_0['trans_male'] == 1]))/ len(non_serviced_df_0)
    else:
        prop_abandoned_among_abands_0_noncis = 0
        
    if len(serviced_without_warmup_0[serviced_without_warmup_0['age'] > 21])>0:
        prop_abandoned_among_abands_0_older21 = len(non_serviced_df_0[non_serviced_df_0['age'] > 21])/ len(non_serviced_df_0)
    else:
        prop_abandoned_among_abands_0_older21 = 0
        
    if len(serviced_without_warmup_0[serviced_without_warmup_0['age'] <= 21])>0:
        prop_abandoned_among_abands_0_younger21 = len(non_serviced_df_0[non_serviced_df_0['age'] <= 21])/ len(non_serviced_df_0)
    else:
        prop_abandoned_among_abands_0_younger21 = 0
        
    if len(serviced_without_warmup_0[serviced_without_warmup_0['child'] == 1]) >0:
        prop_abandoned_among_abands_0_child = len(non_serviced_df_0[non_serviced_df_0['child'] == 1])/ len(non_serviced_df_0)
    else:
        prop_abandoned_among_abands_0_child = 0
        
    if len(serviced_without_warmup_0[serviced_without_warmup_0['child'] == 0]) >0:
        prop_abandoned_among_abands_0_no_child = len(non_serviced_df_0[non_serviced_df_0['child'] == 0])/ len(non_serviced_df_0)
    else:
        prop_abandoned_among_abands_0_no_child = 0
                                                                                           
    if len(serviced_without_warmup_0[serviced_without_warmup_0['immigrant'] == 1])>0:                                                                                  
        prop_abandoned_among_abands_0_immigrant = len(non_serviced_df_0[non_serviced_df_0['immigrant'] == 1])/ len(non_serviced_df_0)
    else:
        prop_abandoned_among_abands_0_immigrant = 0
        
    if len(serviced_without_warmup_0[serviced_without_warmup_0['immigrant'] == 0])>0:                                                                                  
        prop_abandoned_among_abands_0_not_immigrant = len(non_serviced_df_0[non_serviced_df_0['immigrant'] == 0])/ len(non_serviced_df_0)
    else:
        prop_abandoned_among_abands_0_not_immigrant = 0
                                                                                           
    if  len(serviced_without_warmup_0[serviced_without_warmup_0['ht_victim'] == 1])>0:
        prop_abandoned_among_abands_0_ht = len(non_serviced_df_0[non_serviced_df_0['ht_victim'] == 1])/ len(non_serviced_df_0)
    else:
        prop_abandoned_among_abands_0_ht = 0
    
    if  len(serviced_without_warmup_0[serviced_without_warmup_0['ht_victim'] == 0])>0:
        prop_abandoned_among_abands_0_not_ht = len(non_serviced_df_0[non_serviced_df_0['ht_victim'] == 0])/ len(non_serviced_df_0)
    else:
        prop_abandoned_among_abands_0_not_ht = 0
    

# print('----------------')
# print('__  Shelter 0 __')
# print('----------------')
# if prop_abandoned_0 >0:
#     print('-Abandonment')
#     print('Female: {:.4}'.format(float(prop_abandoned_among_abands_0_female*100))+'% of abandonments ')
#     print('Male: {:.4}'.format(float(prop_abandoned_among_abands_0_male*100))+'% of abandonments ')
#     print('Non cis gender: {:.4}'.format(float(prop_abandoned_among_abands_0_noncis*100))+'% of abandonments ')
#     print('Older than 21: {:.4}'.format(float(prop_abandoned_among_abands_0_older21*100))+'% of abandonments ')
#     print('Has child: {:.4}'.format(float(prop_abandoned_among_abands_0_child*100))+'% of abandonments ')
#     print('Immigrant: {:.4}'.format(float(prop_abandoned_among_abands_0_immigrant*100))+'% of abandonments')
#     print('HT Victim: {:.4}'.format(float(prop_abandoned_among_abands_0_ht*100))+'% of abandonments ')
# else: 
#     print('No youth abandoned the system.')
    
# Shelter 1

if prop_abandoned_1 >0:
    if len(serviced_without_warmup_1[serviced_without_warmup_1[' female'] == 1]) >0:
        prop_abandoned_among_abands_1_female = len(non_serviced_df_1[non_serviced_df_1[' female'] == 1])/ len(non_serviced_df_1)
    else:
        prop_abandoned_among_abands_1_female = 0
        
    if len(serviced_without_warmup_1[serviced_without_warmup_1['male'] == 1]) >0:
        prop_abandoned_among_abands_1_male   = len(non_serviced_df_1[non_serviced_df_1['male'] == 1])/ len(non_serviced_df_1)
    else:
        prop_abandoned_among_abands_1_male   = 0
        
    if (len(serviced_without_warmup_1[serviced_without_warmup_1['g_queer'] == 1])+
                             len(serviced_without_warmup_1[serviced_without_warmup_1['nonbinary'] == 1])+
                            len(serviced_without_warmup_1[serviced_without_warmup_1['trans_female'] == 1])+
                             len(serviced_without_warmup_1[serviced_without_warmup_1['trans_male'] == 1])) >0:
        prop_abandoned_among_abands_1_noncis = (len(non_serviced_df_1[non_serviced_df_1['nonbinary'] == 1])+
                                   len(non_serviced_df_1[non_serviced_df_1['g_queer'] == 1])+
                                   len(non_serviced_df_1[non_serviced_df_1['trans_female'] == 1])+
                                   len(non_serviced_df_1[non_serviced_df_1['trans_male'] == 1]))/ len(non_serviced_df_1)
    else:
        prop_abandoned_among_abands_1_noncis = 0
        
    if len(serviced_without_warmup_1[serviced_without_warmup_1['age'] > 21])>0:
        prop_abandoned_among_abands_1_older21 = len(non_serviced_df_1[non_serviced_df_1['age'] > 21])/ len(non_serviced_df_1)
    else:
        prop_abandoned_among_abands_1_older21 = 0
        
    if len(serviced_without_warmup_1[serviced_without_warmup_1['age'] <= 21])>0:
        prop_abandoned_among_abands_1_younger21 = len(non_serviced_df_1[non_serviced_df_1['age'] <= 21])/ len(non_serviced_df_1)
    else:
        prop_abandoned_among_abands_1_younger21 = 0
        
    if len(serviced_without_warmup_1[serviced_without_warmup_1['child'] == 1]) >0:
        prop_abandoned_among_abands_1_child = len(non_serviced_df_1[non_serviced_df_1['child'] == 1])/ len(non_serviced_df_1)
    else:
        prop_abandoned_among_abands_1_child = 0
        
    if len(serviced_without_warmup_1[serviced_without_warmup_1['child'] == 0]) >0:
        prop_abandoned_among_abands_1_no_child = len(non_serviced_df_1[non_serviced_df_1['child'] == 0])/ len(non_serviced_df_1)
    else:
        prop_abandoned_among_abands_1_no_child = 0
                                                                                           
    if len(serviced_without_warmup_1[serviced_without_warmup_1['immigrant'] == 1])>0:                                                                                  
        prop_abandoned_among_abands_1_immigrant = len(non_serviced_df_1[non_serviced_df_1['immigrant'] == 1])/ len(non_serviced_df_1)
    else:
        prop_abandoned_among_abands_1_immigrant = 0
        
    if len(serviced_without_warmup_1[serviced_without_warmup_1['immigrant'] == 0])>0:                                                                                  
        prop_abandoned_among_abands_1_not_immigrant = len(non_serviced_df_1[non_serviced_df_1['immigrant'] == 0])/ len(non_serviced_df_1)
    else:
        prop_abandoned_among_abands_1_not_immigrant = 0
                                                                                           
    if  len(serviced_without_warmup_1[serviced_without_warmup_1['ht_victim'] == 1])>0:
        prop_abandoned_among_abands_1_ht = len(non_serviced_df_1[non_serviced_df_1['ht_victim'] == 1])/ len(non_serviced_df_1)
    else:
        prop_abandoned_among_abands_1_ht = 0      
        
    if  len(serviced_without_warmup_1[serviced_without_warmup_1['ht_victim'] == 0])>0:
        prop_abandoned_among_abands_1_not_ht = len(non_serviced_df_1[non_serviced_df_1['ht_victim'] == 0])/ len(non_serviced_df_1)
    else:
        prop_abandoned_among_abands_1_not_ht = 0 
    

# print('----------------')
# print('__  Shelter 1 __')
# print('----------------')
# if prop_abandoned_1 >0:
#     print('-Abandonment')
#     print('Female: {:.4}'.format(float(prop_abandoned_among_abands_1_female*100))+'% of abandonments ')
#     print('Male: {:.4}'.format(float(prop_abandoned_among_abands_1_male*100))+'% of abandonments ')
#     print('Non cis gender: {:.4}'.format(float(prop_abandoned_among_abands_1_gqueer*100))+'% of abandonments ')
#     print('Older than 21: {:.4}'.format(float(prop_abandoned_among_abands_1_older21*100))+'% of abandonments ')
#     print('Has child: {:.4}'.format(float(prop_abandoned_among_abands_1_child*100))+'% of abandonments ')
#     print('Immigrant: {:.4}'.format(float(prop_abandoned_among_abands_1_immigrant*100))+'% of abandonments')
#     print('HT Victim: {:.4}'.format(float(prop_abandoned_among_abands_1_ht*100))+'% of abandonments ')
# else: 
#     print('No youth abandoned the system.')
    
# Shelter 2

if prop_abandoned_2 >0:
    if len(serviced_without_warmup_2[serviced_without_warmup_2[' female'] == 1]) >0:
        prop_abandoned_among_abands_2_female = len(non_serviced_df_2[non_serviced_df_2[' female'] == 1])/ len(non_serviced_df_2)
    else:
        prop_abandoned_among_abands_2_female = 0
        
    if len(serviced_without_warmup_2[serviced_without_warmup_2['male'] == 1]) >0:
        prop_abandoned_among_abands_2_male   = len(non_serviced_df_2[non_serviced_df_2['male'] == 1])/ len(non_serviced_df_2)
    else:
        prop_abandoned_among_abands_2_male   = 0
        
    if (len(serviced_without_warmup_2[serviced_without_warmup_2['g_queer'] == 1])+
                             len(serviced_without_warmup_2[serviced_without_warmup_2['nonbinary'] == 1])+
                            len(serviced_without_warmup_2[serviced_without_warmup_2['trans_female'] == 1])+
                             len(serviced_without_warmup_2[serviced_without_warmup_2['trans_male'] == 1])) >0:
        prop_abandoned_among_abands_2_noncis = (len(non_serviced_df_2[non_serviced_df_2['nonbinary'] == 1])+
                                   len(non_serviced_df_2[non_serviced_df_2['g_queer'] == 1])+
                                   len(non_serviced_df_2[non_serviced_df_2['trans_female'] == 1])+
                                   len(non_serviced_df_2[non_serviced_df_2['trans_male'] == 1]))/ len(non_serviced_df_2)
    else:
        prop_abandoned_among_abands_2_noncis = 0
        
    if len(serviced_without_warmup_2[serviced_without_warmup_2['age'] > 21])>0:
        prop_abandoned_among_abands_2_older21 = len(non_serviced_df_2[non_serviced_df_2['age'] > 21])/ len(non_serviced_df_2)
    else:
        prop_abandoned_among_abands_2_older21 = 0
        
    if len(serviced_without_warmup_2[serviced_without_warmup_2['age'] <= 21])>0:
        prop_abandoned_among_abands_2_younger21 = len(non_serviced_df_2[non_serviced_df_2['age'] <= 21])/ len(non_serviced_df_2)
    else:
        prop_abandoned_among_abands_2_younger21 = 0
        
    if len(serviced_without_warmup_2[serviced_without_warmup_2['child'] == 1]) >0:
        prop_abandoned_among_abands_2_child = len(non_serviced_df_2[non_serviced_df_2['child'] == 1])/ len(non_serviced_df_2)
    else:
        prop_abandoned_among_abands_2_child = 0
        
    if len(serviced_without_warmup_2[serviced_without_warmup_2['child'] == 0]) >0:
        prop_abandoned_among_abands_2_no_child = len(non_serviced_df_2[non_serviced_df_2['child'] == 0])/ len(non_serviced_df_2)
    else:
        prop_abandoned_among_abands_2_no_child = 0
                                                                                           
    if len(serviced_without_warmup_2[serviced_without_warmup_2['immigrant'] == 1])>0:                                                                                  
        prop_abandoned_among_abands_2_immigrant = len(non_serviced_df_2[non_serviced_df_2['immigrant'] == 1])/ len(non_serviced_df_2)
    else:
        prop_abandoned_among_abands_2_immigrant = 0
        
    if len(serviced_without_warmup_2[serviced_without_warmup_2['immigrant'] == 0])>0:                                                                                  
        prop_abandoned_among_abands_2_not_immigrant = len(non_serviced_df_2[non_serviced_df_2['immigrant'] == 0])/ len(non_serviced_df_2)
    else:
        prop_abandoned_among_abands_2_not_immigrant = 0
                                                                                           
    if  len(serviced_without_warmup_2[serviced_without_warmup_2['ht_victim'] == 1])>0:
        prop_abandoned_among_abands_2_ht = len(non_serviced_df_2[non_serviced_df_2['ht_victim'] == 1])/ len(non_serviced_df_2)
    else:
        prop_abandoned_among_abands_2_ht = 0    
        
    if  len(serviced_without_warmup_2[serviced_without_warmup_2['ht_victim'] == 0])>0:
        prop_abandoned_among_abands_2_not_ht = len(non_serviced_df_2[non_serviced_df_2['ht_victim'] == 0])/ len(non_serviced_df_2)
    else:
        prop_abandoned_among_abands_2_not_ht = 0  
    

# print('----------------')
# print('__  Shelter 2 __')
# print('----------------')
# if prop_abandoned_2 >0:
#     print('-Abandonment')
#     print('Female: {:.4}'.format(float(prop_abandoned_among_abands_2_female*100))+'% of abandonments ')
#     print('Male: {:.4}'.format(float(prop_abandoned_among_abands_2_male*100))+'% of abandonments ')
#     print('Non cis gender: {:.4}'.format(float(prop_abandoned_among_abands_2_noncis*100))+'% of abandonments ')
#     print('Older than 21: {:.4}'.format(float(prop_abandoned_among_abands_2_older21*100))+'% of abandonments ')
#     print('Has child: {:.4}'.format(float(prop_abandoned_among_abands_2_child*100))+'% of abandonments ')
#     print('Immigrant: {:.4}'.format(float(prop_abandoned_among_abands_2_immigrant*100))+'% of abandonments')
#     print('HT Victim: {:.4}'.format(float(prop_abandoned_among_abands_2_ht*100))+'% of abandonments ')
# else: 
#     print('No youth abandoned the system.')
    
# Shelter 3

if prop_abandoned_3 >0:
    if len(serviced_without_warmup_3[serviced_without_warmup_3[' female'] == 1]) >0:
        prop_abandoned_among_abands_3_female = len(non_serviced_df_3[non_serviced_df_3[' female'] == 1])/ len(non_serviced_df_3)
    else:
        prop_abandoned_among_abands_3_female = 0
        
    if len(serviced_without_warmup_3[serviced_without_warmup_3['male'] == 1]) >0:
        prop_abandoned_among_abands_3_male   = len(non_serviced_df_3[non_serviced_df_3['male'] == 1])/ len(non_serviced_df_3)
    else:
        prop_abandoned_among_abands_3_male   = 0
        
    if (len(serviced_without_warmup_3[serviced_without_warmup_3['g_queer'] == 1])+
                             len(serviced_without_warmup_3[serviced_without_warmup_3['nonbinary'] == 1])+
                            len(serviced_without_warmup_3[serviced_without_warmup_3['trans_female'] == 1])+
                             len(serviced_without_warmup_3[serviced_without_warmup_3['trans_male'] == 1])) >0:
        prop_abandoned_among_abands_3_noncis = (len(non_serviced_df_3[non_serviced_df_3['nonbinary'] == 1])+
                                   len(non_serviced_df_3[non_serviced_df_3['g_queer'] == 1])+
                                   len(non_serviced_df_3[non_serviced_df_3['trans_female'] == 1])+
                                   len(non_serviced_df_3[non_serviced_df_3['trans_male'] == 1]))/ len(non_serviced_df_3)
    else:
        prop_abandoned_among_abands_3_noncis = 0
        
    if len(serviced_without_warmup_3[serviced_without_warmup_3['age'] > 21])>0:
        prop_abandoned_among_abands_3_older21 = len(non_serviced_df_3[non_serviced_df_3['age'] > 21])/ len(non_serviced_df_3)
    else:
        prop_abandoned_among_abands_3_older21 = 0
        
    if len(serviced_without_warmup_3[serviced_without_warmup_3['age'] <= 21])>0:
        prop_abandoned_among_abands_3_younger21 = len(non_serviced_df_3[non_serviced_df_3['age'] <= 21])/ len(non_serviced_df_3)
    else:
        prop_abandoned_among_abands_3_younger21 = 0
        
    if len(serviced_without_warmup_3[serviced_without_warmup_3['child'] == 1]) >0:
        prop_abandoned_among_abands_3_child = len(non_serviced_df_3[non_serviced_df_3['child'] == 1])/ len(non_serviced_df_3)
    else:
        prop_abandoned_among_abands_3_child = 0
        
    if len(serviced_without_warmup_3[serviced_without_warmup_3['child'] == 0]) >0:
        prop_abandoned_among_abands_3_no_child = len(non_serviced_df_3[non_serviced_df_3['child'] == 0])/ len(non_serviced_df_3)
    else:
        prop_abandoned_among_abands_3_no_child = 0
                                                                                           
    if len(serviced_without_warmup_3[serviced_without_warmup_3['immigrant'] == 1])>0:                                                                                  
        prop_abandoned_among_abands_3_immigrant = len(non_serviced_df_3[non_serviced_df_3['immigrant'] == 1])/ len(non_serviced_df_3)
    else:
        prop_abandoned_among_abands_3_immigrant = 0
        
    if len(serviced_without_warmup_3[serviced_without_warmup_3['immigrant'] == 0])>0:                                                                                  
        prop_abandoned_among_abands_3_not_immigrant = len(non_serviced_df_3[non_serviced_df_3['immigrant'] == 0])/ len(non_serviced_df_3)
    else:
        prop_abandoned_among_abands_3_not_immigrant = 0
        
    if  len(serviced_without_warmup_3[serviced_without_warmup_3['ht_victim'] == 1])>0:
        prop_abandoned_among_abands_3_ht = len(non_serviced_df_3[non_serviced_df_3['ht_victim'] == 1])/ len(non_serviced_df_3)
    else:
        prop_abandoned_among_abands_3_ht = 0  
                                                                                           
    if  len(serviced_without_warmup_3[serviced_without_warmup_3['ht_victim'] == 0])>0:
        prop_abandoned_among_abands_3_not_ht = len(non_serviced_df_3[non_serviced_df_3['ht_victim'] == 0])/ len(non_serviced_df_3)
    else:
        prop_abandoned_among_abands_3_not_ht = 0        
    
    

# print('----------------')
# print('__  Shelter 3 __')
# print('----------------')
# if prop_abandoned_3 >0:
#     print('-Abandonment')
#     print('Female: {:.4}'.format(float(prop_abandoned_among_abands_3_female*100))+'% of abandonments ')
#     print('Male: {:.4}'.format(float(prop_abandoned_among_abands_3_male*100))+'% of abandonments ')
#     print('Non cis gender: {:.4}'.format(float(prop_abandoned_among_abands_3_noncis*100))+'% of abandonments ')
#     print('Older than 21: {:.4}'.format(float(prop_abandoned_among_abands_3_older21*100))+'% of abandonments ')
#     print('Has child: {:.4}'.format(float(prop_abandoned_among_abands_3_child*100))+'% of abandonments ')
#     print('Immigrant: {:.4}'.format(float(prop_abandoned_among_abands_3_immigrant*100))+'% of abandonments')
#     print('HT Victim: {:.4}'.format(float(prop_abandoned_among_abands_3_ht*100))+'% of abandonments ')
# else: 
#     print('No youth abandoned the system.')


# All shelters

if (len(serviced_without_warmup_0[serviced_without_warmup_0[' female'] == 1]) + len(serviced_without_warmup_1[serviced_without_warmup_1[' female'] == 1])+
    len(serviced_without_warmup_2[serviced_without_warmup_2[' female'] == 1])+ len(serviced_without_warmup_3[serviced_without_warmup_3[' female'] == 1]))>0:
    
    prop_abandoned_among_abands_all_female = (len(non_serviced_df_0[non_serviced_df_0[' female'] == 1]) + 
                                              len(non_serviced_df_1[non_serviced_df_1[' female'] == 1])+
                                                len(non_serviced_df_2[non_serviced_df_2[' female'] == 1])+ 
                                              len(non_serviced_df_3[non_serviced_df_3[' female'] == 1]))/ (len(non_serviced_df_0)+ 
                                                                                                                           len(non_serviced_df_1)+
                                                                                                                           len(non_serviced_df_2)+
                                                                                                                           len(non_serviced_df_3))
                                            
else:
    prop_abandoned_among_abands_all_female = 0

    
if (len(serviced_without_warmup_0[serviced_without_warmup_0['male'] == 1]) + len(serviced_without_warmup_1[serviced_without_warmup_1['male'] == 1])+
    len(serviced_without_warmup_2[serviced_without_warmup_2['male'] == 1])+ len(serviced_without_warmup_3[serviced_without_warmup_3['male'] == 1]))>0:
    
    prop_abandoned_among_abands_all_male   = (len(non_serviced_df_0[non_serviced_df_0['male'] == 1]) + 
                                            len(non_serviced_df_1[non_serviced_df_1['male'] == 1])+
                                            len(non_serviced_df_2[non_serviced_df_2['male'] == 1])+ 
                                            len(non_serviced_df_3[non_serviced_df_3['male'] == 1]))/ (len(non_serviced_df_0)+ 
                                                                                                                           len(non_serviced_df_1)+
                                                                                                                           len(non_serviced_df_2)+
                                                                                                                           len(non_serviced_df_3))
else:
    prop_abandoned_among_abands_all_male   = 0
        
if (len(serviced_without_warmup_3[serviced_without_warmup_3['g_queer'] == 1])+
                            len(serviced_without_warmup_3[serviced_without_warmup_3['nonbinary'] == 1])+
                        len(serviced_without_warmup_3[serviced_without_warmup_3['trans_female'] == 1])+
                            len(serviced_without_warmup_3[serviced_without_warmup_3['trans_male'] == 1])+
   len(serviced_without_warmup_2[serviced_without_warmup_2['g_queer'] == 1])+
                            len(serviced_without_warmup_2[serviced_without_warmup_2['nonbinary'] == 1])+
                        len(serviced_without_warmup_2[serviced_without_warmup_2['trans_female'] == 1])+
                            len(serviced_without_warmup_2[serviced_without_warmup_2['trans_male'] == 1])+
   len(serviced_without_warmup_1[serviced_without_warmup_1['g_queer'] == 1])+
                            len(serviced_without_warmup_1[serviced_without_warmup_1['nonbinary'] == 1])+
                        len(serviced_without_warmup_1[serviced_without_warmup_1['trans_female'] == 1])+
                            len(serviced_without_warmup_1[serviced_without_warmup_1['trans_male'] == 1])+
      len(serviced_without_warmup_0[serviced_without_warmup_0['g_queer'] == 1])+
                            len(serviced_without_warmup_0[serviced_without_warmup_0['nonbinary'] == 1])+
                        len(serviced_without_warmup_0[serviced_without_warmup_0['trans_female'] == 1])+
                            len(serviced_without_warmup_0[serviced_without_warmup_0['trans_male'] == 1])) >0:
    
    
    
    prop_abandoned_among_abands_all_noncis = (len(non_serviced_df_3[non_serviced_df_3['g_queer'] == 1])+
                            len(non_serviced_df_3[non_serviced_df_3['nonbinary'] == 1])+
                        len(non_serviced_df_3[non_serviced_df_3['trans_female'] == 1])+
                            len(non_serviced_df_3[non_serviced_df_3['trans_male'] == 1])+
   len(non_serviced_df_2[non_serviced_df_2['g_queer'] == 1])+
                            len(non_serviced_df_2[non_serviced_df_2['nonbinary'] == 1])+
                        len(non_serviced_df_2[non_serviced_df_2['trans_female'] == 1])+
                            len(non_serviced_df_2[non_serviced_df_2['trans_male'] == 1])+
   len(non_serviced_df_1[non_serviced_df_1['g_queer'] == 1])+
                            len(non_serviced_df_1[non_serviced_df_1['nonbinary'] == 1])+
                        len(non_serviced_df_1[non_serviced_df_1['trans_female'] == 1])+
                            len(non_serviced_df_1[non_serviced_df_1['trans_male'] == 1])+
      len(non_serviced_df_0[non_serviced_df_0['g_queer'] == 1])+
                            len(non_serviced_df_0[non_serviced_df_0['nonbinary'] == 1])+
                        len(non_serviced_df_0[non_serviced_df_0['trans_female'] == 1])+
                            len(non_serviced_df_0[non_serviced_df_0['trans_male'] == 1]))/ (len(non_serviced_df_0)+ 
                                                                                                            len(non_serviced_df_1)+
                                                                                                            len(non_serviced_df_2)+
                                                                                                            len(non_serviced_df_3))
else:
    prop_abandoned_among_abands_all_noncis = 0
        
if (len(serviced_without_warmup_3[serviced_without_warmup_3['age'] > 21]) + len(serviced_without_warmup_2[serviced_without_warmup_2['age'] > 21])+
    len(serviced_without_warmup_1[serviced_without_warmup_1['age'] > 21]) +len(serviced_without_warmup_0[serviced_without_warmup_0['age'] > 21]))>0:
    
    prop_abandoned_among_abands_all_older21 = (len(non_serviced_df_3[non_serviced_df_3['age'] > 21]) + 
                                               len(non_serviced_df_2[non_serviced_df_2['age'] > 21])+
                                                len(non_serviced_df_1[non_serviced_df_1['age'] > 21]) +
                                               len(non_serviced_df_0[non_serviced_df_0['age'] > 21]))/ (len(non_serviced_df_0)+ 
                                                                                                                        len(non_serviced_df_1)+
                                                                                                                        len(non_serviced_df_2)+
                                                                                                                        len(non_serviced_df_3))
else:
    prop_abandoned_among_abands_all_older21 = 0
    
if (len(serviced_without_warmup_3[serviced_without_warmup_3['age'] <= 21]) + len(serviced_without_warmup_2[serviced_without_warmup_2['age'] <= 21])+
    len(serviced_without_warmup_1[serviced_without_warmup_1['age'] <= 21]) +len(serviced_without_warmup_0[serviced_without_warmup_0['age'] <= 21]))>0:
    
    prop_abandoned_among_abands_all_younger21 = (len(non_serviced_df_3[non_serviced_df_3['age'] <= 21]) + 
                                               len(non_serviced_df_2[non_serviced_df_2['age'] <= 21])+
                                                len(non_serviced_df_1[non_serviced_df_1['age'] <= 21]) +
                                               len(non_serviced_df_0[non_serviced_df_0['age'] <= 21]))/ (len(non_serviced_df_0)+ 
                                                                                                                        len(non_serviced_df_1)+
                                                                                                                        len(non_serviced_df_2)+
                                                                                                                        len(non_serviced_df_3))
else:
    prop_abandoned_among_abands_all_younger21 = 0
    
    
        
if (len(serviced_without_warmup_3[serviced_without_warmup_3['child'] == 1]) +
    len(serviced_without_warmup_2[serviced_without_warmup_2['child'] == 1]) +
    len(serviced_without_warmup_1[serviced_without_warmup_1['child'] == 1])+ 
    len(serviced_without_warmup_0[serviced_without_warmup_0['child'] == 1]))>0:
    
    prop_abandoned_among_abands_all_child = (len(non_serviced_df_3[non_serviced_df_3['child'] == 1]) +
    len(non_serviced_df_2[non_serviced_df_2['child'] == 1]) +
    len(non_serviced_df_1[non_serviced_df_1['child'] == 1])+ 
    len(non_serviced_df_0[non_serviced_df_0['child'] == 1]))/ (len(non_serviced_df_0)+ 
                                                                               len(non_serviced_df_1)+
                                                                               len(non_serviced_df_2)+
                                                                               len(non_serviced_df_3))
else:
    prop_abandoned_among_abands_all_child = 0
    
if (len(serviced_without_warmup_3[serviced_without_warmup_3['child'] == 0]) +
    len(serviced_without_warmup_2[serviced_without_warmup_2['child'] == 0]) +
    len(serviced_without_warmup_1[serviced_without_warmup_1['child'] == 0])+ 
    len(serviced_without_warmup_0[serviced_without_warmup_0['child'] == 0]))>0:
    
    prop_abandoned_among_abands_all_no_child = (len(non_serviced_df_3[non_serviced_df_3['child'] == 0]) +
    len(non_serviced_df_2[non_serviced_df_2['child'] == 0]) +
    len(non_serviced_df_1[non_serviced_df_1['child'] == 0])+ 
    len(non_serviced_df_0[non_serviced_df_0['child'] == 0]))/ (len(non_serviced_df_0)+ 
                                                                               len(non_serviced_df_1)+
                                                                               len(non_serviced_df_2)+
                                                                               len(non_serviced_df_3))
else:
    prop_abandoned_among_abands_all_no_child = 0
    
                                                                                           
if (len(serviced_without_warmup_3[serviced_without_warmup_3['immigrant'] == 1]) +
    len(serviced_without_warmup_2[serviced_without_warmup_2['immigrant'] == 1])+
    len(serviced_without_warmup_1[serviced_without_warmup_1['immigrant'] == 1])+
    len(serviced_without_warmup_0[serviced_without_warmup_0['immigrant'] == 1]))>0:                                                                                  
    prop_abandoned_among_abands_all_immigrant = (len(non_serviced_df_3[non_serviced_df_3['immigrant'] == 1]) +
                                                len(non_serviced_df_2[non_serviced_df_2['immigrant'] == 1])+
                                                len(non_serviced_df_1[non_serviced_df_1['immigrant'] == 1])+
                                                len(non_serviced_df_0[non_serviced_df_0['immigrant'] == 1]))/ (len(non_serviced_df_0)+ 
                                                                                                                               len(non_serviced_df_1)+
                                                                                                                               len(non_serviced_df_2)+
                                                                                                                               len(non_serviced_df_3))

else:
    prop_abandoned_among_abands_all_immigrant = 0
    
if (len(serviced_without_warmup_3[serviced_without_warmup_3['immigrant'] == 0]) +
    len(serviced_without_warmup_2[serviced_without_warmup_2['immigrant'] == 0])+
    len(serviced_without_warmup_1[serviced_without_warmup_1['immigrant'] == 0])+
    len(serviced_without_warmup_0[serviced_without_warmup_0['immigrant'] == 0]))>0:                                                                                  
    prop_abandoned_among_abands_all_not_immigrant = (len(non_serviced_df_3[non_serviced_df_3['immigrant'] == 0]) +
                                                len(non_serviced_df_2[non_serviced_df_2['immigrant'] == 0])+
                                                len(non_serviced_df_1[non_serviced_df_1['immigrant'] == 0])+
                                                len(non_serviced_df_0[non_serviced_df_0['immigrant'] == 0]))/ (len(non_serviced_df_0)+ 
                                                                                                                               len(non_serviced_df_1)+
                                                                                                                               len(non_serviced_df_2)+
                                                                                                                               len(non_serviced_df_3))

else:
    prop_abandoned_among_abands_all_not_immigrant = 0
                                                                                           
if  (len(serviced_without_warmup_3[serviced_without_warmup_3['ht_victim'] == 1])+
    len(serviced_without_warmup_2[serviced_without_warmup_2['ht_victim'] == 1])+
    len(serviced_without_warmup_1[serviced_without_warmup_1['ht_victim'] == 1])+
    len(serviced_without_warmup_0[serviced_without_warmup_0['ht_victim'] == 1]))>0:
    prop_abandoned_among_abands_all_ht = (len(non_serviced_df_3[non_serviced_df_3['ht_victim'] == 1])+
                                        len(non_serviced_df_2[non_serviced_df_2['ht_victim'] == 1])+
                                        len(non_serviced_df_1[non_serviced_df_1['ht_victim'] == 1])+
                                        len(non_serviced_df_0[non_serviced_df_0['ht_victim'] == 1]))/ (len(non_serviced_df_0)+ 
                                                                                                                        len(non_serviced_df_1)+
                                                                                                                        len(non_serviced_df_2)+
                                                                                                                        len(non_serviced_df_3))
else:
    prop_abandoned_among_abands_all_ht = 0
    
if  (len(serviced_without_warmup_3[serviced_without_warmup_3['ht_victim'] == 0])+
    len(serviced_without_warmup_2[serviced_without_warmup_2['ht_victim'] == 0])+
    len(serviced_without_warmup_1[serviced_without_warmup_1['ht_victim'] == 0])+
    len(serviced_without_warmup_0[serviced_without_warmup_0['ht_victim'] == 0]))>0:
    prop_abandoned_among_abands_all_not_ht = (len(non_serviced_df_3[non_serviced_df_3['ht_victim'] == 0])+
                                        len(non_serviced_df_2[non_serviced_df_2['ht_victim'] == 0])+
                                        len(non_serviced_df_1[non_serviced_df_1['ht_victim'] == 0])+
                                        len(non_serviced_df_0[non_serviced_df_0['ht_victim'] == 0]))/ (len(non_serviced_df_0)+ 
                                                                                                                        len(non_serviced_df_1)+
                                                                                                                        len(non_serviced_df_2)+
                                                                                                                        len(non_serviced_df_3))
else:
    prop_abandoned_among_abands_all_not_ht = 0

In [210]:
#########################################
# Tables
#########################################

#Summary table of number of youth arriving to shelters who are from different demographic groups
number_arriving_list_0 = [number_arriving_0_female, number_arriving_0_male, number_arriving_0_noncis, 
                          number_arriving_0_older21, number_arriving_0_younger21 , number_arriving_0_child, 
                          number_arriving_0_no_child , number_arriving_0_immigrant, number_arriving_0_not_immigrant ,
                          number_arriving_0_ht, number_arriving_0_not_ht]
number_arriving_list_1 = [number_arriving_1_female, number_arriving_1_male, number_arriving_1_noncis, 
                          number_arriving_1_older21, number_arriving_1_younger21 , number_arriving_1_child, 
                          number_arriving_1_no_child , number_arriving_1_immigrant, number_arriving_1_not_immigrant ,
                          number_arriving_1_ht, number_arriving_1_not_ht]
number_arriving_list_2 = [number_arriving_2_female, number_arriving_2_male, number_arriving_2_noncis, 
                          number_arriving_2_older21, number_arriving_2_younger21 , number_arriving_2_child, 
                          number_arriving_2_no_child , number_arriving_2_immigrant, number_arriving_2_not_immigrant ,
                          number_arriving_2_ht, number_arriving_2_not_ht]
number_arriving_list_3 = [number_arriving_3_female, number_arriving_3_male, number_arriving_3_noncis, 
                          number_arriving_3_older21, number_arriving_3_younger21 , number_arriving_3_child, 
                          number_arriving_3_no_child , number_arriving_3_immigrant, number_arriving_3_not_immigrant ,
                          number_arriving_3_ht, number_arriving_3_not_ht]
number_arriving_list_all = [number_arriving_all_female, number_arriving_all_male, number_arriving_all_noncis, 
                          number_arriving_all_older21, number_arriving_all_younger21 , number_arriving_all_child, 
                          number_arriving_all_no_child , number_arriving_all_immigrant, number_arriving_all_not_immigrant ,
                          number_arriving_all_ht, number_arriving_all_not_ht]

dict_number_arriving = {'Number of youth arriving:' : ['Females','Males','Non cisgenders','Older 21','Younger 21','With child',
                                             'No child','Immigrants','Non-Immigrants','HT Victims', 'Non-HT Victims'],
        'Shelter 0' : number_arriving_list_0,
        'Shelter 1' : number_arriving_list_1,
        'Shelter 2' : number_arriving_list_2,
        'Shelter 3' : number_arriving_list_3,
        'All Shelters': number_arriving_list_all       
       }
df_number_arriving = pd.DataFrame(dict_number_arriving)
  
# displaying the DataFrame
display(df_number_arriving)

#Abandonment summary tables for different demographic groups

if prop_abandoned_0 >0:
    prop_abandoned_list_0 = [prop_abandoned_0_female, prop_abandoned_0_male, 
                             prop_abandoned_0_noncis, prop_abandoned_0_older21, 
                             prop_abandoned_0_younger21, prop_abandoned_0_child, 
                             prop_abandoned_0_no_child, prop_abandoned_0_immigrant, 
                             prop_abandoned_0_not_immigrant, prop_abandoned_0_ht, prop_abandoned_0_not_ht]
    
    prop_abandoned_among_abans_list_0 = [prop_abandoned_among_abands_0_female, prop_abandoned_among_abands_0_male, 
                                         prop_abandoned_among_abands_0_noncis, prop_abandoned_among_abands_0_older21, 
                                         prop_abandoned_among_abands_0_younger21, prop_abandoned_among_abands_0_child, 
                                         prop_abandoned_among_abands_0_no_child, prop_abandoned_among_abands_0_immigrant, 
                                         prop_abandoned_among_abands_0_not_immigrant, prop_abandoned_among_abands_0_ht, 
                                         prop_abandoned_among_abands_0_not_ht]
else:
    prop_abandoned_list_0 = [0,0,0,0,0,0,0,0,0,0,0]
    prop_abandoned_among_abans_list_0 = [0,0,0,0,0,0,0,0,0,0,0]
    
if prop_abandoned_1 >0:
    prop_abandoned_list_1 = [prop_abandoned_1_female, prop_abandoned_1_male, 
                             prop_abandoned_1_noncis, prop_abandoned_1_older21, 
                             prop_abandoned_1_younger21, prop_abandoned_1_child, 
                             prop_abandoned_1_no_child, prop_abandoned_1_immigrant, 
                             prop_abandoned_1_not_immigrant, prop_abandoned_1_ht, prop_abandoned_1_not_ht]
    
    prop_abandoned_among_abans_list_1 = [prop_abandoned_among_abands_1_female, prop_abandoned_among_abands_1_male, 
                                         prop_abandoned_among_abands_1_noncis, prop_abandoned_among_abands_1_older21, 
                                         prop_abandoned_among_abands_1_younger21, prop_abandoned_among_abands_1_child, 
                                         prop_abandoned_among_abands_1_no_child, prop_abandoned_among_abands_1_immigrant, 
                                         prop_abandoned_among_abands_1_not_immigrant, prop_abandoned_among_abands_1_ht, 
                                         prop_abandoned_among_abands_1_not_ht]
else:
    prop_abandoned_list_1 = [0,0,0,0,0,0,0,0,0,0,0]
    prop_abandoned_among_abans_list_1 = [0,0,0,0,0,0,0,0,0,0,0]
    
if prop_abandoned_2 >0:
    prop_abandoned_list_2 = [prop_abandoned_2_female, prop_abandoned_2_male, 
                             prop_abandoned_2_noncis, prop_abandoned_2_older21, 
                             prop_abandoned_2_younger21, prop_abandoned_2_child, 
                             prop_abandoned_2_no_child, prop_abandoned_2_immigrant, 
                             prop_abandoned_2_not_immigrant, prop_abandoned_2_ht, prop_abandoned_2_not_ht]
    
    prop_abandoned_among_abans_list_2 = [prop_abandoned_among_abands_2_female, prop_abandoned_among_abands_2_male, 
                                         prop_abandoned_among_abands_2_noncis, prop_abandoned_among_abands_2_older21, 
                                         prop_abandoned_among_abands_2_younger21, prop_abandoned_among_abands_2_child, 
                                         prop_abandoned_among_abands_2_no_child, prop_abandoned_among_abands_2_immigrant, 
                                         prop_abandoned_among_abands_2_not_immigrant, prop_abandoned_among_abands_2_ht, 
                                         prop_abandoned_among_abands_2_not_ht]
else:
    prop_abandoned_list_2 = [0,0,0,0,0,0,0,0,0,0,0]
    prop_abandoned_among_abans_list_2 = [0,0,0,0,0,0,0,0,0,0,0]
    
if prop_abandoned_3 >0:
    prop_abandoned_list_3 = [prop_abandoned_3_female, prop_abandoned_3_male, 
                             prop_abandoned_3_noncis, prop_abandoned_3_older21, 
                             prop_abandoned_3_younger21, prop_abandoned_3_child, 
                             prop_abandoned_3_no_child, prop_abandoned_3_immigrant, 
                             prop_abandoned_3_not_immigrant, prop_abandoned_3_ht, prop_abandoned_3_not_ht]
    prop_abandoned_among_abans_list_3 = [prop_abandoned_among_abands_3_female, prop_abandoned_among_abands_3_male, 
                                         prop_abandoned_among_abands_3_noncis, prop_abandoned_among_abands_3_older21, 
                                         prop_abandoned_among_abands_3_younger21, prop_abandoned_among_abands_3_child, 
                                         prop_abandoned_among_abands_3_no_child, prop_abandoned_among_abands_3_immigrant, 
                                         prop_abandoned_among_abands_3_not_immigrant, prop_abandoned_among_abands_3_ht, 
                                         prop_abandoned_among_abands_3_not_ht]
else:
    prop_abandoned_list_3 = [0,0,0,0,0,0,0,0,0,0,0]
    prop_abandoned_among_abans_list_3 = [0,0,0,0,0,0,0,0,0,0,0]
    
prop_abandoned_among_abans_list_all = [prop_abandoned_among_abands_all_female, prop_abandoned_among_abands_all_male, 
                                         prop_abandoned_among_abands_all_noncis, prop_abandoned_among_abands_all_older21, 
                                         prop_abandoned_among_abands_all_younger21, prop_abandoned_among_abands_all_child, 
                                         prop_abandoned_among_abands_all_no_child, prop_abandoned_among_abands_all_immigrant, 
                                         prop_abandoned_among_abands_all_not_immigrant, prop_abandoned_among_abands_all_ht, 
                                         prop_abandoned_among_abands_all_not_ht]


prop_abandoned_list_all = [prop_abandoned_all_female, prop_abandoned_all_male, 
                             prop_abandoned_all_noncis, prop_abandoned_all_older21, 
                             prop_abandoned_all_younger21, prop_abandoned_all_child, 
                             prop_abandoned_all_no_child, prop_abandoned_all_immigrant, 
                             prop_abandoned_all_not_immigrant, prop_abandoned_all_ht, prop_abandoned_all_not_ht]    

dict_prop_aban = {'% abandonment among their demographic groups:' : ['Females','Males','Non cisgenders','Older 21','Younger 21','With child',
                                             'No child','Immigrants','Non-Immigrants','HT Victims','Non-HT Victims'],
        'Shelter 0' : prop_abandoned_list_0,
        'Shelter 1' : prop_abandoned_list_1,
        'Shelter 2' : prop_abandoned_list_2,
        'Shelter 3' : prop_abandoned_list_3,
        'All Shelters': prop_abandoned_list_all       
       }
df_among_demographics = pd.DataFrame(dict_prop_aban)
  
# displaying the DataFrame
display(df_among_demographics)


#Average Wait summary tables for different demographic groups

average_wait_list_0 = [avg_wait_0_female,  avg_wait_0_male, avg_wait_0_noncis, avg_wait_0_older21, avg_wait_0_younger21, 
                       avg_wait_0_child, avg_wait_0_no_child, avg_wait_0_immigrant, avg_wait_0_not_immigrant, avg_wait_0_ht, 
                       avg_wait_0_not_ht]
average_wait_list_1 = [avg_wait_1_female,  avg_wait_1_male, avg_wait_1_noncis, avg_wait_1_older21, avg_wait_1_younger21, 
                       avg_wait_1_child, avg_wait_1_no_child, avg_wait_1_immigrant, avg_wait_1_not_immigrant, avg_wait_1_ht, 
                       avg_wait_1_not_ht]
average_wait_list_2 = [avg_wait_2_female,  avg_wait_2_male, avg_wait_2_noncis, avg_wait_2_older21, avg_wait_2_younger21, 
                       avg_wait_2_child, avg_wait_2_no_child, avg_wait_2_immigrant, avg_wait_2_not_immigrant, avg_wait_2_ht, 
                       avg_wait_2_not_ht]
average_wait_list_3 = [avg_wait_3_female,  avg_wait_3_male, avg_wait_3_noncis, avg_wait_3_older21, avg_wait_3_younger21, 
                       avg_wait_3_child, avg_wait_3_no_child, avg_wait_3_immigrant, avg_wait_3_not_immigrant, avg_wait_3_ht, 
                       avg_wait_3_not_ht]

average_wait_list_all = [avg_wait_all_female, avg_wait_all_male, avg_wait_all_noncis, avg_wait_all_older21, 
                         avg_wait_all_younger21, avg_wait_all_child, avg_wait_all_no_child, avg_wait_all_not_immigrant , 
                         avg_wait_all_immigrant, avg_wait_all_ht, avg_wait_all_not_ht]

dict_average_wait = {'Average Wait Time:' : ['Females','Males','Non cisgenders','Older 21','Younger 21','With child',
                                             'No child','Immigrants','Non-Immigrants','HT Victims','Non-HT Victims' ],
        'Shelter 0' : average_wait_list_0,
        'Shelter 1' : average_wait_list_1,
        'Shelter 2' : average_wait_list_2,
        'Shelter 3' : average_wait_list_3,
        'All' : average_wait_list_all
       }
df_average_wait = pd.DataFrame(dict_average_wait)
  
# displaying the DataFrame
display(df_average_wait)


#Abandonment summary tables for different demographic groups


dict_prop_among_abandonment = {'% abandonment among total abandonments:' : ['Females','Males','Non cisgenders','Older 21','Younger 21','With child',
                                             'No child','Immigrants','Non-Immigrants', 'Non-HT Victims','HT Victims'],
        'Shelter 0' : prop_abandoned_among_abans_list_0,
        'Shelter 1' : prop_abandoned_among_abans_list_1,
        'Shelter 2' : prop_abandoned_among_abans_list_2,
        'Shelter 3' : prop_abandoned_among_abans_list_3,
        'All Shelters': prop_abandoned_among_abans_list_all
       }
df_among_abandonments = pd.DataFrame(dict_prop_among_abandonment)
  
# displaying the DataFrame
display(df_among_abandonments)

# EXTREMELY useful command to copy any data frame as a table "df.to_clipboard()" 
# then you can just paste the table into ppt or excel 

,Number of youth arriving:,Shelter 0,Shelter 1,Shelter 2,Shelter 3,All Shelters
0,Females,0,462,173,229,864
1,Males,0,415,153,198,766
2,Non cisgenders,46,262,101,95,504
3,Older 21,22,0,0,198,220
4,Younger 21,24,1139,427,324,1914
5,With child,0,62,18,23,103
6,No child,46,1077,409,499,2031
7,Immigrants,16,216,55,0,287
8,Non-Immigrants,30,923,372,522,1847
9,HT Victims,5,237,80,113,435


,% abandonment among their demographic groups:,Shelter 0,Shelter 1,Shelter 2,Shelter 3,All Shelters
0,Females,0,0.132035,0.693642,0.689956,0.392361
1,Males,0,0.113253,0.653595,0.707071,0.374674
2,Non cisgenders,0,0.099237,0.613861,0.736842,0.313492
3,Older 21,0,0.000000,0.000000,0.702020,0.631818
4,Younger 21,0,0.117647,0.660422,0.706790,0.336991
5,With child,0,0.080645,0.666667,0.652174,0.310680
6,No child,0,0.119777,0.660147,0.707415,0.370261
7,Immigrants,0,0.129630,0.581818,0.000000,0.209059
8,Non-Immigrants,0,0.114843,0.672043,0.704981,0.391987
9,HT Victims,0,0.143460,0.675000,0.672566,0.377011


,Average Wait Time:,Shelter 0,Shelter 1,Shelter 2,Shelter 3,All
0,Females,NaN,0.481684,5.072540,4.368408,1.470775
1,Males,NaN,0.461969,4.302468,5.087379,1.446980
2,Non cisgenders,0.0,0.451754,4.915118,5.101977,1.221722
3,Older 21,0.0,NaN,NaN,4.415751,3.216411
4,Younger 21,0.0,0.467591,4.733506,4.959570,1.282464
5,With child,NaN,0.314381,3.980017,4.858294,1.136143
6,No child,0.0,0.476803,4.766030,4.745357,1.413065
7,Immigrants,0.0,0.516934,3.868595,NaN,1.515418
8,Non-Immigrants,0.0,0.456237,4.896562,4.751223,1.515418
9,HT Victims,0.0,0.558251,5.788377,4.614077,1.603482


,% abandonment among total abandonments:,Shelter 0,Shelter 1,Shelter 2,Shelter 3,All Shelters
0,Females,0,0.455224,0.425532,0.429348,0.432398
1,Males,0,0.350746,0.354610,0.380435,0.366071
2,Non cisgenders,0,0.194030,0.219858,0.190217,0.201531
3,Older 21,0,0.000000,0.000000,0.377717,0.177296
4,Younger 21,0,1.000000,1.000000,0.622283,0.822704
5,With child,0,0.037313,0.042553,0.040761,0.040816
6,No child,0,0.962687,0.957447,0.959239,0.959184
7,Immigrants,0,0.208955,0.113475,0.000000,0.076531
8,Non-Immigrants,0,0.791045,0.886525,1.000000,0.923469
9,Non-HT Victims,0,0.253731,0.191489,0.206522,0.209184


In [211]:
# This creates the original excel file and will store the data frames we created
#Additional sheets will be added on top of it to store the replication summaries

# writer = pd.ExcelWriter('MNSOutputs.xlsx',engine='xlsxwriter')   
# workbook=writer.book
# worksheet=workbook.add_worksheet('1')
# writer.sheets['1'] = worksheet
# df_overall_summary.to_excel(writer,sheet_name='1',startrow=1 , startcol=1) 
# df_number_arriving.to_excel(writer,sheet_name='1',startrow=8 , startcol=1)  
# df_among_demographics.to_excel(writer,sheet_name='1',startrow=21 , startcol=1)  
# df_among_abandonments.to_excel(writer,sheet_name='1',startrow=34 , startcol=1)   
# df_average_wait.to_excel(writer,sheet_name='1',startrow=47, startcol=1) 

# writer.save()

# print('=====================================================')
# print('________________ Outputs are ready___________________')
# print('=====================================================')


In [212]:
#This is basically to add other sheets on top of an existing excel file
#Will be used for the replications

ExcelWorkbook = load_workbook('MNSOutputs.xlsx')

# Generating the writer engine
writer = pd.ExcelWriter('MNSOutputs.xlsx', engine = 'openpyxl')
 
# Assigning the workbook to the writer engine
writer.book = ExcelWorkbook
 
# Adding the DataFrames to the excel as a new sheet
df_overall_summary.to_excel(writer,sheet_name='10',startrow=1 , startcol=1) 
df_number_arriving.to_excel(writer,sheet_name='10',startrow=8 , startcol=1)  
df_among_demographics.to_excel(writer,sheet_name='10',startrow=21 , startcol=1)  
df_among_abandonments.to_excel(writer,sheet_name='10',startrow=34 , startcol=1)   
df_average_wait.to_excel(writer,sheet_name='10',startrow=47, startcol=1) 
writer.save()
writer.close()

print('=====================================================')
print('________________ Outputs are ready___________________')
print('=====================================================')

________________ Outputs are ready___________________
